In [18]:
# Instalación de librerías actualizadas (Diciembre 2024)
!pip install --upgrade pip
!pip install pandas==2.1.4
!pip install numpy==1.26.2
!pip install requests==2.31.0
!pip install psycopg2-binary==2.9.9
!pip install sqlalchemy==2.0.23
!pip install plotly==5.18.0
!pip install dash==2.14.2
!pip install dash-bootstrap-components==1.5.0
!pip install scikit-learn==1.3.2
!pip install prophet==1.1.5
!pip install folium==0.15.1
!pip install matplotlib==3.8.2
!pip install seaborn==0.13.0
!pip install python-dotenv==1.0.0
!pip install schedule==1.2.0
!pip install openmeteo-requests==1.1.0
!pip install requests-cache==1.1.1
!pip install retry-requests==2.0.0

print("✅ Todas las dependencias instaladas correctamente")

  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
D:\Miguel\Anaconda_AIEP\python.exe -m pip install --upgrade pip


✅ Todas las dependencias instaladas correctamente


In [1]:
# SOLUCIÓN PARA EL ERROR DE IMPORTACIÓN

# Paso 1: Desinstalar las versiones conflictivas
import subprocess
import sys

print("🔧 Reparando dependencias...")

# Desinstalar paquetes problemáticos
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "requests-cache"])
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "cattrs"])
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "attrs"])
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "typing-extensions"])

print("✅ Paquetes conflictivos removidos")

# Paso 2: Reinstalar con versiones compatibles
subprocess.run([sys.executable, "-m", "pip", "install", "typing-extensions==4.8.0"])
subprocess.run([sys.executable, "-m", "pip", "install", "attrs==23.1.0"])
subprocess.run([sys.executable, "-m", "pip", "install", "cattrs==23.1.2"])

print("✅ Dependencias base instaladas")

# Paso 3: Verificar importaciones básicas
try:
    import pandas as pd
    import numpy as np
    import requests
    import plotly.graph_objects as go
    from dash import Dash
    import dash_bootstrap_components as dbc
    from prophet import Prophet
    from sqlalchemy import create_engine
    
    print("✅ Todas las importaciones básicas funcionan correctamente")
except ImportError as e:
    print(f"❌ Error en importación: {e}")

print("\n🎉 Reparación completada!")

🔧 Reparando dependencias...
✅ Paquetes conflictivos removidos
✅ Dependencias base instaladas
✅ Todas las importaciones básicas funcionan correctamente

🎉 Reparación completada!


In [2]:
# SOLUCIÓN: Instalar los módulos faltantes

import subprocess
import sys

print("📦 Instalando módulos faltantes...")

# Instalar requests-cache y retry-requests
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "requests-cache"])
    print("✅ requests-cache instalado")
except:
    print("❌ Error instalando requests-cache")

try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "retry-requests"])
    print("✅ retry-requests instalado")
except:
    # Si falla con la versión específica, instalar la última disponible
    subprocess.check_call([sys.executable, "-m", "pip", "install", "retry-requests==2.0.0"])
    print("✅ retry-requests instalado (versión 2.0.0)")

print("\n✅ Instalación completada")

📦 Instalando módulos faltantes...
✅ requests-cache instalado
✅ retry-requests instalado

✅ Instalación completada


In [3]:
import os
import sys
import json
import warnings
from datetime import datetime, timedelta, timezone
import pandas as pd
import numpy as np
import requests
import openmeteo_requests
import requests_cache
from retry_requests import retry

# Visualización
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Dashboard
import dash
from dash import dcc, html, Input, Output, State, callback
import dash_bootstrap_components as dbc

# Base de datos
import psycopg2
from sqlalchemy import create_engine, text, MetaData, Table, Column, Integer, String, Float, DateTime
from sqlalchemy.orm import sessionmaker

# Machine Learning
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Utilidades
import folium
import schedule
import time
from dotenv import load_dotenv

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')

# Cargar variables de entorno
load_dotenv()

print("📊 Sistema de Pronóstico Meteorológico MIP Quillota")
print("="*60)
print(f"📅 Fecha de inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🐍 Python version: {sys.version.split()[0]}")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📈 Plotly version: {plotly.__version__}")

📊 Sistema de Pronóstico Meteorológico MIP Quillota
📅 Fecha de inicio: 2025-07-24 13:51:47
🐍 Python version: 3.12.7
📦 Pandas version: 2.1.4
📈 Plotly version: 5.18.0


In [4]:
# Configuración de la zona de Quillota
QUILLOTA_CONFIG = {
    "centro": {
        "lat": -32.8833,
        "lon": -71.2489,
        "nombre": "Quillota Centro",
        "altitud": 130
    },
    "estaciones": [
        {"lat": -32.8833, "lon": -71.2489, "nombre": "Quillota Centro", "id": "QLL01"},
        {"lat": -32.9167, "lon": -71.2667, "nombre": "La Cruz", "id": "LCZ01"},
        {"lat": -32.8500, "lon": -71.2333, "nombre": "La Calera", "id": "LCL01"},
        {"lat": -32.7833, "lon": -71.2167, "nombre": "Hijuelas", "id": "HJL01"},
        {"lat": -32.9500, "lon": -71.2833, "nombre": "Limache", "id": "LMC01"},
        {"lat": -32.8000, "lon": -71.2000, "nombre": "Nogales", "id": "NGL01"}
    ],
    "timezone": "America/Santiago"
}

# Configuración detallada de cultivos
CULTIVOS_CONFIG = {
    "palta": {
        "nombre_cientifico": "Persea americana",
        "variedades": ["Hass", "Fuerte", "Bacon", "Zutano"],
        "temp_min": 10,
        "temp_max": 30,
        "temp_optima": 20,
        "temp_critica_min": -1,
        "temp_critica_max": 40,
        "humedad_min": 60,
        "humedad_max": 80,
        "humedad_optima": 70,
        "precipitacion_anual": 1200,
        "ph_suelo": [6.0, 7.0],
        "helada_critica": -1,
        "meses_siembra": [3, 4, 5, 9, 10],
        "meses_floracion": [8, 9, 10],
        "meses_cosecha": [3, 4, 5, 6, 7, 8],
        "riego_mm_dia": 4.5,
        "grados_dia_desarrollo": 2800
    },
    "citricos": {
        "nombre_cientifico": "Citrus spp.",
        "variedades": ["Naranja", "Limón", "Mandarina", "Pomelo"],
        "temp_min": 13,
        "temp_max": 35,
        "temp_optima": 25,
        "temp_critica_min": -2,
        "temp_critica_max": 42,
        "humedad_min": 50,
        "humedad_max": 70,
        "humedad_optima": 60,
        "precipitacion_anual": 1000,
        "ph_suelo": [5.5, 6.5],
        "helada_critica": -2,
        "meses_siembra": [3, 4, 5, 8, 9],
        "meses_floracion": [8, 9],
        "meses_cosecha": [4, 5, 6, 7, 8],
        "riego_mm_dia": 3.8,
        "grados_dia_desarrollo": 2500
    },
    "tomate": {
        "nombre_cientifico": "Solanum lycopersicum",
        "variedades": ["Raf", "Cherry", "Pera", "Beef"],
        "temp_min": 15,
        "temp_max": 30,
        "temp_optima": 22,
        "temp_critica_min": 2,
        "temp_critica_max": 35,
        "humedad_min": 55,
        "humedad_max": 75,
        "humedad_optima": 65,
        "precipitacion_anual": 600,
        "ph_suelo": [6.0, 6.8],
        "helada_critica": 2,
        "meses_siembra": [8, 9, 10, 2, 3],
        "meses_floracion": [10, 11, 12, 1],
        "meses_cosecha": [12, 1, 2, 3, 4],
        "riego_mm_dia": 5.2,
        "grados_dia_desarrollo": 1200
    },
    "uva_mesa": {
        "nombre_cientifico": "Vitis vinifera",
        "variedades": ["Thompson", "Red Globe", "Crimson", "Flame"],
        "temp_min": 15,
        "temp_max": 35,
        "temp_optima": 26,
        "temp_critica_min": -1,
        "temp_critica_max": 42,
        "humedad_min": 40,
        "humedad_max": 60,
        "humedad_optima": 50,
        "precipitacion_anual": 350,
        "ph_suelo": [6.5, 7.5],
        "helada_critica": -1,
        "meses_siembra": [7, 8],
        "meses_floracion": [9, 10],
        "meses_cosecha": [2, 3, 4],
        "riego_mm_dia": 4.0,
        "grados_dia_desarrollo": 1800
    }
}

# Umbrales de alertas
UMBRALES_ALERTAS = {
    "helada": {
        "critica": 0,
        "advertencia": 3,
        "precaucion": 5
    },
    "temperatura_extrema": {
        "calor_extremo": 35,
        "calor_alto": 32,
        "frio_extremo": 2,
        "frio_alto": 5
    },
    "precipitacion": {
        "lluvia_intensa": 50,  # mm/día
        "lluvia_moderada": 25,
        "sequia": 0  # días sin lluvia
    },
    "viento": {
        "huracan": 120,  # km/h
        "temporal": 90,
        "fuerte": 60,
        "moderado": 40
    }
}

print("✅ Configuración cargada exitosamente")
print(f"📍 Estaciones configuradas: {len(QUILLOTA_CONFIG['estaciones'])}")
print(f"🌱 Cultivos configurados: {len(CULTIVOS_CONFIG)}")

✅ Configuración cargada exitosamente
📍 Estaciones configuradas: 6
🌱 Cultivos configurados: 4


In [15]:
# Voy a repetir el codigo pero con más celdas abajo pero es el mismo Configuración de PostgreSQL
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'mip_quillota'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', '1478')
}

def crear_conexion_db():
    """Crear conexión a PostgreSQL con manejo de errores"""
    try:
        conn_string = (
            f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
            f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        )
        
        engine = create_engine(conn_string, pool_pre_ping=True, pool_size=10)
        
        with engine.connect() as conn:
            result = conn.execute(text("SELECT version()"))
            version = result.fetchone()[0]
            print(f"✅ Conexión exitosa a PostgreSQL")
            print(f"📊 Versión: {version.split(',')[0]}")
            
        return engine
        
    except Exception as e:
        print(f"❌ Error conectando a PostgreSQL: {e}")
        return None

def verificar_estructura_existente(engine):
    """Verificar si las tablas ya existen y su estructura"""
    tablas_existentes = {}
    
    try:
        with engine.connect() as conn:
            # Obtener lista de tablas
            result = conn.execute(text("""
                SELECT table_name 
                FROM information_schema.tables 
                WHERE table_schema = 'public' 
                AND table_type = 'BASE TABLE'
            """))
            
            for row in result:
                tabla = row[0]
                # Obtener columnas de cada tabla
                cols_result = conn.execute(text(f"""
                    SELECT column_name 
                    FROM information_schema.columns 
                    WHERE table_name = '{tabla}'
                """))
                
                columnas = [col[0] for col in cols_result]
                tablas_existentes[tabla] = columnas
                
    except Exception as e:
        print(f"Error verificando estructura: {e}")
        
    return tablas_existentes

def crear_o_actualizar_tablas():
    """Crear tablas nuevas o trabajar con las existentes"""
    engine = crear_conexion_db()
    if not engine:
        return False
    
    # Verificar estructura existente
    tablas_existentes = verificar_estructura_existente(engine)
    print(f"\n📋 Tablas existentes: {list(tablas_existentes.keys())}")
    
    try:
        with engine.begin() as conn:
            # Verificar si la tabla datos_meteorologicos existe
            if 'datos_meteorologicos' in tablas_existentes:
                columnas = tablas_existentes['datos_meteorologicos']
                print(f"\n✅ Tabla 'datos_meteorologicos' ya existe con columnas: {columnas[:5]}...")
                
                # Verificar si necesitamos agregar columnas nuevas
                columnas_requeridas = {
                    'estacion_id': 'VARCHAR(10)',
                    'estacion_nombre': 'VARCHAR(100)',
                    'temperatura_aparente': 'FLOAT',
                    'humedad_relativa': 'FLOAT',
                    'precipitacion_probabilidad': 'FLOAT',
                    'rafaga_viento': 'FLOAT',
                    'presion_atmosferica': 'FLOAT',
                    'radiacion_solar': 'FLOAT',
                    'evapotranspiracion': 'FLOAT',
                    'indice_uv': 'FLOAT',
                    'punto_rocio': 'FLOAT'
                }
                
                for columna, tipo in columnas_requeridas.items():
                    if columna not in columnas:
                        try:
                            conn.execute(text(f"""
                                ALTER TABLE datos_meteorologicos 
                                ADD COLUMN IF NOT EXISTS {columna} {tipo}
                            """))
                            print(f"  ➕ Columna '{columna}' agregada")
                        except Exception as e:
                            print(f"  ⚠️ No se pudo agregar columna '{columna}': {e}")
            else:
                # Crear tabla nueva con la estructura requerida
                conn.execute(text("""
                    CREATE TABLE datos_meteorologicos (
                        id SERIAL PRIMARY KEY,
                        estacion VARCHAR(100),
                        estacion_id VARCHAR(10),
                        estacion_nombre VARCHAR(100),
                        fecha TIMESTAMP NOT NULL,
                        temperatura FLOAT,
                        temperatura_max FLOAT,
                        temperatura_min FLOAT,
                        temperatura_aparente FLOAT,
                        humedad FLOAT,
                        humedad_relativa FLOAT,
                        precipitacion FLOAT,
                        precipitacion_probabilidad FLOAT,
                        velocidad_viento FLOAT,
                        direccion_viento FLOAT,
                        rafaga_viento FLOAT,
                        presion FLOAT,
                        presion_atmosferica FLOAT,
                        radiacion_solar FLOAT,
                        evapotranspiracion FLOAT,
                        indice_uv FLOAT,
                        punto_rocio FLOAT,
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        UNIQUE(estacion, fecha)
                    )
                """))
                print("✅ Tabla 'datos_meteorologicos' creada")
            
            # Verificar tabla de pronósticos
            if 'pronosticos' not in tablas_existentes:
                conn.execute(text("""
                    CREATE TABLE pronosticos (
                        id SERIAL PRIMARY KEY,
                        estacion VARCHAR(100),
                        estacion_id VARCHAR(10),
                        fecha_pronostico TIMESTAMP NOT NULL,
                        fecha_prediccion TIMESTAMP,
                        tipo_pronostico VARCHAR(20),
                        temperatura_pred FLOAT,
                        temperatura_max_pred FLOAT,
                        temperatura_min_pred FLOAT,
                        precipitacion_pred FLOAT,
                        humedad_pred FLOAT,
                        probabilidad_helada FLOAT,
                        confianza FLOAT,
                        modelo_usado VARCHAR(50),
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    )
                """))
                print("✅ Tabla 'pronosticos' creada")
            else:
                print("✅ Tabla 'pronosticos' ya existe")
            
            # Verificar tabla de alertas
            if 'alertas' not in tablas_existentes:
                conn.execute(text("""
                    CREATE TABLE alertas (
                        id SERIAL PRIMARY KEY,
                        tipo_alerta VARCHAR(50) NOT NULL,
                        nivel VARCHAR(20) NOT NULL,
                        nivel_alerta VARCHAR(20),
                        mensaje TEXT,
                        descripcion TEXT,
                        fecha_alerta TIMESTAMP NOT NULL,
                        fecha_inicio TIMESTAMP,
                        fecha_fin TIMESTAMP,
                        estacion VARCHAR(100),
                        estacion_id VARCHAR(10),
                        cultivo_afectado VARCHAR(50),
                        acciones_recomendadas TEXT,
                        estado VARCHAR(20) DEFAULT 'activa',
                        activa BOOLEAN DEFAULT TRUE,
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    )
                """))
                print("✅ Tabla 'alertas' creada")
            else:
                print("✅ Tabla 'alertas' ya existe")
        
        # Crear índices basados en la estructura real
        print("\n📋 Creando/verificando índices...")
        
        indices_a_crear = []
        
        # Determinar qué índices crear basados en las columnas existentes
        with engine.connect() as conn:
            # Para datos_meteorologicos
            if 'datos_meteorologicos' in tablas_existentes:
                columnas = tablas_existentes['datos_meteorologicos']
                
                if 'fecha' in columnas:
                    indices_a_crear.append(
                        "CREATE INDEX IF NOT EXISTS idx_datos_met_fecha ON datos_meteorologicos(fecha)"
                    )
                if 'estacion' in columnas:
                    indices_a_crear.append(
                        "CREATE INDEX IF NOT EXISTS idx_datos_met_estacion ON datos_meteorologicos(estacion)"
                    )
                if 'estacion_id' in columnas:
                    indices_a_crear.append(
                        "CREATE INDEX IF NOT EXISTS idx_datos_met_estacion_id ON datos_meteorologicos(estacion_id)"
                    )
            
            # Para pronosticos
            if 'pronosticos' in tablas_existentes:
                indices_a_crear.append(
                    "CREATE INDEX IF NOT EXISTS idx_pronosticos_fecha ON pronosticos(fecha_pronostico)"
                )
            
            # Para alertas
            if 'alertas' in tablas_existentes:
                indices_a_crear.append(
                    "CREATE INDEX IF NOT EXISTS idx_alertas_fecha ON alertas(fecha_alerta)"
                )
                indices_a_crear.append(
                    "CREATE INDEX IF NOT EXISTS idx_alertas_estado ON alertas(estado)"
                )
            
            # Crear cada índice
            for indice in indices_a_crear:
                try:
                    conn.execute(text(indice))
                    conn.commit()
                    nombre_indice = indice.split('IF NOT EXISTS')[1].split('ON')[0].strip()
                    print(f"  ✅ Índice '{nombre_indice}' creado/verificado")
                except Exception as e:
                    if "already exists" in str(e):
                        print(f"  ℹ️ Índice ya existe: {indice.split()[-3]}")
                    else:
                        print(f"  ⚠️ Error con índice: {e}")
        
        print("\n✅ Estructura de base de datos lista")
        return True
        
    except Exception as e:
        print(f"❌ Error en la configuración de tablas: {e}")
        return False
    finally:
        engine.dispose()

def mostrar_resumen_estructura():
    """Mostrar un resumen de la estructura actual"""
    engine = crear_conexion_db()
    if not engine:
        return
    
    try:
        with engine.connect() as conn:
            print("\n📊 RESUMEN DE ESTRUCTURA DE BASE DE DATOS")
            print("=" * 50)
            
            # Contar registros en cada tabla
            tablas = ['datos_meteorologicos', 'pronosticos', 'alertas', 
                     'recomendaciones_cultivos', 'analisis_historicos']
            
            for tabla in tablas:
                try:
                    result = conn.execute(text(f"SELECT COUNT(*) FROM {tabla}"))
                    count = result.scalar()
                    print(f"📋 {tabla}: {count} registros")
                except:
                    print(f"📋 {tabla}: No existe o error")
            
            # Mostrar índices
            print("\n🔍 ÍNDICES EXISTENTES:")
            result = conn.execute(text("""
                SELECT indexname, tablename 
                FROM pg_indexes 
                WHERE schemaname = 'public' 
                AND indexname LIKE 'idx_%'
                ORDER BY tablename, indexname
            """))
            
            for idx in result:
                print(f"  - {idx[0]} en tabla {idx[1]}")
                
    except Exception as e:
        print(f"Error mostrando resumen: {e}")
    finally:
        engine.dispose()

# Ejecutar configuración
if __name__ == "__main__":
    if crear_o_actualizar_tablas():
        print("\n🗄️ Base de datos configurada correctamente")
        mostrar_resumen_estructura()
    else:
        print("\n⚠️ Problemas con la configuración de la base de datos")

# ============================================
# FUNCIONES DE UTILIDAD PARA TRABAJAR CON LA BD EXISTENTE
# ============================================

def obtener_mapeo_columnas():
    """Obtener mapeo entre nombres nuevos y existentes"""
    return {
        # Mapeo para insertar datos
        'nueva_a_existente': {
            'estacion_id': 'estacion',
            'fecha_hora': 'fecha',
            'humedad_relativa': 'humedad',
            'presion_atmosferica': 'presion'
        },
        # Mapeo para leer datos
        'existente_a_nueva': {
            'estacion': 'estacion_id',
            'fecha': 'fecha_hora',
            'humedad': 'humedad_relativa',
            'presion': 'presion_atmosferica'
        }
    }

def insertar_datos_meteorologicos(datos, engine=None):
    """Insertar datos adaptándose a la estructura existente"""
    if engine is None:
        engine = crear_conexion_db()
    
    if not engine:
        return False
    
    mapeo = obtener_mapeo_columnas()['nueva_a_existente']
    
    try:
        with engine.begin() as conn:
            # Verificar qué columnas existen realmente
            result = conn.execute(text("""
                SELECT column_name 
                FROM information_schema.columns 
                WHERE table_name = 'datos_meteorologicos'
            """))
            columnas_existentes = [row[0] for row in result]
            
            # Preparar datos para inserción
            datos_adaptados = {}
            for key, value in datos.items():
                # Si la columna existe directamente, usarla
                if key in columnas_existentes:
                    datos_adaptados[key] = value
                # Si existe con otro nombre, usar el mapeo
                elif key in mapeo and mapeo[key] in columnas_existentes:
                    datos_adaptados[mapeo[key]] = value
                # Si es una columna nueva que agregamos
                elif key in ['temperatura_aparente', 'punto_rocio', 'rafaga_viento']:
                    if key in columnas_existentes:
                        datos_adaptados[key] = value
            
            # Construir query dinámicamente
            columnas = list(datos_adaptados.keys())
            valores = list(datos_adaptados.values())
            placeholders = [f":{col}" for col in columnas]
            
            query = f"""
                INSERT INTO datos_meteorologicos ({', '.join(columnas)})
                VALUES ({', '.join(placeholders)})
                ON CONFLICT (estacion, fecha) DO UPDATE SET
                    {', '.join([f"{col} = EXCLUDED.{col}" for col in columnas if col not in ['estacion', 'fecha']])}
            """
            
            conn.execute(text(query), datos_adaptados)
            print(f"✅ Datos insertados para estación: {datos_adaptados.get('estacion', 'N/A')}")
            return True
            
    except Exception as e:
        print(f"❌ Error insertando datos: {e}")
        return False

def leer_datos_recientes(horas=24, estacion=None):
    """Leer datos recientes adaptándose a la estructura existente"""
    engine = crear_conexion_db()
    if not engine:
        return pd.DataFrame()
    
    try:
        query = """
            SELECT * FROM datos_meteorologicos 
            WHERE fecha >= NOW() - INTERVAL '%s hours'
        """
        params = [horas]
        
        if estacion:
            query += " AND estacion = %"
            params.append(estacion)
            
        query += " ORDER BY fecha DESC"
        
        df = pd.read_sql_query(query, engine, params=params)
        
        # Renombrar columnas para compatibilidad
        mapeo = obtener_mapeo_columnas()['existente_a_nueva']
        df.rename(columns=mapeo, inplace=True)
        
        return df
        
    except Exception as e:
        print(f"Error leyendo datos: {e}")
        return pd.DataFrame()
    finally:
        engine.dispose()

def obtener_alertas_activas():
    """Obtener alertas activas - VERSIÓN CORREGIDA"""
    engine = crear_conexion_db()
    if not engine:
        return pd.DataFrame()
    
    try:
        # Usar sintaxis correcta de INTERVAL
        query = """
            SELECT * FROM alertas
            WHERE activa = TRUE
            AND fecha_alerta >= NOW() - INTERVAL '24 hours'
            ORDER BY fecha_alerta DESC
        """
        
        df = pd.read_sql_query(query, engine)
        return df
        
    except Exception as e:
        print(f"Error obteniendo alertas: {e}")
        return pd.DataFrame()
    finally:
        engine.dispose()
        
def crear_alerta_compatible(tipo_alerta, nivel, mensaje, estacion=None, detalles=None):
    """Crear alerta compatible con la estructura existente"""
    engine = crear_conexion_db()
    if not engine:
        return False
    
    try:
        with engine.begin() as conn:
            # Verificar columnas existentes en alertas
            result = conn.execute(text("""
                SELECT column_name 
                FROM information_schema.columns 
                WHERE table_name = 'alertas'
            """))
            columnas_alertas = [row[0] for row in result]
            
            # Preparar datos base
            datos_alerta = {
                'tipo_alerta': tipo_alerta,
                'mensaje': mensaje,
                'fecha_alerta': datetime.now()
            }
            
            # Agregar campos según existan
            if 'nivel' in columnas_alertas:
                datos_alerta['nivel'] = nivel
            elif 'nivel_alerta' in columnas_alertas:
                datos_alerta['nivel_alerta'] = nivel
                
            if estacion:
                if 'estacion' in columnas_alertas:
                    datos_alerta['estacion'] = estacion
                if 'estacion_id' in columnas_alertas:
                    datos_alerta['estacion_id'] = estacion[:10]  # Limitar a 10 caracteres
            
            if detalles:
                if 'descripcion' in columnas_alertas:
                    datos_alerta['descripcion'] = detalles.get('descripcion', '')
                if 'acciones_recomendadas' in columnas_alertas:
                    datos_alerta['acciones_recomendadas'] = detalles.get('recomendaciones', '')
            
            # Construir query
            columnas = list(datos_alerta.keys())
            placeholders = [f":{col}" for col in columnas]
            
            query = f"""
                INSERT INTO alertas ({', '.join(columnas)})
                VALUES ({', '.join(placeholders)})
            """
            
            conn.execute(text(query), datos_alerta)
            print(f"✅ Alerta creada: {tipo_alerta}")
            return True
            
    except Exception as e:
        print(f"❌ Error creando alerta: {e}")
        return False
    finally:
        engine.dispose()

# ============================================
# FUNCIONES DE PRUEBA
# ============================================

def probar_sistema():
    """Probar las funciones principales del sistema"""
    print("\n🧪 INICIANDO PRUEBAS DEL SISTEMA")
    print("=" * 50)
    
    # 1. Probar inserción de datos
    print("\n1️⃣ Probando inserción de datos...")
    datos_prueba = {
        'estacion': 'Quillota_Centro',
        'fecha': datetime.now(),
        'temperatura': 22.5,
        'temperatura_max': 28.0,
        'temperatura_min': 15.0,
        'humedad': 65.0,
        'precipitacion': 0.0,
        'velocidad_viento': 12.5,
        'direccion_viento': 180.0,
        'presion': 1013.25,
        'radiacion_solar': 650.0
    }
    
    if insertar_datos_meteorologicos(datos_prueba):
        print("   ✅ Inserción exitosa")
    else:
        print("   ❌ Fallo en inserción")
    
    # 2. Probar lectura de datos
    print("\n2️⃣ Probando lectura de datos...")
    df = leer_datos_recientes(horas=1)
    if not df.empty:
        print(f"   ✅ Datos leídos: {len(df)} registros")
        print(f"   📊 Columnas: {list(df.columns)[:5]}...")
    else:
        print("   ❌ No se pudieron leer datos")
    
    # 3. Probar creación de alertas
    print("\n3️⃣ Probando creación de alertas...")
    if crear_alerta_compatible(
        tipo_alerta="Temperatura Alta",
        nivel="warning",
        mensaje="Temperatura superior a 30°C detectada",
        estacion="Quillota_Centro",
        detalles={
            'descripcion': 'Se ha detectado una temperatura elevada que puede afectar los cultivos',
            'recomendaciones': 'Activar sistema de riego, proporcionar sombra a cultivos sensibles'
        }
    ):
        print("   ✅ Alerta creada exitosamente")
    else:
        print("   ❌ Fallo en creación de alerta")
    
    # 4. Verificar integridad
    print("\n4️⃣ Verificando integridad del sistema...")
    engine = crear_conexion_db()
    if engine:
        try:
            with engine.connect() as conn:
                # Contar registros
                for tabla in ['datos_meteorologicos', 'alertas', 'pronosticos']:
                    try:
                        result = conn.execute(text(f"SELECT COUNT(*) FROM {tabla}"))
                        count = result.scalar()
                        print(f"   📊 {tabla}: {count} registros")
                    except:
                        pass
        except Exception as e:
            print(f"   ❌ Error verificando integridad: {e}")
        finally:
            engine.dispose()
    
    print("\n✅ PRUEBAS COMPLETADAS")
    print("=" * 50)

# ============================================
# FUNCIONES AUXILIARES PARA CONSULTAS COMUNES
# ============================================

def obtener_resumen_diario(fecha=None):
    """Obtener resumen de datos del día"""
    if fecha is None:
        fecha = datetime.now().date()
    
    engine = crear_conexion_db()
    if not engine:
        return None
    
    try:
        query = """
            SELECT 
                estacion,
                DATE(fecha) as fecha_datos,
                COUNT(*) as num_registros,
                AVG(temperatura) as temp_promedio,
                MAX(temperatura_max) as temp_maxima,
                MIN(temperatura_min) as temp_minima,
                AVG(humedad) as humedad_promedio,
                SUM(precipitacion) as precipitacion_total,
                AVG(velocidad_viento) as viento_promedio
            FROM datos_meteorologicos
            WHERE DATE(fecha) = :fecha
            GROUP BY estacion, DATE(fecha)
            ORDER BY estacion
        """
        
        df = pd.read_sql_query(text(query), engine, params={'fecha': fecha})
        return df
        
    except Exception as e:
        print(f"Error obteniendo resumen: {e}")
        return None
    finally:
        engine.dispose()

def obtener_alertas_activas():
    """Obtener todas las alertas activas"""
    engine = crear_conexion_db()
    if not engine:
        return pd.DataFrame()
    
    try:
        # Adaptarse a las columnas existentes
        query = """
            SELECT * FROM alertas
            WHERE (activa = TRUE OR estado = 'activa')
            AND fecha_alerta >= NOW() - INTERVAL '24 HOUR'
            ORDER BY fecha_alerta DESC
        """
        
        df = pd.read_sql_query(text(query), engine)
        return df
        
    except Exception as e:
        # Si falla, intentar sin filtro de estado
        try:
            query = """
                SELECT * FROM alertas
                WHERE fecha_alerta >= NOW() - INTERVAL '24 HOUR'
                ORDER BY fecha_alerta DESC
            """
            df = pd.read_sql_query(text(query), engine)
            return df
        except:
            print(f"Error obteniendo alertas: {e}")
            return pd.DataFrame()
    finally:
        engine.dispose()

# Ejecutar pruebas si se ejecuta directamente
if __name__ == "__main__":
    # Primero configurar la BD
    if crear_o_actualizar_tablas():
        print("\n🗄️ Base de datos configurada correctamente")
        mostrar_resumen_estructura()
        
        # Ejecutar pruebas
        probar_sistema()
        
        # Mostrar resumen del día
        print("\n📊 RESUMEN DEL DÍA")
        resumen = obtener_resumen_diario()
        if resumen is not None and not resumen.empty:
            print(resumen)
        else:
            print("No hay datos para hoy")
            
        # Mostrar alertas activas
        print("\n⚠️ ALERTAS ACTIVAS")
        alertas = obtener_alertas_activas()
        if not alertas.empty:
            print(f"Se encontraron {len(alertas)} alertas activas")
            for _, alerta in alertas.head(5).iterrows():
                print(f"  - {alerta.get('tipo_alerta', 'N/A')}: {alerta.get('mensaje', 'Sin mensaje')}")
        else:
            print("No hay alertas activas")
    else:
        print("\n⚠️ Problemas con la configuración de la base de datos")

✅ Conexión exitosa a PostgreSQL
📊 Versión: PostgreSQL 17.4 on x86_64-windows

📋 Tablas existentes: ['weather_stations', 'weather_measurements', 'crops', 'fields', 'pest_monitoring', 'alerts', 'pronosticos', 'alertas', 'recomendaciones_cultivos', 'analisis_historicos', 'datos_meteorologicos']

✅ Tabla 'datos_meteorologicos' ya existe con columnas: ['punto_rocio', 'evapotranspiracion', 'indice_uv', 'id', 'fecha']...
✅ Tabla 'pronosticos' ya existe
✅ Tabla 'alertas' ya existe

📋 Creando/verificando índices...
  ✅ Índice 'idx_datos_met_fecha' creado/verificado
  ✅ Índice 'idx_datos_met_estacion' creado/verificado
  ✅ Índice 'idx_datos_met_estacion_id' creado/verificado
  ✅ Índice 'idx_pronosticos_fecha' creado/verificado
  ✅ Índice 'idx_alertas_fecha' creado/verificado
  ⚠️ Error con índice: (psycopg2.errors.UndefinedColumn) column "estado" does not exist

[SQL: CREATE INDEX IF NOT EXISTS idx_alertas_estado ON alertas(estado)]
(Background on this error at: https://sqlalche.me/e/20/f405)

✅

In [5]:
# Configuración de PostgreSQL
import os
import psycopg2
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import datetime
import requests

DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'mip_quillota'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', '1478')
}

def crear_conexion_db():
    """Crear conexión a PostgreSQL con manejo de errores"""
    try:
        # String de conexión
        conn_string = (
            f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
            f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        )
        
        # Crear engine
        engine = create_engine(conn_string, pool_pre_ping=True, pool_size=10)
        
        # Verificar conexión
        with engine.connect() as conn:
            result = conn.execute(text("SELECT version()"))
            version = result.fetchone()[0]
            print(f"✅ Conexión exitosa a PostgreSQL")
            print(f"📊 Versión: {version.split(',')[0]}")
            
        return engine
        
    except Exception as e:
        print(f"❌ Error conectando a PostgreSQL: {e}")
        print("Intentando crear base de datos...")
        return crear_base_datos()

def crear_base_datos():
    """Crear base de datos si no existe"""
    try:
        # Conectar a postgres default
        conn = psycopg2.connect(
            host=DB_CONFIG['host'],
            port=DB_CONFIG['port'],
            user=DB_CONFIG['user'],
            password=DB_CONFIG['password'],
            database='postgres'
        )
        conn.autocommit = True
        cursor = conn.cursor()
        
        # Crear base de datos
        cursor.execute(f"CREATE DATABASE {DB_CONFIG['database']}")
        print(f"✅ Base de datos '{DB_CONFIG['database']}' creada exitosamente")
        
        cursor.close()
        conn.close()
        
        # Reconectar a la nueva base de datos
        return crear_conexion_db()
        
    except psycopg2.errors.DuplicateDatabase:
        print("Base de datos ya existe, reconectando...")
        return crear_conexion_db()
    except Exception as e:
        print(f"❌ Error creando base de datos: {e}")
        return None

def verificar_estructura_existente(engine):
    """Verificar si las tablas ya existen y su estructura"""
    tablas_existentes = {}
    
    try:
        with engine.connect() as conn:
            # Obtener lista de tablas
            result = conn.execute(text("""
                SELECT table_name 
                FROM information_schema.tables 
                WHERE table_schema = 'public' 
                AND table_type = 'BASE TABLE'
            """))
            
            for row in result:
                tabla = row[0]
                # Obtener columnas de cada tabla
                cols_result = conn.execute(text(f"""
                    SELECT column_name 
                    FROM information_schema.columns 
                    WHERE table_name = '{tabla}'
                """))
                
                columnas = [col[0] for col in cols_result]
                tablas_existentes[tabla] = columnas
                
    except Exception as e:
        print(f"Error verificando estructura: {e}")
        
    return tablas_existentes

def crear_o_actualizar_tablas():
    """Crear tablas nuevas o trabajar con las existentes"""
    engine = crear_conexion_db()
    if not engine:
        return False
    
    # Verificar estructura existente
    tablas_existentes = verificar_estructura_existente(engine)
    print(f"\n📋 Tablas existentes: {list(tablas_existentes.keys())}")
    
    try:
        with engine.begin() as conn:
            # Verificar si la tabla datos_meteorologicos existe
            if 'datos_meteorologicos' in tablas_existentes:
                columnas = tablas_existentes['datos_meteorologicos']
                print(f"\n✅ Tabla 'datos_meteorologicos' ya existe con columnas: {columnas[:5]}...")
                
                # Verificar si necesitamos agregar columnas nuevas
                columnas_requeridas = {
                    'estacion_id': 'VARCHAR(10)',
                    'estacion_nombre': 'VARCHAR(100)',
                    'temperatura_aparente': 'FLOAT',
                    'humedad_relativa': 'FLOAT',
                    'precipitacion_probabilidad': 'FLOAT',
                    'rafaga_viento': 'FLOAT',
                    'presion_atmosferica': 'FLOAT',
                    'radiacion_solar': 'FLOAT',
                    'evapotranspiracion': 'FLOAT',
                    'indice_uv': 'FLOAT',
                    'punto_rocio': 'FLOAT'
                }
                
                for columna, tipo in columnas_requeridas.items():
                    if columna not in columnas:
                        try:
                            conn.execute(text(f"""
                                ALTER TABLE datos_meteorologicos 
                                ADD COLUMN IF NOT EXISTS {columna} {tipo}
                            """))
                            print(f"  ➕ Columna '{columna}' agregada")
                        except Exception as e:
                            print(f"  ⚠️ No se pudo agregar columna '{columna}': {e}")
            else:
                # Crear tabla nueva con la estructura requerida
                conn.execute(text("""
                    CREATE TABLE datos_meteorologicos (
                        id SERIAL PRIMARY KEY,
                        estacion VARCHAR(100),
                        estacion_id VARCHAR(10),
                        estacion_nombre VARCHAR(100),
                        fecha TIMESTAMP NOT NULL,
                        temperatura FLOAT,
                        temperatura_max FLOAT,
                        temperatura_min FLOAT,
                        temperatura_aparente FLOAT,
                        humedad FLOAT,
                        humedad_relativa FLOAT,
                        precipitacion FLOAT,
                        precipitacion_probabilidad FLOAT,
                        velocidad_viento FLOAT,
                        direccion_viento FLOAT,
                        rafaga_viento FLOAT,
                        presion FLOAT,
                        presion_atmosferica FLOAT,
                        radiacion_solar FLOAT,
                        evapotranspiracion FLOAT,
                        indice_uv FLOAT,
                        punto_rocio FLOAT,
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        UNIQUE(estacion, fecha)
                    )
                """))
                print("✅ Tabla 'datos_meteorologicos' creada")
            
            # Verificar tabla de pronósticos
            if 'pronosticos' not in tablas_existentes:
                conn.execute(text("""
                    CREATE TABLE pronosticos (
                        id SERIAL PRIMARY KEY,
                        estacion VARCHAR(100),
                        estacion_id VARCHAR(10),
                        fecha_pronostico TIMESTAMP NOT NULL,
                        fecha_prediccion TIMESTAMP,
                        tipo_pronostico VARCHAR(20),
                        temperatura_pred FLOAT,
                        temperatura_max_pred FLOAT,
                        temperatura_min_pred FLOAT,
                        precipitacion_pred FLOAT,
                        humedad_pred FLOAT,
                        probabilidad_helada FLOAT,
                        confianza FLOAT,
                        modelo_usado VARCHAR(50),
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    )
                """))
                print("✅ Tabla 'pronosticos' creada")
            else:
                print("✅ Tabla 'pronosticos' ya existe")
            
            # Verificar tabla de alertas
            if 'alertas' not in tablas_existentes:
                conn.execute(text("""
                    CREATE TABLE alertas (
                        id SERIAL PRIMARY KEY,
                        tipo_alerta VARCHAR(50) NOT NULL,
                        nivel VARCHAR(20) NOT NULL,
                        nivel_alerta VARCHAR(20),
                        mensaje TEXT,
                        descripcion TEXT,
                        fecha_alerta TIMESTAMP NOT NULL,
                        fecha_inicio TIMESTAMP,
                        fecha_fin TIMESTAMP,
                        estacion VARCHAR(100),
                        estacion_id VARCHAR(10),
                        cultivo_afectado VARCHAR(50),
                        acciones_recomendadas TEXT,
                        estado VARCHAR(20) DEFAULT 'activa',
                        activa BOOLEAN DEFAULT TRUE,
                        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                    )
                """))
                print("✅ Tabla 'alertas' creada")
            else:
                print("✅ Tabla 'alertas' ya existe")
        
        # Crear índices basados en la estructura real
        print("\n📋 Creando/verificando índices...")
        
        indices_a_crear = []
        
        # Determinar qué índices crear basados en las columnas existentes
        with engine.connect() as conn:
            # Para datos_meteorologicos
            if 'datos_meteorologicos' in tablas_existentes:
                columnas = tablas_existentes['datos_meteorologicos']
                
                if 'fecha' in columnas:
                    indices_a_crear.append(
                        "CREATE INDEX IF NOT EXISTS idx_datos_met_fecha ON datos_meteorologicos(fecha)"
                    )
                if 'estacion' in columnas:
                    indices_a_crear.append(
                        "CREATE INDEX IF NOT EXISTS idx_datos_met_estacion ON datos_meteorologicos(estacion)"
                    )
                if 'estacion_id' in columnas:
                    indices_a_crear.append(
                        "CREATE INDEX IF NOT EXISTS idx_datos_met_estacion_id ON datos_meteorologicos(estacion_id)"
                    )
            
            # Para pronosticos
            if 'pronosticos' in tablas_existentes:
                indices_a_crear.append(
                    "CREATE INDEX IF NOT EXISTS idx_pronosticos_fecha ON pronosticos(fecha_pronostico)"
                )
            
            # Para alertas
            if 'alertas' in tablas_existentes:
                indices_a_crear.append(
                    "CREATE INDEX IF NOT EXISTS idx_alertas_fecha ON alertas(fecha_alerta)"
                )
                # Solo crear índice de estado si la columna existe
                columnas_alertas = tablas_existentes.get('alertas', [])
                if 'estado' in columnas_alertas:
                    indices_a_crear.append(
                        "CREATE INDEX IF NOT EXISTS idx_alertas_estado ON alertas(estado)"
                    )
            
            # Crear cada índice
            for indice in indices_a_crear:
                try:
                    conn.execute(text(indice))
                    conn.commit()
                    nombre_indice = indice.split('IF NOT EXISTS')[1].split('ON')[0].strip()
                    print(f"  ✅ Índice '{nombre_indice}' creado/verificado")
                except Exception as e:
                    if "already exists" in str(e):
                        print(f"  ℹ️ Índice ya existe: {indice.split()[-3]}")
                    else:
                        print(f"  ⚠️ Error con índice: {e}")
        
        print("\n✅ Estructura de base de datos lista")
        return True
        
    except Exception as e:
        print(f"❌ Error en la configuración de tablas: {e}")
        return False
    finally:
        engine.dispose()

def mostrar_resumen_estructura():
    """Mostrar un resumen de la estructura actual"""
    engine = crear_conexion_db()
    if not engine:
        return
    
    try:
        with engine.connect() as conn:
            print("\n📊 RESUMEN DE ESTRUCTURA DE BASE DE DATOS")
            print("=" * 50)
            
            # Contar registros en cada tabla
            tablas = ['datos_meteorologicos', 'pronosticos', 'alertas', 
                     'recomendaciones_cultivos', 'analisis_historicos']
            
            for tabla in tablas:
                try:
                    result = conn.execute(text(f"SELECT COUNT(*) FROM {tabla}"))
                    count = result.scalar()
                    print(f"📋 {tabla}: {count} registros")
                except:
                    print(f"📋 {tabla}: No existe o error")
            
            # Mostrar índices
            print("\n🔍 ÍNDICES EXISTENTES:")
            result = conn.execute(text("""
                SELECT indexname, tablename 
                FROM pg_indexes 
                WHERE schemaname = 'public' 
                AND indexname LIKE 'idx_%'
                ORDER BY tablename, indexname
            """))
            
            for idx in result:
                print(f"  - {idx[0]} en tabla {idx[1]}")
                
    except Exception as e:
        print(f"Error mostrando resumen: {e}")
    finally:
        engine.dispose()

# Continúa con las funciones de mapeo, inserción, etc...

In [6]:
def configurar_cliente_openmeteo():
    """Configurar cliente de OpenMeteo con cache y retry"""
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    return openmeteo_requests.Client(session=retry_session)

def obtener_datos_actuales(cliente, lat, lon, nombre_estacion, estacion_id):
    """Obtener datos meteorológicos actuales y pronóstico de OpenMeteo"""
    
    url = "https://api.open-meteo.com/v1/forecast"
    
    params = {
        "latitude": lat,
        "longitude": lon,
        "current": [
            "temperature_2m", "relative_humidity_2m", "apparent_temperature",
            "precipitation", "rain", "weather_code", "surface_pressure",
            "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"
        ],
        "hourly": [
            "temperature_2m", "relative_humidity_2m", "dew_point_2m",
            "apparent_temperature", "precipitation_probability", "precipitation",
            "rain", "weather_code", "surface_pressure", "visibility",
            "evapotranspiration", "et0_fao_evapotranspiration",
            "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m",
            "temperature_80m", "soil_temperature_0cm", "soil_moisture_0_to_1cm",
            "uv_index", "direct_radiation", "diffuse_radiation",
            "direct_normal_irradiance", "terrestrial_radiation"
        ],
        "daily": [
            "weather_code", "temperature_2m_max", "temperature_2m_min",
            "apparent_temperature_max", "apparent_temperature_min",
            "sunrise", "sunset", "daylight_duration", "sunshine_duration",
            "uv_index_max", "precipitation_sum", "rain_sum",
            "precipitation_hours", "precipitation_probability_max",
            "wind_speed_10m_max", "wind_gusts_10m_max",
            "wind_direction_10m_dominant", "et0_fao_evapotranspiration"
        ],
        "timezone": "America/Santiago",
        "past_days": 7,
        "forecast_days": 16
    }
    
    try:
        responses = cliente.weather_api(url, params=params)
        response = responses[0]
        
        print(f"📡 Datos obtenidos para {nombre_estacion}")
        print(f"   Coordenadas: {response.Latitude()}°, {response.Longitude()}°")
        print(f"   Elevación: {response.Elevation()} m")
        
        # Procesar datos actuales
        current = response.Current()
        current_data = {
            "estacion_id": estacion_id,
            "estacion_nombre": nombre_estacion,
            "fecha_hora": pd.to_datetime(current.Time(), unit='s'),
            "temperatura": current.Variables(0).Value(),
            "humedad_relativa": current.Variables(1).Value(),
            "temperatura_aparente": current.Variables(2).Value(),
            "precipitacion": current.Variables(3).Value(),
            "presion_atmosferica": current.Variables(6).Value(),
            "velocidad_viento": current.Variables(7).Value(),
            "direccion_viento": current.Variables(8).Value(),
            "rafaga_viento": current.Variables(9).Value()
        }
        
        # Procesar datos horarios
        hourly = response.Hourly()
        hourly_data = pd.DataFrame({
            "fecha_hora": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit='s'),
                end=pd.to_datetime(hourly.TimeEnd(), unit='s'),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "temperatura": hourly.Variables(0).ValuesAsNumpy(),
            "humedad_relativa": hourly.Variables(1).ValuesAsNumpy(),
            "punto_rocio": hourly.Variables(2).ValuesAsNumpy(),
            "temperatura_aparente": hourly.Variables(3).ValuesAsNumpy(),
            "precipitacion_probabilidad": hourly.Variables(4).ValuesAsNumpy(),
            "precipitacion": hourly.Variables(5).ValuesAsNumpy(),
            "presion_atmosferica": hourly.Variables(8).ValuesAsNumpy(),
            "evapotranspiracion": hourly.Variables(11).ValuesAsNumpy(),
            "velocidad_viento": hourly.Variables(12).ValuesAsNumpy(),
            "direccion_viento": hourly.Variables(13).ValuesAsNumpy(),
            "rafaga_viento": hourly.Variables(14).ValuesAsNumpy(),
            "indice_uv": hourly.Variables(18).ValuesAsNumpy(),
            "radiacion_solar": hourly.Variables(19).ValuesAsNumpy()
        })
        
        hourly_data["estacion_id"] = estacion_id
        hourly_data["estacion_nombre"] = nombre_estacion
        
        # Procesar datos diarios
        daily = response.Daily()
        daily_data = pd.DataFrame({
            "fecha": pd.date_range(
                start=pd.to_datetime(daily.Time(), unit='s'),
                end=pd.to_datetime(daily.TimeEnd(), unit='s'),
                freq=pd.Timedelta(seconds=daily.Interval()),
                inclusive="left"
            ),
            "temperatura_max": daily.Variables(1).ValuesAsNumpy(),
            "temperatura_min": daily.Variables(2).ValuesAsNumpy(),
            "temperatura_aparente_max": daily.Variables(3).ValuesAsNumpy(),
            "temperatura_aparente_min": daily.Variables(4).ValuesAsNumpy(),
            "amanecer": daily.Variables(5).ValuesAsNumpy(),
            "atardecer": daily.Variables(6).ValuesAsNumpy(),
            "horas_luz": daily.Variables(7).ValuesAsNumpy(),
            "horas_sol": daily.Variables(8).ValuesAsNumpy(),
            "indice_uv_max": daily.Variables(9).ValuesAsNumpy(),
            "precipitacion_suma": daily.Variables(10).ValuesAsNumpy(),
            "horas_precipitacion": daily.Variables(12).ValuesAsNumpy(),
            "probabilidad_precipitacion_max": daily.Variables(13).ValuesAsNumpy(),
            "velocidad_viento_max": daily.Variables(14).ValuesAsNumpy(),
            "rafaga_viento_max": daily.Variables(15).ValuesAsNumpy(),
            "direccion_viento_dominante": daily.Variables(16).ValuesAsNumpy(),
            "evapotranspiracion": daily.Variables(17).ValuesAsNumpy()
        })
        
        daily_data["estacion_id"] = estacion_id
        daily_data["estacion_nombre"] = nombre_estacion
        
        return current_data, hourly_data, daily_data
        
    except Exception as e:
        print(f"❌ Error obteniendo datos para {nombre_estacion}: {e}")
        return None, None, None

def recopilar_datos_todas_estaciones():
    """Recopilar datos de todas las estaciones configuradas"""
    cliente = configurar_cliente_openmeteo()
    
    datos_actuales = []
    datos_horarios = []
    datos_diarios = []
    
    for estacion in QUILLOTA_CONFIG["estaciones"]:
        print(f"\n🌡️ Procesando estación: {estacion['nombre']}")
        
        current, hourly, daily = obtener_datos_actuales(
            cliente, 
            estacion["lat"], 
            estacion["lon"],
            estacion["nombre"],
            estacion["id"]
        )
        
        if current is not None:
            datos_actuales.append(current)
            datos_horarios.append(hourly)
            datos_diarios.append(daily)
            
        time.sleep(0.5)  # Evitar sobrecarga de API
    
        # Consolidar datos
    if datos_horarios:
        df_horarios_consolidado = pd.concat(datos_horarios, ignore_index=True)
        df_diarios_consolidado = pd.concat(datos_diarios, ignore_index=True)
        
        print(f"\n✅ Datos consolidados:")
        print(f"   - Registros horarios: {len(df_horarios_consolidado)}")
        print(f"   - Registros diarios: {len(df_diarios_consolidado)}")
        print(f"   - Estaciones procesadas: {len(datos_actuales)}")
        
        return datos_actuales, df_horarios_consolidado, df_diarios_consolidado
    else:
        print("❌ No se pudieron obtener datos")
        return [], pd.DataFrame(), pd.DataFrame()

# Ejecutar recopilación inicial
datos_actuales, df_horarios, df_diarios = recopilar_datos_todas_estaciones()


🌡️ Procesando estación: Quillota Centro
📡 Datos obtenidos para Quillota Centro
   Coordenadas: -32.875°, -71.25°
   Elevación: 124.0 m

🌡️ Procesando estación: La Cruz
📡 Datos obtenidos para La Cruz
   Coordenadas: -32.875°, -71.25°
   Elevación: 95.0 m

🌡️ Procesando estación: La Calera
📡 Datos obtenidos para La Calera
   Coordenadas: -32.875°, -71.25°
   Elevación: 149.0 m

🌡️ Procesando estación: Hijuelas
📡 Datos obtenidos para Hijuelas
   Coordenadas: -32.625°, -71.375°
   Elevación: 195.0 m

🌡️ Procesando estación: Limache
📡 Datos obtenidos para Limache
   Coordenadas: -33.0°, -71.375°
   Elevación: 91.0 m

🌡️ Procesando estación: Nogales
📡 Datos obtenidos para Nogales
   Coordenadas: -32.625°, -71.375°
   Elevación: 211.0 m

✅ Datos consolidados:
   - Registros horarios: 3312
   - Registros diarios: 138
   - Estaciones procesadas: 6


In [7]:
def calcular_indices_agrometeorologicos(df_datos):
    """Calcular índices específicos para agricultura"""
    
    df_indices = df_datos.copy()
    
    # 1. Grados Día de Crecimiento (GDD)
    def calcular_gdd(temp_max, temp_min, temp_base=10, temp_tope=30):
        """Calcular grados día para desarrollo de cultivos"""
        temp_media = (temp_max + temp_min) / 2
        temp_media = np.clip(temp_media, temp_base, temp_tope)
        return max(0, temp_media - temp_base)
    
    if 'temperatura_max' in df_indices.columns and 'temperatura_min' in df_indices.columns:
        df_indices['gdd_base10'] = df_indices.apply(
            lambda x: calcular_gdd(x['temperatura_max'], x['temperatura_min'], 10), axis=1
        )
        df_indices['gdd_base15'] = df_indices.apply(
            lambda x: calcular_gdd(x['temperatura_max'], x['temperatura_min'], 15), axis=1
        )
    
    # 2. Horas Frío (Chilling Hours)
    def calcular_horas_frio(df_hourly, umbral=7):
        """Calcular horas bajo umbral de temperatura"""
        horas_frio = (df_hourly['temperatura'] < umbral).groupby(
            df_hourly['fecha_hora'].dt.date
        ).sum()
        return horas_frio
    
    # 3. Índice de Estrés Térmico (THI)
    if 'temperatura' in df_indices.columns and 'humedad_relativa' in df_indices.columns:
        df_indices['indice_estres_termico'] = (
            df_indices['temperatura'] + 
            0.36 * df_indices['punto_rocio'] + 41.2
        )
    
    # 4. Déficit de Presión de Vapor (VPD)
    def calcular_vpd(temp, humedad):
        """Calcular déficit de presión de vapor en kPa"""
        # Presión de vapor de saturación
        es = 0.6108 * np.exp(17.27 * temp / (temp + 237.3))
        # Presión de vapor actual
        ea = es * (humedad / 100)
        # VPD
        return es - ea
    
    if 'temperatura' in df_indices.columns and 'humedad_relativa' in df_indices.columns:
        df_indices['vpd'] = df_indices.apply(
            lambda x: calcular_vpd(x['temperatura'], x['humedad_relativa']), axis=1
        )
    
    # 5. Evapotranspiración de Referencia (ET0) - Penman-Monteith simplificado
    def calcular_et0_hargreaves(temp_max, temp_min, temp_media, lat, dia_juliano):
        """Método Hargreaves para ET0"""
        # Radiación extraterrestre
        dr = 1 + 0.033 * np.cos(2 * np.pi * dia_juliano / 365)
        delta = 0.409 * np.sin(2 * np.pi * dia_juliano / 365 - 1.39)
        ws = np.arccos(-np.tan(lat * np.pi / 180) * np.tan(delta))
        Ra = 24 * 60 / np.pi * 0.082 * dr * (
            ws * np.sin(lat * np.pi / 180) * np.sin(delta) +
            np.cos(lat * np.pi / 180) * np.cos(delta) * np.sin(ws)
        )
        
        # ET0 Hargreaves
        ET0 = 0.0023 * (temp_media + 17.8) * np.sqrt(temp_max - temp_min) * Ra
        return max(0, ET0)
    
    # 6. Probabilidad de Heladas
    def calcular_prob_helada(temp_min, humedad, viento, nubosidad=50):
        """Calcular probabilidad de helada basado en múltiples factores"""
        prob_base = 0
        
        # Factor temperatura
        if temp_min <= 0:
            prob_base = 0.9
        elif temp_min <= 2:
            prob_base = 0.7
        elif temp_min <= 4:
            prob_base = 0.5
        elif temp_min <= 6:
            prob_base = 0.3
        else:
            prob_base = 0.1
        
        # Ajuste por humedad (menor humedad = mayor probabilidad)
        factor_humedad = 1 - (humedad / 100) * 0.3
        
        # Ajuste por viento (menos viento = mayor probabilidad)
        factor_viento = 1 - min(viento / 20, 1) * 0.2
        
        # Ajuste por nubosidad (cielo despejado = mayor probabilidad)
        factor_nubes = 1 - (nubosidad / 100) * 0.2
        
        prob_final = prob_base * factor_humedad * factor_viento * factor_nubes
        return min(1, max(0, prob_final))
    
    # 7. Índice de Sequía
    def calcular_indice_sequia(precipitacion_actual, precipitacion_normal, periodo_dias=30):
        """Calcular índice de sequía simplificado"""
        if precipitacion_normal == 0:
            return 0
        
        ratio = precipitacion_actual / precipitacion_normal
        
        if ratio >= 1.2:
            return 3  # Muy húmedo
        elif ratio >= 0.8:
            return 2  # Normal
        elif ratio >= 0.6:
            return 1  # Sequía leve
        elif ratio >= 0.4:
            return 0  # Sequía moderada
        else:
            return -1  # Sequía severa
    
    # 8. Condiciones para Enfermedades
    def calcular_riesgo_enfermedades(temp, humedad, horas_mojado=0):
        """Calcular riesgo de enfermedades fúngicas"""
        riesgo = 0
        
        # Condiciones óptimas para hongos: 15-25°C y >80% humedad
        if 15 <= temp <= 25 and humedad > 80:
            riesgo = 0.8
        elif 10 <= temp <= 30 and humedad > 70:
            riesgo = 0.6
        elif humedad > 60:
            riesgo = 0.4
        else:
            riesgo = 0.2
        
        # Ajuste por horas de mojado foliar
        if horas_mojado > 6:
            riesgo = min(1, riesgo * 1.5)
        
        return riesgo
    
    return df_indices

def analizar_condiciones_cultivo(df_datos, cultivo, fecha_analisis=None):
    """Analizar condiciones específicas para un cultivo"""
    
    if fecha_analisis is None:
        fecha_analisis = datetime.now()
    
    config = CULTIVOS_CONFIG.get(cultivo, {})
    analisis = {
        "cultivo": cultivo,
        "fecha_analisis": fecha_analisis,
        "condiciones": {},
        "recomendaciones": [],
        "alertas": [],
        "indices": {}
    }
    
    # Filtrar datos relevantes
    df_reciente = df_datos[df_datos['fecha_hora'] >= fecha_analisis - timedelta(days=7)]
    
    if df_reciente.empty:
        return analisis
    
    # Análisis de temperatura
    temp_actual = df_reciente['temperatura'].iloc[-1]
    temp_promedio = df_reciente['temperatura'].mean()
    temp_max_semana = df_reciente['temperatura'].max()
    temp_min_semana = df_reciente['temperatura'].min()
    
    analisis['condiciones']['temperatura'] = {
        "actual": round(temp_actual, 1),
        "promedio_7d": round(temp_promedio, 1),
        "max_7d": round(temp_max_semana, 1),
        "min_7d": round(temp_min_semana, 1),
        "optima": config.get('temp_optima', 20)
    }
    
    # Evaluación de aptitud térmica
    if temp_min_semana < config.get('temp_critica_min', 0):
        analisis['alertas'].append({
            "tipo": "temperatura_critica",
            "nivel": "alto",
            "mensaje": f"Temperatura mínima crítica alcanzada ({temp_min_semana}°C)"
        })
    
    if temp_max_semana > config.get('temp_critica_max', 40):
        analisis['alertas'].append({
            "tipo": "temperatura_extrema",
            "nivel": "alto",
            "mensaje": f"Temperatura máxima crítica alcanzada ({temp_max_semana}°C)"
        })
    
    # Análisis de humedad
    humedad_actual = df_reciente['humedad_relativa'].iloc[-1]
    humedad_promedio = df_reciente['humedad_relativa'].mean()
    
    analisis['condiciones']['humedad'] = {
        "actual": round(humedad_actual, 1),
        "promedio_7d": round(humedad_promedio, 1),
        "optima": config.get('humedad_optima', 65)
    }
    
    # Análisis de precipitación
    precipitacion_total = df_reciente['precipitacion'].sum()
    dias_lluvia = (df_reciente['precipitacion'] > 0.1).sum()
    
    analisis['condiciones']['precipitacion'] = {
        "total_7d": round(precipitacion_total, 1),
        "dias_lluvia": dias_lluvia,
        "promedio_diario": round(precipitacion_total / 7, 1)
    }
    
    # Cálculo de necesidades de riego
    et0_promedio = df_reciente['evapotranspiracion'].mean() if 'evapotranspiracion' in df_reciente else 3.5
    kc = 0.85  # Coeficiente de cultivo (varía según etapa fenológica)
    etc = et0_promedio * kc
    deficit_hidrico = etc * 7 - precipitacion_total
    
    analisis['indices']['necesidad_riego'] = {
        "et0_promedio": round(et0_promedio, 1),
        "etc_diario": round(etc, 1),
        "deficit_semanal": round(max(0, deficit_hidrico), 1),
        "recomendacion_riego_mm": round(max(0, deficit_hidrico) / 0.85, 1)  # Eficiencia 85%
    }
    
    # Análisis de viento
    viento_promedio = df_reciente['velocidad_viento'].mean()
    viento_max = df_reciente['rafaga_viento'].max() if 'rafaga_viento' in df_reciente else df_reciente['velocidad_viento'].max()
    
    analisis['condiciones']['viento'] = {
        "promedio": round(viento_promedio, 1),
        "maximo": round(viento_max, 1)
    }
    
    # Generación de recomendaciones específicas
    # 1. Recomendaciones de temperatura
    if temp_promedio < config.get('temp_min', 15):
        analisis['recomendaciones'].append({
            "categoria": "temperatura",
            "prioridad": "alta",
            "accion": "Implementar medidas de protección térmica",
            "detalles": [
                "Considerar uso de mantas térmicas",
                "Riego por aspersión antes del amanecer",
                "Verificar sistemas de calefacción si disponibles"
            ]
        })
    
        # 2. Recomendaciones de riego
    if deficit_hidrico > 20:
        analisis['recomendaciones'].append({
            "categoria": "riego",
            "prioridad": "alta",
            "accion": f"Aplicar riego de {round(deficit_hidrico/0.85, 1)} mm",
            "detalles": [
                f"Dividir en {max(2, int(deficit_hidrico/10))} aplicaciones",
                "Preferir riego en horas de menor evaporación",
                "Verificar uniformidad del sistema de riego",
                "Monitorear humedad del suelo a 30 y 60 cm"
            ]
        })
    elif deficit_hidrico > 10:
        analisis['recomendaciones'].append({
            "categoria": "riego",
            "prioridad": "media",
            "accion": f"Riego moderado de {round(deficit_hidrico/0.85, 1)} mm",
            "detalles": [
                "Aplicar en 1-2 riegos semanales",
                "Ajustar según estado fenológico del cultivo"
            ]
        })
    
    # 3. Recomendaciones de humedad
    if humedad_promedio > config.get('humedad_max', 80):
        analisis['recomendaciones'].append({
            "categoria": "humedad",
            "prioridad": "alta",
            "accion": "Mejorar ventilación y prevenir enfermedades",
            "detalles": [
                "Aumentar espaciamiento entre plantas si es posible",
                "Aplicar fungicidas preventivos",
                "Evitar riegos por aspersión",
                "Monitorear aparición de hongos"
            ]
        })
    
    # 4. Recomendaciones fenológicas
    mes_actual = fecha_analisis.month
    
    if mes_actual in config.get('meses_siembra', []):
        analisis['recomendaciones'].append({
            "categoria": "siembra",
            "prioridad": "alta",
            "accion": "Período óptimo para siembra",
            "detalles": [
                "Preparar suelo con 15 días de anticipación",
                "Verificar pronóstico de 10 días",
                "Asegurar disponibilidad de agua para riego inicial",
                f"Temperatura del suelo debe ser > {config.get('temp_min', 15)}°C"
            ]
        })
    
    if mes_actual in config.get('meses_floracion', []):
        analisis['recomendaciones'].append({
            "categoria": "floracion",
            "prioridad": "alta",
            "accion": "Cuidados especiales en floración",
            "detalles": [
                "Evitar estrés hídrico",
                "Proteger de vientos fuertes",
                "Monitorear polinizadores",
                "Aplicar bioestimulantes si necesario"
            ]
        })
    
    if mes_actual in config.get('meses_cosecha', []):
        analisis['recomendaciones'].append({
            "categoria": "cosecha",
            "prioridad": "alta",
            "accion": "Preparar para cosecha",
            "detalles": [
                "Monitorear índices de madurez",
                "Planificar logística de cosecha",
                "Evitar cosecha con humedad alta",
                "Verificar pronóstico para ventana de cosecha"
            ]
        })
    
    # 5. Análisis de riesgos específicos
    # Riesgo de heladas
    if 'probabilidad_helada' in df_reciente:
        prob_helada_max = df_reciente['probabilidad_helada'].max()
        if prob_helada_max > 0.5:
            analisis['alertas'].append({
                "tipo": "helada",
                "nivel": "alto" if prob_helada_max > 0.7 else "medio",
                "mensaje": f"Probabilidad de helada: {prob_helada_max*100:.0f}%",
                "acciones": [
                    "Activar sistemas antiheladas",
                    "Preparar quemadores o calefactores",
                    "Riego por aspersión preventivo",
                    "Cubrir cultivos sensibles"
                ]
            })
    
    # Riesgo de enfermedades
    if humedad_promedio > 70 and 15 <= temp_promedio <= 25:
        riesgo_enfermedad = 0.8
        analisis['indices']['riesgo_enfermedades'] = {
            "nivel": riesgo_enfermedad,
            "condiciones_favorables": True,
            "enfermedades_probables": [
                "Botrytis (moho gris)",
                "Mildiu",
                "Antracnosis",
                "Alternaria"
            ]
        }
        
        analisis['recomendaciones'].append({
            "categoria": "fitosanidad",
            "prioridad": "alta",
            "accion": "Implementar programa preventivo fitosanitario",
            "detalles": [
                "Aplicar fungicidas preventivos",
                "Mejorar drenaje del suelo",
                "Eliminar material vegetal infectado",
                "Aumentar ventilación en el cultivo"
            ]
        })
    
    # 6. Índices agronómicos calculados
    # Grados día acumulados
    if 'gdd_base10' in df_reciente:
        gdd_acumulado = df_reciente['gdd_base10'].sum()
        analisis['indices']['grados_dia'] = {
            "acumulado_7d": round(gdd_acumulado, 1),
            "promedio_diario": round(gdd_acumulado/7, 1),
            "requerimiento_total": config.get('grados_dia_desarrollo', 2000)
        }
    
    # Horas frío (para frutales)
    if cultivo in ['palta', 'citricos', 'uva_mesa']:
        horas_frio = (df_reciente['temperatura'] < 7).sum()
        analisis['indices']['horas_frio'] = {
            "acumulado_7d": horas_frio,
            "requerimiento_anual": 250 if cultivo == 'palta' else 0
        }
    
    return analisis

def generar_reporte_detallado_cultivo(analisis, formato='completo'):
    """Generar reporte detallado para un cultivo específico"""
    
    reporte = []
    reporte.append(f"\n{'='*80}")
    reporte.append(f"REPORTE AGROMETEOROLÓGICO - {analisis['cultivo'].upper()}")
    reporte.append(f"Fecha: {analisis['fecha_analisis'].strftime('%Y-%m-%d %H:%M')}")
    reporte.append(f"{'='*80}\n")
    
    # Condiciones actuales
    reporte.append("📊 CONDICIONES METEOROLÓGICAS")
    reporte.append("-"*40)
    
    cond = analisis['condiciones']
    reporte.append(f"Temperatura:")
    reporte.append(f"  • Actual: {cond['temperatura']['actual']}°C")
    reporte.append(f"  • Promedio 7 días: {cond['temperatura']['promedio_7d']}°C")
    reporte.append(f"  • Rango: {cond['temperatura']['min_7d']} - {cond['temperatura']['max_7d']}°C")
    reporte.append(f"  • Óptima para cultivo: {cond['temperatura']['optima']}°C")
    
    reporte.append(f"\nHumedad:")
    reporte.append(f"  • Actual: {cond['humedad']['actual']}%")
    reporte.append(f"  • Promedio 7 días: {cond['humedad']['promedio_7d']}%")
    
    reporte.append(f"\nPrecipitación:")
    reporte.append(f"  • Total última semana: {cond['precipitacion']['total_7d']} mm")
    reporte.append(f"  • Días con lluvia: {cond['precipitacion']['dias_lluvia']}")
    
    # Alertas
    if analisis['alertas']:
        reporte.append(f"\n⚠️ ALERTAS ACTIVAS")
        reporte.append("-"*40)
        for alerta in analisis['alertas']:
            icono = "🔴" if alerta['nivel'] == 'alto' else "🟡"
            reporte.append(f"{icono} {alerta['mensaje']}")
            if 'acciones' in alerta:
                for accion in alerta['acciones']:
                    reporte.append(f"    → {accion}")
    
    # Índices agronómicos
    if analisis['indices']:
        reporte.append(f"\n📈 ÍNDICES AGRONÓMICOS")
        reporte.append("-"*40)
        
        if 'necesidad_riego' in analisis['indices']:
            riego = analisis['indices']['necesidad_riego']
            reporte.append(f"Necesidades hídricas:")
            reporte.append(f"  • ET0 promedio: {riego['et0_promedio']} mm/día")
            reporte.append(f"  • ETc cultivo: {riego['etc_diario']} mm/día")
            reporte.append(f"  • Déficit semanal: {riego['deficit_semanal']} mm")
            reporte.append(f"  • Riego recomendado: {riego['recomendacion_riego_mm']} mm")
        
        if 'grados_dia' in analisis['indices']:
            gdd = analisis['indices']['grados_dia']
            reporte.append(f"\nGrados día (base 10°C):")
            reporte.append(f"  • Acumulado 7 días: {gdd['acumulado_7d']}°C")
            reporte.append(f"  • Promedio diario: {gdd['promedio_diario']}°C")
        
        if 'riesgo_enfermedades' in analisis['indices']:
            riesgo = analisis['indices']['riesgo_enfermedades']
            nivel_texto = "Alto" if riesgo['nivel'] > 0.7 else "Medio" if riesgo['nivel'] > 0.4 else "Bajo"
            reporte.append(f"\nRiesgo de enfermedades: {nivel_texto}")
            if riesgo['condiciones_favorables']:
                reporte.append("  Enfermedades probables:")
                for enfermedad in riesgo['enfermedades_probables']:
                    reporte.append(f"    • {enfermedad}")
    
    # Recomendaciones
    if analisis['recomendaciones']:
        reporte.append(f"\n💡 RECOMENDACIONES")
        reporte.append("-"*40)
        
        # Ordenar por prioridad
        recs_alta = [r for r in analisis['recomendaciones'] if r['prioridad'] == 'alta']
        recs_media = [r for r in analisis['recomendaciones'] if r['prioridad'] == 'media']
        
        if recs_alta:
            reporte.append("Prioridad ALTA:")
            for rec in recs_alta:
                reporte.append(f"\n🔴 {rec['categoria'].upper()}: {rec['accion']}")
                for detalle in rec['detalles']:
                    reporte.append(f"   • {detalle}")
        
        if recs_media:
            reporte.append("\nPrioridad MEDIA:")
            for rec in recs_media:
                reporte.append(f"\n🟡 {rec['categoria'].upper()}: {rec['accion']}")
                for detalle in rec['detalles']:
                    reporte.append(f"   • {detalle}")
    
    return "\n".join(reporte)

In [8]:
def preparar_datos_pronostico(df_historico):
    """Preparar datos para modelos de pronóstico"""
    
    df = df_historico.copy()
    
    # Agregar características temporales
    df['hora'] = df['fecha_hora'].dt.hour
    df['dia'] = df['fecha_hora'].dt.day
    df['mes'] = df['fecha_hora'].dt.month
    df['dia_semana'] = df['fecha_hora'].dt.dayofweek
    df['dia_año'] = df['fecha_hora'].dt.dayofyear
    
    # Agregar características cíclicas
    df['hora_sin'] = np.sin(2 * np.pi * df['hora'] / 24)
    df['hora_cos'] = np.cos(2 * np.pi * df['hora'] / 24)
    df['mes_sin'] = np.sin(2 * np.pi * df['mes'] / 12)
    df['mes_cos'] = np.cos(2 * np.pi * df['mes'] / 12)
    
    # Agregar lags
    for lag in [1, 6, 12, 24, 48, 72]:
        df[f'temp_lag_{lag}'] = df['temperatura'].shift(lag)
        df[f'humedad_lag_{lag}'] = df['humedad_relativa'].shift(lag)
    
    # Agregar medias móviles
    for window in [6, 12, 24, 48]:
        df[f'temp_ma_{window}'] = df['temperatura'].rolling(window).mean()
        df[f'humedad_ma_{window}'] = df['humedad_relativa'].rolling(window).mean()
    
    return df.dropna()

def entrenar_modelo_prophet(df_datos, variable='temperatura'):
    """Entrenar modelo Prophet para pronósticos"""
    
    # Preparar datos para Prophet
    df_prophet = df_datos[['fecha_hora', variable]].copy()
    df_prophet.columns = ['ds', 'y']
    
    # Configurar modelo con parámetros optimizados para Chile
    modelo = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True,
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05,
        interval_width=0.95,
        n_changepoints=25
    )
    
        # Agregar regresores adicionales si están disponibles
    if 'humedad_relativa' in df_datos.columns and variable != 'humedad_relativa':
        df_prophet['humedad'] = df_datos['humedad_relativa']
        modelo.add_regressor('humedad')
    
    if 'presion_atmosferica' in df_datos.columns:
        df_prophet['presion'] = df_datos['presion_atmosferica']
        modelo.add_regressor('presion')
    
    if 'velocidad_viento' in df_datos.columns:
        df_prophet['viento'] = df_datos['velocidad_viento']
        modelo.add_regressor('viento')
    
    # Agregar feriados chilenos
    feriados_chile = pd.DataFrame({
        'holiday': 'feriado',
        'ds': pd.to_datetime([
            '2024-01-01', '2024-03-29', '2024-03-30', '2024-05-01',
            '2024-05-21', '2024-06-20', '2024-07-16', '2024-08-15',
            '2024-09-18', '2024-09-19', '2024-10-12', '2024-10-31',
            '2024-11-01', '2024-12-08', '2024-12-25'
        ]),
        'lower_window': 0,
        'upper_window': 1,
    })
    
    # Entrenar modelo
    with suppress_stdout_stderr():
        modelo.fit(df_prophet)
    
    return modelo, df_prophet

def generar_pronostico_multivariable(df_datos, horizonte_dias=7, incluir_incertidumbre=True):
    """Generar pronósticos para múltiples variables meteorológicas"""
    
    pronosticos = {}
    modelos = {}
    
    variables_pronostico = [
        'temperatura', 'humedad_relativa', 'precipitacion',
        'velocidad_viento', 'presion_atmosferica'
    ]
    
    for variable in variables_pronostico:
        if variable not in df_datos.columns:
            continue
            
        print(f"🔄 Entrenando modelo para {variable}...")
        
        try:
            # Entrenar modelo Prophet
            modelo, df_prophet = entrenar_modelo_prophet(df_datos, variable)
            
            # Crear dataframe futuro
            future = modelo.make_future_dataframe(
                periods=horizonte_dias * 24, 
                freq='H',
                include_history=True
            )
            
            # Agregar regresores al futuro
            if 'humedad' in df_prophet.columns:
                future['humedad'] = df_prophet['humedad'].iloc[-1]
            if 'presion' in df_prophet.columns:
                future['presion'] = df_prophet['presion'].iloc[-1]
            if 'viento' in df_prophet.columns:
                future['viento'] = df_prophet['viento'].iloc[-1]
            
            # Generar pronóstico
            forecast = modelo.predict(future)
            
            # Guardar resultados
            pronosticos[variable] = forecast
            modelos[variable] = modelo
            
            print(f"✅ Pronóstico completado para {variable}")
            
        except Exception as e:
            print(f"❌ Error en pronóstico de {variable}: {e}")
    
    return pronosticos, modelos

def calcular_probabilidad_helada_ml(df_pronostico, estacion_id):
    """Calcular probabilidad de helada usando ML y múltiples factores"""
    
    probabilidades = []
    
    for idx, row in df_pronostico.iterrows():
        # Factores base
        temp = row.get('yhat', 20)
        temp_lower = row.get('yhat_lower', temp - 2)
        hora = row['ds'].hour
        mes = row['ds'].month
        
        # Modelo probabilístico
        prob_base = 0
        
        # Factor temperatura
        if temp_lower <= -2:
            prob_base = 0.95
        elif temp_lower <= 0:
            prob_base = 0.85
        elif temp_lower <= 2:
            prob_base = 0.70
        elif temp <= 4:
            prob_base = 0.50
        elif temp <= 6:
            prob_base = 0.30
        elif temp <= 8:
            prob_base = 0.15
        else:
            prob_base = 0.05
        
        # Factor horario (heladas más probables en la madrugada)
        factor_hora = 1.0
        if 3 <= hora <= 7:
            factor_hora = 1.3
        elif 0 <= hora <= 2 or 8 <= hora <= 9:
            factor_hora = 1.1
        elif 10 <= hora <= 16:
            factor_hora = 0.5
        
        # Factor estacional
        factor_estacion = 1.0
        if mes in [6, 7, 8]:  # Invierno
            factor_estacion = 1.4
        elif mes in [5, 9]:  # Otoño/Primavera temprana
            factor_estacion = 1.2
        elif mes in [4, 10]:
            factor_estacion = 1.0
        else:
            factor_estacion = 0.7
        
        # Factor geográfico (según estación)
        factor_geo = 1.0
        if estacion_id in ['HJL01', 'NGL01']:  # Zonas más altas
            factor_geo = 1.2
        elif estacion_id in ['LCZ01', 'LMC01']:  # Zonas costeras
            factor_geo = 0.9
        
        # Calcular probabilidad final
        prob_final = min(1.0, prob_base * factor_hora * factor_estacion * factor_geo)
        
        # Clasificar tipo de helada
        tipo_helada = "Sin riesgo"
        intensidad = 0
        
        if prob_final > 0.3:
            if temp_lower <= -2:
                tipo_helada = "Helada negra"
                intensidad = 3
            elif temp_lower <= 0:
                tipo_helada = "Helada blanca fuerte"
                intensidad = 2
            elif temp_lower <= 2:
                tipo_helada = "Helada blanca débil"
                intensidad = 1
            else:
                tipo_helada = "Helada advectiva"
                intensidad = 1
        
        probabilidades.append({
            'fecha_hora': row['ds'],
            'temperatura_esperada': round(temp, 1),
            'temperatura_minima': round(temp_lower, 1),
            'probabilidad': round(prob_final, 3),
            'tipo_helada': tipo_helada,
            'intensidad': intensidad,
            'hora': hora,
            'acciones_recomendadas': generar_acciones_helada(prob_final, tipo_helada)
        })
    
    return pd.DataFrame(probabilidades)

def generar_acciones_helada(probabilidad, tipo_helada):
    """Generar acciones específicas según probabilidad y tipo de helada"""
    
    acciones = []
    
    if probabilidad > 0.7:
        acciones.extend([
            "🚨 ACTIVAR PROTOCOLO DE EMERGENCIA ANTIHELADAS",
            "Encender sistemas de calefacción o quemadores",
            "Iniciar riego por aspersión continuo",
            "Cubrir cultivos sensibles con mantas térmicas",
            "Aplicar productos antiheladas (sales potásicas)",
            "Movilizar personal para monitoreo nocturno"
        ])
    elif probabilidad > 0.5:
        acciones.extend([
            "⚠️ ALERTA NARANJA - Preparar sistemas antiheladas",
            "Verificar funcionamiento de aspersores",
            "Preparar combustible para quemadores",
            "Revisar pronóstico cada 3 horas",
            "Tener personal en alerta"
        ])
    elif probabilidad > 0.3:
        acciones.extend([
            "📢 PRECAUCIÓN - Monitorear condiciones",
            "Revisar pronóstico actualizado",
            "Verificar sistemas de protección",
            "Considerar riego preventivo al atardecer"
        ])
    
    # Acciones específicas por tipo
    if tipo_helada == "Helada negra":
        acciones.append("⚫ Helada negra: Máxima protección requerida")
        acciones.append("Combinar todos los métodos disponibles")
    elif tipo_helada == "Helada blanca fuerte":
        acciones.append("⚪ Helada blanca: Riego por aspersión efectivo")
        acciones.append("Mantener follaje húmedo durante el evento")
    
    return acciones

def generar_pronostico_integrado(df_datos, estacion):
    """Generar pronóstico integrado con todos los modelos"""
    
    print(f"\n📊 Generando pronóstico integrado para {estacion['nombre']}")
    
    # Filtrar datos de la estación
    df_estacion = df_datos[df_datos['estacion_id'] == estacion['id']].copy()
    
    if df_estacion.empty:
        print(f"❌ Sin datos para {estacion['nombre']}")
        return None
    
    resultados = {
        'estacion': estacion,
        'fecha_generacion': datetime.now(),
        'pronosticos': {},
        'alertas': [],
        'recomendaciones': []
    }
    
    # 1. Pronóstico a corto plazo (48 horas)
    pronosticos_48h, modelos = generar_pronostico_multivariable(
        df_estacion, 
        horizonte_dias=2
    )
    
    # 2. Pronóstico a mediano plazo (7 días)
    pronosticos_7d, _ = generar_pronostico_multivariable(
        df_estacion, 
        horizonte_dias=7
    )
    
    # 3. Pronóstico a largo plazo (30 días)
    pronosticos_30d, _ = generar_pronostico_multivariable(
        df_estacion, 
        horizonte_dias=30
    )
    
    # 4. Análisis de heladas
    if 'temperatura' in pronosticos_48h:
        df_heladas = calcular_probabilidad_helada_ml(
            pronosticos_48h['temperatura'][pronosticos_48h['temperatura']['ds'] > datetime.now()],
            estacion['id']
        )
        
        # Identificar alertas de helada
        heladas_criticas = df_heladas[df_heladas['probabilidad'] > 0.5]
        if not heladas_criticas.empty:
            for _, helada in heladas_criticas.iterrows():
                resultados['alertas'].append({
                    'tipo': 'helada',
                    'fecha': helada['fecha_hora'],
                    'probabilidad': helada['probabilidad'],
                    'tipo_helada': helada['tipo_helada'],
                    'acciones': helada['acciones_recomendadas']
                })
    
    # 5. Análisis de eventos extremos
    for variable in ['temperatura', 'precipitacion', 'velocidad_viento']:
        if variable not in pronosticos_7d:
            continue
            
        df_var = pronosticos_7d[variable]
        df_futuro = df_var[df_var['ds'] > datetime.now()]
        
        if variable == 'temperatura':
            # Detectar olas de calor
            dias_calor = df_futuro[df_futuro['yhat'] > 32]
            if len(dias_calor) >= 3:
                resultados['alertas'].append({
                    'tipo': 'ola_calor',
                    'fecha_inicio': dias_calor.iloc[0]['ds'],
                    'duracion_dias': len(dias_calor) / 24,
                    'temp_maxima': dias_calor['yhat_upper'].max()
                })
        
        elif variable == 'precipitacion':
            # Detectar lluvias intensas
            lluvias_intensas = df_futuro[df_futuro['yhat'] > 25]
            for _, lluvia in lluvias_intensas.iterrows():
                if lluvia['yhat'] > 50:
                    nivel = 'extrema'
                elif lluvia['yhat'] > 35:
                    nivel = 'intensa'
                else:
                    nivel = 'moderada'
                
                resultados['alertas'].append({
                    'tipo': 'precipitacion',
                    'nivel': nivel,
                    'fecha': lluvia['ds'],
                    'cantidad_mm': lluvia['yhat']
                })
    
    # 6. Guardar resultados
    resultados['pronosticos'] = {
        '48_horas': pronosticos_48h,
        '7_dias': pronosticos_7d,
        '30_dias': pronosticos_30d,
        'heladas': df_heladas if 'df_heladas' in locals() else None
    }
    
    return resultados

# Funciones auxiliares
class suppress_stdout_stderr:
    """Contexto para suprimir output de Prophet"""
    def __enter__(self):
        self.old_stdout = sys.stdout
        self.old_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stderr.close()
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr

In [13]:
import psycopg2
import os
from datetime import datetime

# Configuración de la base de datos
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'mip_quillota'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', '1478')
}

def corregir_bd_con_manejo_errores():
    """Corrige la BD manejando casos especiales como datos existentes"""
    
    print("🔧 CORRECCIÓN DE BASE DE DATOS CON MANEJO DE CASOS ESPECIALES")
    print("="*70)
    
    conn = None
    
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        conn.autocommit = True  # Para evitar problemas de transacciones
        cursor = conn.cursor()
        print(f"✅ Conectado a '{DB_CONFIG['database']}' (modo autocommit)")
        
        # 1. CORREGIR TABLA ALERTAS
        print("\n📋 1. Corrigiendo tabla 'alertas'...")
        try:
            columnas_alertas = [
                "ALTER TABLE alertas ADD COLUMN IF NOT EXISTS descripcion TEXT",
                "ALTER TABLE alertas ADD COLUMN IF NOT EXISTS fecha_inicio TIMESTAMP DEFAULT CURRENT_TIMESTAMP",
                "ALTER TABLE alertas ADD COLUMN IF NOT EXISTS fecha_fin TIMESTAMP",
                "ALTER TABLE alertas ADD COLUMN IF NOT EXISTS estacion_id INTEGER",
                "ALTER TABLE alertas ADD COLUMN IF NOT EXISTS estado VARCHAR(50) DEFAULT 'activa'",
                "ALTER TABLE alertas ADD COLUMN IF NOT EXISTS cultivo_afectado VARCHAR(100)",
                "ALTER TABLE alertas ADD COLUMN IF NOT EXISTS acciones_recomendadas TEXT"
            ]
            
            for sql in columnas_alertas:
                try:
                    cursor.execute(sql)
                    print(f"   ✅ Ejecutado: {sql.split('EXISTS')[1].split()[0]}")
                except Exception as e:
                    print(f"   ⚠️ {sql.split('EXISTS')[1].split()[0]}: {str(e)[:50]}")
                    
        except Exception as e:
            print(f"   ❌ Error en alertas: {e}")
        
        # 2. CORREGIR TABLA PRONOSTICOS
        print("\n📋 2. Corrigiendo tabla 'pronosticos'...")
        try:
            columnas_pronosticos = [
                "ALTER TABLE pronosticos ADD COLUMN IF NOT EXISTS fecha_prediccion TIMESTAMP",
                "ALTER TABLE pronosticos ADD COLUMN IF NOT EXISTS temperatura_max_pred FLOAT",
                "ALTER TABLE pronosticos ADD COLUMN IF NOT EXISTS temperatura_min_pred FLOAT",
                "ALTER TABLE pronosticos ADD COLUMN IF NOT EXISTS humedad_pred FLOAT",
                "ALTER TABLE pronosticos ADD COLUMN IF NOT EXISTS modelo_usado VARCHAR(100)",
                "ALTER TABLE pronosticos ADD COLUMN IF NOT EXISTS confianza FLOAT"
            ]
            
            for sql in columnas_pronosticos:
                try:
                    cursor.execute(sql)
                    print(f"   ✅ Ejecutado: {sql.split('EXISTS')[1].split()[0]}")
                except Exception as e:
                    print(f"   ⚠️ {sql.split('EXISTS')[1].split()[0]}: {str(e)[:50]}")
                    
        except Exception as e:
            print(f"   ❌ Error en pronosticos: {e}")
        
        # 3. CORREGIR TABLA DATOS_METEOROLOGICOS - CASO ESPECIAL
        print("\n📋 3. Corrigiendo tabla 'datos_meteorologicos' (caso especial)...")
        try:
            # Primero verificar si fecha_hora existe
            cursor.execute("""
                SELECT column_name 
                FROM information_schema.columns 
                WHERE table_name = 'datos_meteorologicos' 
                AND column_name = 'fecha_hora'
            """)
            
            if not cursor.fetchone():
                print("   ⚠️ Columna 'fecha_hora' no existe")
                
                # Verificar si hay datos en la tabla
                cursor.execute("SELECT COUNT(*) FROM datos_meteorologicos")
                count = cursor.fetchone()[0]
                
                if count > 0:
                    print(f"   ℹ️ La tabla tiene {count} registros")
                    
                    # Buscar columnas de fecha alternativas
                    cursor.execute("""
                        SELECT column_name 
                        FROM information_schema.columns 
                        WHERE table_name = 'datos_meteorologicos' 
                        AND (column_name LIKE '%fecha%' OR column_name LIKE '%time%' OR column_name LIKE '%date%')
                    """)
                    columnas_fecha = cursor.fetchall()
                    
                    if columnas_fecha:
                        fecha_col = columnas_fecha[0][0]
                        print(f"   ℹ️ Encontrada columna de fecha: '{fecha_col}'")
                        
                        # Opción 1: Renombrar columna existente
                        try:
                            cursor.execute(f"ALTER TABLE datos_meteorologicos RENAME COLUMN {fecha_col} TO fecha_hora")
                            print(f"   ✅ Columna '{fecha_col}' renombrada a 'fecha_hora'")
                        except:
                            # Opción 2: Crear nueva columna y copiar datos
                            cursor.execute("ALTER TABLE datos_meteorologicos ADD COLUMN fecha_hora TIMESTAMP")
                            cursor.execute(f"UPDATE datos_meteorologicos SET fecha_hora = {fecha_col}")
                            print(f"   ✅ Columna 'fecha_hora' creada y datos copiados desde '{fecha_col}'")
                    else:
                        # No hay columna de fecha, crear con valor por defecto
                        cursor.execute("ALTER TABLE datos_meteorologicos ADD COLUMN fecha_hora TIMESTAMP DEFAULT CURRENT_TIMESTAMP")
                        cursor.execute("UPDATE datos_meteorologicos SET fecha_hora = CURRENT_TIMESTAMP WHERE fecha_hora IS NULL")
                        print("   ✅ Columna 'fecha_hora' creada con timestamp actual")
                else:
                    # Tabla vacía, crear columna normalmente
                    cursor.execute("ALTER TABLE datos_meteorologicos ADD COLUMN fecha_hora TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP")
                    print("   ✅ Columna 'fecha_hora' creada (tabla vacía)")
            else:
                print("   ✅ Columna 'fecha_hora' ya existe")
                
        except Exception as e:
            print(f"   ❌ Error en datos_meteorologicos: {e}")
        
        # 4. CREAR O VERIFICAR TABLA ANALISIS_HISTORICOS
        print("\n📋 4. Verificando tabla 'analisis_historicos'...")
        try:
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS analisis_historicos (
                    id SERIAL PRIMARY KEY,
                    estacion_id INTEGER,
                    mes INTEGER,
                    año INTEGER,
                    temp_promedio FLOAT,
                    temp_max_promedio FLOAT,
                    temp_min_promedio FLOAT,
                    precipitacion_total FLOAT,
                    dias_helada INTEGER,
                    dias_lluvia INTEGER,
                    humedad_promedio FLOAT,
                    evapotranspiracion_total FLOAT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    UNIQUE(estacion_id, mes, año)
                )
            """)
            print("   ✅ Tabla 'analisis_historicos' verificada/creada")
        except Exception as e:
            print(f"   ❌ Error en analisis_historicos: {e}")
        
        # 5. VERIFICACIÓN FINAL
        print("\n📊 VERIFICACIÓN FINAL DE COLUMNAS CRÍTICAS:")
        print("-"*60)
        
        verificaciones = [
            ('alertas', ['descripcion', 'nivel_alerta', 'estacion_id']),
            ('pronosticos', ['fecha_prediccion', 'estacion_id']),
            ('datos_meteorologicos', ['fecha_hora']),
        ]
        
        for tabla, columnas in verificaciones:
            print(f"\n{tabla}:")
            for columna in columnas:
                cursor.execute(f"""
                    SELECT column_name, data_type, is_nullable
                    FROM information_schema.columns 
                    WHERE table_name = '{tabla}' 
                    AND column_name = '{columna}'
                """)
                resultado = cursor.fetchone()
                if resultado:
                    nullable = "NULL" if resultado[2] == 'YES' else "NOT NULL"
                    print(f"   ✅ {columna}: {resultado[1]} {nullable}")
                else:
                    print(f"   ❌ {columna}: NO EXISTE")
        
        cursor.close()
        print("\n✅ PROCESO COMPLETADO")
        return True
        
    except Exception as e:
        print(f"\n❌ ERROR GENERAL: {e}")
        return False
        
    finally:
        if conn:
            conn.close()

# Script para limpiar y reiniciar tablas si es necesario
def limpiar_tablas_problematicas():
    """Opción nuclear: recrear tablas problemáticas"""
    print("\n⚠️ OPCIÓN DE RECREACIÓN DE TABLAS")
    print("="*60)
    
    respuesta = input("¿Desea RECREAR las tablas problemáticas? Esto ELIMINARÁ todos los datos (s/n): ")
    
    if respuesta.lower() != 's':
        print("Operación cancelada")
        return
    
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        conn.autocommit = True
        cursor = conn.cursor()
        
        # Eliminar y recrear tablas
        tablas_recrear = ['alertas', 'pronosticos', 'datos_meteorologicos', 'analisis_historicos']
        
        for tabla in tablas_recrear:
            print(f"\n🔄 Recreando tabla '{tabla}'...")
            cursor.execute(f"DROP TABLE IF EXISTS {tabla} CASCADE")
            print(f"   ✅ Tabla eliminada")
        
        # Recrear con estructura correcta
        cursor.execute("""
            CREATE TABLE datos_meteorologicos (
                id SERIAL PRIMARY KEY,
                estacion_id INTEGER,
                estacion_nombre VARCHAR(255),
                fecha_hora TIMESTAMP NOT NULL,
                temperatura FLOAT,
                temperatura_aparente FLOAT,
                humedad_relativa FLOAT,
                precipitacion FLOAT,
                precipitacion_probabilidad FLOAT,
                velocidad_viento FLOAT,
                direccion_viento FLOAT,
                rafaga_viento FLOAT,
                presion_atmosferica FLOAT,
                radiacion_solar FLOAT,
                evapotranspiracion FLOAT,
                indice_uv FLOAT,
                punto_rocio FLOAT,
                temperatura_max FLOAT,
                temperatura_min FLOAT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        cursor.execute("""
            CREATE TABLE pronosticos (
                id SERIAL PRIMARY KEY,
                estacion_id INTEGER,
                fecha_pronostico TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                fecha_prediccion TIMESTAMP,
                tipo_pronostico VARCHAR(50),
                temperatura_pred FLOAT,
                temperatura_max_pred FLOAT,
                temperatura_min_pred FLOAT,
                precipitacion_pred FLOAT,
                humedad_pred FLOAT,
                probabilidad_helada FLOAT,
                modelo_usado VARCHAR(100),
                confianza FLOAT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        cursor.execute("""
            CREATE TABLE alertas (
                id SERIAL PRIMARY KEY,
                tipo_alerta VARCHAR(100),
                nivel_alerta VARCHAR(50) DEFAULT 'medio',
                mensaje TEXT,
                descripcion TEXT,
                fecha_inicio TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                fecha_fin TIMESTAMP,
                estacion_id INTEGER,
                estado VARCHAR(50) DEFAULT 'activa',
                cultivo_afectado VARCHAR(100),
                acciones_recomendadas TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        cursor.execute("""
            CREATE TABLE analisis_historicos (
                id SERIAL PRIMARY KEY,
                estacion_id INTEGER,
                mes INTEGER,
                año INTEGER,
                temp_promedio FLOAT,
                temp_max_promedio FLOAT,
                temp_min_promedio FLOAT,
                precipitacion_total FLOAT,
                dias_helada INTEGER,
                dias_lluvia INTEGER,
                humedad_promedio FLOAT,
                evapotranspiracion_total FLOAT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                UNIQUE(estacion_id, mes, año)
            )
        """)
        
        print("\n✅ Todas las tablas recreadas exitosamente")
        
        cursor.close()
        conn.close()
        
    except Exception as e:
        print(f"❌ Error recreando tablas: {e}")

# Ejecutar
if __name__ == "__main__":
    print("🚀 INICIANDO CORRECCIÓN DE BASE DE DATOS")
    print("="*70)
    
    # Intentar corrección normal primero
    exito = corregir_bd_con_manejo_errores()
    
    if not exito:
        print("\n" + "="*70)
        print("❌ La corrección normal falló")
        print("📌 Puede intentar la opción de recrear las tablas (perderá los datos)")
        limpiar_tablas_problematicas()
    else:
        print("\n" + "="*70)
        print("✅ BASE DE DATOS LISTA")
        print("📌 Puede ejecutar nuevamente el código de guardado")

🚀 INICIANDO CORRECCIÓN DE BASE DE DATOS
🔧 CORRECCIÓN DE BASE DE DATOS CON MANEJO DE CASOS ESPECIALES
✅ Conectado a 'mip_quillota' (modo autocommit)

📋 1. Corrigiendo tabla 'alertas'...
   ✅ Ejecutado: descripcion
   ✅ Ejecutado: fecha_inicio
   ✅ Ejecutado: fecha_fin
   ✅ Ejecutado: estacion_id
   ✅ Ejecutado: estado
   ✅ Ejecutado: cultivo_afectado
   ✅ Ejecutado: acciones_recomendadas

📋 2. Corrigiendo tabla 'pronosticos'...
   ✅ Ejecutado: fecha_prediccion
   ✅ Ejecutado: temperatura_max_pred
   ✅ Ejecutado: temperatura_min_pred
   ✅ Ejecutado: humedad_pred
   ✅ Ejecutado: modelo_usado
   ✅ Ejecutado: confianza

📋 3. Corrigiendo tabla 'datos_meteorologicos' (caso especial)...
   ⚠️ Columna 'fecha_hora' no existe
   ℹ️ La tabla tiene 2 registros
   ℹ️ Encontrada columna de fecha: 'fecha'
   ✅ Columna 'fecha' renombrada a 'fecha_hora'

📋 4. Verificando tabla 'analisis_historicos'...
   ✅ Tabla 'analisis_historicos' verificada/creada

📊 VERIFICACIÓN FINAL DE COLUMNAS CRÍTICAS:
--------

In [15]:
# CÓDIGO DE CORRECCIÓN DE ERRORES DE BASE DE DATOS
import psycopg2
from psycopg2 import sql
import os

# Configuración de conexión
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'mip_quillota'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', '1478')
}

def corregir_estructura_bd():
    """Corrige los problemas de estructura en la base de datos"""
    
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        print("✅ Conectado a la base de datos")
        
        correcciones_aplicadas = []
        
        # 1. Modificar columna estacion_id en pronosticos para aceptar VARCHAR
        print("\n🔧 Corrigiendo tabla 'pronosticos'...")
        try:
            cursor.execute("""
                ALTER TABLE pronosticos 
                ALTER COLUMN estacion_id TYPE VARCHAR(10) 
                USING estacion_id::VARCHAR
            """)
            correcciones_aplicadas.append("✅ Columna estacion_id en pronosticos cambiada a VARCHAR")
        except Exception as e:
            if "already exists" not in str(e):
                print(f"⚠️ Advertencia en pronosticos: {e}")
        
        # 2. Modificar columna estacion_id en alertas para aceptar VARCHAR
        print("\n🔧 Corrigiendo tabla 'alertas'...")
        try:
            cursor.execute("""
                ALTER TABLE alertas 
                ALTER COLUMN estacion_id TYPE VARCHAR(10) 
                USING estacion_id::VARCHAR
            """)
            correcciones_aplicadas.append("✅ Columna estacion_id en alertas cambiada a VARCHAR")
        except Exception as e:
            if "already exists" not in str(e):
                print(f"⚠️ Advertencia en alertas: {e}")
        
        # 3. Modificar columna estacion_id en datos_meteorologicos para aceptar VARCHAR
        print("\n🔧 Corrigiendo tabla 'datos_meteorologicos'...")
        try:
            cursor.execute("""
                ALTER TABLE datos_meteorologicos 
                ALTER COLUMN estacion_id TYPE VARCHAR(10) 
                USING estacion_id::VARCHAR
            """)
            correcciones_aplicadas.append("✅ Columna estacion_id en datos_meteorologicos cambiada a VARCHAR")
        except Exception as e:
            if "already exists" not in str(e):
                print(f"⚠️ Advertencia en datos_meteorologicos: {e}")
        
        # 4. Modificar columna estacion_id en recomendaciones_cultivos para aceptar VARCHAR
        print("\n🔧 Corrigiendo tabla 'recomendaciones_cultivos'...")
        try:
            cursor.execute("""
                ALTER TABLE recomendaciones_cultivos 
                ALTER COLUMN estacion_id TYPE VARCHAR(10) 
                USING estacion_id::VARCHAR
            """)
            correcciones_aplicadas.append("✅ Columna estacion_id en recomendaciones_cultivos cambiada a VARCHAR")
        except Exception as e:
            if "already exists" not in str(e):
                print(f"⚠️ Advertencia en recomendaciones_cultivos: {e}")
        
        # 5. Modificar columna estacion_id en analisis_historicos para aceptar VARCHAR
        print("\n🔧 Corrigiendo tabla 'analisis_historicos'...")
        try:
            cursor.execute("""
                ALTER TABLE analisis_historicos 
                ALTER COLUMN estacion_id TYPE VARCHAR(10) 
                USING estacion_id::VARCHAR
            """)
            correcciones_aplicadas.append("✅ Columna estacion_id en analisis_historicos cambiada a VARCHAR")
        except Exception as e:
            if "already exists" not in str(e):
                print(f"⚠️ Advertencia en analisis_historicos: {e}")
        
        # 6. Agregar columna updated_at a analisis_historicos si no existe
        print("\n🔧 Agregando columna updated_at...")
        try:
            cursor.execute("""
                ALTER TABLE analisis_historicos 
                ADD COLUMN IF NOT EXISTS updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            """)
            correcciones_aplicadas.append("✅ Columna updated_at agregada a analisis_historicos")
        except Exception as e:
            print(f"⚠️ Advertencia al agregar updated_at: {e}")
        
        # 7. Crear trigger para actualizar updated_at automáticamente
        print("\n🔧 Creando trigger para updated_at...")
        try:
            # Primero crear la función si no existe
            cursor.execute("""
                CREATE OR REPLACE FUNCTION update_updated_at_column()
                RETURNS TRIGGER AS $$
                BEGIN
                    NEW.updated_at = CURRENT_TIMESTAMP;
                    RETURN NEW;
                END;
                $$ language 'plpgsql';
            """)
            
            # Luego crear el trigger
            cursor.execute("""
                DROP TRIGGER IF EXISTS update_analisis_historicos_updated_at ON analisis_historicos;
                
                CREATE TRIGGER update_analisis_historicos_updated_at 
                BEFORE UPDATE ON analisis_historicos 
                FOR EACH ROW 
                EXECUTE FUNCTION update_updated_at_column();
            """)
            correcciones_aplicadas.append("✅ Trigger para updated_at creado")
        except Exception as e:
            print(f"⚠️ Advertencia al crear trigger: {e}")
        
        # Aplicar cambios
        conn.commit()
        
        # Mostrar resumen
        print("\n📊 RESUMEN DE CORRECCIONES:")
        print("="*60)
        for correccion in correcciones_aplicadas:
            print(f"  {correccion}")
        print("="*60)
        
        # Verificar estructura actual
        print("\n🔍 Verificando estructura actual de las tablas...")
        
        tablas = ['pronosticos', 'alertas', 'datos_meteorologicos', 
                  'recomendaciones_cultivos', 'analisis_historicos']
        
        for tabla in tablas:
            cursor.execute(f"""
                SELECT column_name, data_type 
                FROM information_schema.columns 
                WHERE table_name = '{tabla}' 
                AND column_name IN ('estacion_id', 'updated_at')
                ORDER BY ordinal_position
            """)
            columnas = cursor.fetchall()
            
            print(f"\n📋 Tabla '{tabla}':")
            for col_name, col_type in columnas:
                print(f"   - {col_name}: {col_type}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error general: {e}")
        if conn:
            conn.rollback()
        return False
        
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Ejecutar las correcciones
print("🚀 Iniciando corrección de base de datos...")
if corregir_estructura_bd():
    print("\n✅ Base de datos corregida exitosamente")
    print("\n📌 Ahora puedes volver a ejecutar el código de guardado")
else:
    print("\n❌ Error en la corrección de base de datos")

🚀 Iniciando corrección de base de datos...
✅ Conectado a la base de datos

🔧 Corrigiendo tabla 'pronosticos'...

🔧 Corrigiendo tabla 'alertas'...

🔧 Corrigiendo tabla 'datos_meteorologicos'...

🔧 Corrigiendo tabla 'recomendaciones_cultivos'...

🔧 Corrigiendo tabla 'analisis_historicos'...

🔧 Agregando columna updated_at...

🔧 Creando trigger para updated_at...

📊 RESUMEN DE CORRECCIONES:
  ✅ Columna estacion_id en pronosticos cambiada a VARCHAR
  ✅ Columna estacion_id en alertas cambiada a VARCHAR
  ✅ Columna estacion_id en datos_meteorologicos cambiada a VARCHAR
  ✅ Columna estacion_id en recomendaciones_cultivos cambiada a VARCHAR
  ✅ Columna estacion_id en analisis_historicos cambiada a VARCHAR
  ✅ Columna updated_at agregada a analisis_historicos
  ✅ Trigger para updated_at creado

🔍 Verificando estructura actual de las tablas...

📋 Tabla 'pronosticos':
   - estacion_id: character varying

📋 Tabla 'alertas':
   - estacion_id: character varying

📋 Tabla 'datos_meteorologicos':
   - 

In [17]:
# CÓDIGO PARA CORREGIR RESTRICCIONES NOT NULL
import psycopg2
import os

DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'mip_quillota'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', '1478')
}

def corregir_restricciones_null():
    """Corrige las restricciones NOT NULL en las tablas"""
    
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        print("✅ Conectado a la base de datos")
        
        correcciones = []
        
        # 1. Ver estructura actual de la tabla alertas
        print("\n🔍 Verificando estructura de 'alertas'...")
        cursor.execute("""
            SELECT column_name, is_nullable, column_default
            FROM information_schema.columns
            WHERE table_name = 'alertas'
            AND column_name IN ('nivel', 'nivel_alerta')
            ORDER BY ordinal_position
        """)
        columnas_alertas = cursor.fetchall()
        print("Columnas encontradas:")
        for col in columnas_alertas:
            print(f"  - {col[0]}: nullable={col[1]}, default={col[2]}")
        
        # 2. Modificar restricciones en alertas
        print("\n🔧 Corrigiendo tabla 'alertas'...")
        
        # Si existe columna 'nivel', eliminarla o hacerla nullable
        cursor.execute("""
            SELECT column_name 
            FROM information_schema.columns 
            WHERE table_name = 'alertas' AND column_name = 'nivel'
        """)
        if cursor.fetchone():
            # Opción 1: Hacer la columna nullable con valor por defecto
            cursor.execute("""
                ALTER TABLE alertas 
                ALTER COLUMN nivel DROP NOT NULL;
                
                ALTER TABLE alertas 
                ALTER COLUMN nivel SET DEFAULT 'medio';
            """)
            correcciones.append("✅ Columna 'nivel' ahora permite NULL con default 'medio'")
        
        # Asegurarse de que nivel_alerta tenga un default
        cursor.execute("""
            ALTER TABLE alertas 
            ALTER COLUMN nivel_alerta SET DEFAULT 'medio';
        """)
        correcciones.append("✅ Default 'medio' establecido para nivel_alerta")
        
        # 3. Corregir tabla analisis_historicos
        print("\n🔧 Corrigiendo tabla 'analisis_historicos'...")
        
        # Hacer estacion_id nullable temporalmente
        cursor.execute("""
            ALTER TABLE analisis_historicos 
            ALTER COLUMN estacion_id DROP NOT NULL;
        """)
        correcciones.append("✅ Columna estacion_id en analisis_historicos ahora permite NULL")
        
        # 4. Actualizar la consulta de análisis histórico para incluir estacion_id
        print("\n🔧 Creando función mejorada para análisis histórico...")
        cursor.execute("""
            CREATE OR REPLACE FUNCTION actualizar_analisis_historico_completo()
            RETURNS void AS $$
            BEGIN
                INSERT INTO analisis_historicos 
                (estacion_id, mes, año, temp_promedio, temp_max_promedio, 
                 temp_min_promedio, precipitacion_total, dias_helada, 
                 dias_lluvia, humedad_promedio, evapotranspiracion_total)
                SELECT 
                    COALESCE(estacion_id, 'GENERAL') as estacion_id,
                    EXTRACT(MONTH FROM fecha_hora)::INTEGER as mes,
                    EXTRACT(YEAR FROM fecha_hora)::INTEGER as año,
                    AVG(temperatura) as temp_promedio,
                    AVG(temperatura_max) as temp_max_promedio,
                    AVG(temperatura_min) as temp_min_promedio,
                    SUM(precipitacion) as precipitacion_total,
                    COUNT(CASE WHEN temperatura_min <= 0 THEN 1 END)::INTEGER as dias_helada,
                    COUNT(DISTINCT CASE WHEN precipitacion > 0.1 THEN DATE(fecha_hora) END)::INTEGER as dias_lluvia,
                    AVG(humedad_relativa) as humedad_promedio,
                    SUM(evapotranspiracion) as evapotranspiracion_total
                FROM datos_meteorologicos
                WHERE fecha_hora >= CURRENT_DATE - INTERVAL '30 days'
                  AND estacion_id IS NOT NULL
                GROUP BY estacion_id, EXTRACT(MONTH FROM fecha_hora), EXTRACT(YEAR FROM fecha_hora)
                ON CONFLICT (estacion_id, mes, año) 
                DO UPDATE SET
                    temp_promedio = EXCLUDED.temp_promedio,
                    temp_max_promedio = EXCLUDED.temp_max_promedio,
                    temp_min_promedio = EXCLUDED.temp_min_promedio,
                    precipitacion_total = EXCLUDED.precipitacion_total,
                    dias_helada = EXCLUDED.dias_helada,
                    dias_lluvia = EXCLUDED.dias_lluvia,
                    humedad_promedio = EXCLUDED.humedad_promedio,
                    evapotranspiracion_total = EXCLUDED.evapotranspiracion_total,
                    updated_at = CURRENT_TIMESTAMP;
            END;
            $$ LANGUAGE plpgsql;
        """)
        correcciones.append("✅ Función de actualización histórica mejorada creada")
        
        # 5. Limpiar datos NULL existentes
        print("\n🧹 Limpiando datos NULL existentes...")
        
        # Actualizar registros de alertas con nivel NULL
        cursor.execute("""
            UPDATE alertas 
            SET nivel = 'medio' 
            WHERE nivel IS NULL
        """)
        filas_actualizadas = cursor.rowcount
        if filas_actualizadas > 0:
            correcciones.append(f"✅ {filas_actualizadas} alertas actualizadas con nivel='medio'")
        
        # Actualizar registros de alertas con nivel_alerta NULL
        cursor.execute("""
            UPDATE alertas 
            SET nivel_alerta = 'medio' 
            WHERE nivel_alerta IS NULL
        """)
        filas_actualizadas = cursor.rowcount
        if filas_actualizadas > 0:
            correcciones.append(f"✅ {filas_actualizadas} alertas actualizadas con nivel_alerta='medio'")
        
        conn.commit()
        
        # Mostrar resumen
        print("\n📊 RESUMEN DE CORRECCIONES:")
        print("="*60)
        for correccion in correcciones:
            print(f"  {correccion}")
        print("="*60)
        
        # Verificar estructura final
        print("\n🔍 Estructura final de las tablas:")
        
        # Verificar alertas
        cursor.execute("""
            SELECT column_name, is_nullable, column_default
            FROM information_schema.columns
            WHERE table_name = 'alertas'
            AND column_name IN ('nivel', 'nivel_alerta', 'mensaje', 'descripcion')
            ORDER BY ordinal_position
        """)
        print("\n📋 Tabla 'alertas':")
        for col in cursor.fetchall():
            print(f"   - {col[0]}: nullable={col[1]}, default={col[2]}")
        
        # Verificar analisis_historicos
        cursor.execute("""
            SELECT column_name, is_nullable
            FROM information_schema.columns
            WHERE table_name = 'analisis_historicos'
            AND column_name = 'estacion_id'
        """)
        print("\n📋 Tabla 'analisis_historicos':")
        for col in cursor.fetchall():
            print(f"   - {col[0]}: nullable={col[1]}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error: {e}")
        if conn:
            conn.rollback()
        return False
        
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Ejecutar correcciones
print("🚀 Iniciando corrección de restricciones NULL...")
if corregir_restricciones_null():
    print("\n✅ Restricciones corregidas exitosamente")
    print("\n📌 Puedes volver a ejecutar el código de guardado")
    print("💡 Todos los datos deberían guardarse correctamente ahora")
else:
    print("\n❌ Error en la corrección")

🚀 Iniciando corrección de restricciones NULL...
✅ Conectado a la base de datos

🔍 Verificando estructura de 'alertas'...
Columnas encontradas:
  - nivel: nullable=NO, default=None
  - nivel_alerta: nullable=YES, default='medio'::character varying

🔧 Corrigiendo tabla 'alertas'...

🔧 Corrigiendo tabla 'analisis_historicos'...

🔧 Creando función mejorada para análisis histórico...

🧹 Limpiando datos NULL existentes...

📊 RESUMEN DE CORRECCIONES:
  ✅ Columna 'nivel' ahora permite NULL con default 'medio'
  ✅ Default 'medio' establecido para nivel_alerta
  ✅ Columna estacion_id en analisis_historicos ahora permite NULL
  ✅ Función de actualización histórica mejorada creada

🔍 Estructura final de las tablas:

📋 Tabla 'alertas':
   - nivel: nullable=YES, default='medio'::character varying
   - mensaje: nullable=YES, default=None
   - nivel_alerta: nullable=YES, default='medio'::character varying
   - descripcion: nullable=YES, default=None

📋 Tabla 'analisis_historicos':
   - estacion_id: nu

In [19]:
# CÓDIGO PARA CORREGIR EL ÚLTIMO ERROR
import psycopg2
import os

DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'mip_quillota'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', '1478')
}

def corregir_tabla_alertas_final():
    """Corrige el problema de fecha_alerta en la tabla alertas"""
    
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        print("✅ Conectado a la base de datos")
        
        # 1. Verificar estructura actual de alertas
        print("\n🔍 Analizando estructura de tabla 'alertas'...")
        cursor.execute("""
            SELECT column_name, is_nullable, column_default
            FROM information_schema.columns
            WHERE table_name = 'alertas'
            AND column_name LIKE '%fecha%'
            ORDER BY ordinal_position
        """)
        
        columnas_fecha = cursor.fetchall()
        print("\nColumnas de fecha encontradas:")
        for col in columnas_fecha:
            print(f"  - {col[0]}: nullable={col[1]}, default={col[2]}")
        
        # 2. Si existe fecha_alerta, manejarla apropiadamente
        cursor.execute("""
            SELECT column_name 
            FROM information_schema.columns 
            WHERE table_name = 'alertas' 
            AND column_name = 'fecha_alerta'
        """)
        
        if cursor.fetchone():
            print("\n🔧 Columna 'fecha_alerta' encontrada. Aplicando correcciones...")
            
            # Opción 1: Hacer la columna nullable con default
            cursor.execute("""
                ALTER TABLE alertas 
                ALTER COLUMN fecha_alerta DROP NOT NULL;
                
                ALTER TABLE alertas 
                ALTER COLUMN fecha_alerta SET DEFAULT CURRENT_TIMESTAMP;
            """)
            print("✅ fecha_alerta ahora permite NULL y tiene default CURRENT_TIMESTAMP")
            
            # Opción 2: Renombrar a fecha_inicio si no existe
            cursor.execute("""
                SELECT column_name 
                FROM information_schema.columns 
                WHERE table_name = 'alertas' 
                AND column_name = 'fecha_inicio'
            """)
            
            if not cursor.fetchone():
                print("\n🔄 Renombrando fecha_alerta a fecha_inicio...")
                cursor.execute("""
                    ALTER TABLE alertas 
                    RENAME COLUMN fecha_alerta TO fecha_inicio;
                """)
                print("✅ Columna renombrada a fecha_inicio")
            else:
                # Si ambas existen, actualizar fecha_alerta con valores de fecha_inicio
                cursor.execute("""
                    UPDATE alertas 
                    SET fecha_alerta = COALESCE(fecha_inicio, CURRENT_TIMESTAMP)
                    WHERE fecha_alerta IS NULL;
                """)
                print("✅ Valores NULL en fecha_alerta actualizados")
        
        # 3. Asegurar que fecha_inicio tenga default
        cursor.execute("""
            ALTER TABLE alertas 
            ALTER COLUMN fecha_inicio SET DEFAULT CURRENT_TIMESTAMP;
        """)
        print("✅ Default establecido para fecha_inicio")
        
        # 4. Crear vista para compatibilidad si es necesario
        print("\n🔧 Creando vista de compatibilidad...")
        cursor.execute("""
            CREATE OR REPLACE VIEW v_alertas_compatible AS
            SELECT 
                id,
                tipo_alerta,
                nivel_alerta,
                mensaje,
                descripcion,
                COALESCE(fecha_inicio, fecha_alerta, CURRENT_TIMESTAMP) as fecha_inicio,
                COALESCE(fecha_alerta, fecha_inicio, CURRENT_TIMESTAMP) as fecha_alerta,
                fecha_fin,
                estacion_id,
                estado,
                cultivo_afectado,
                acciones_recomendadas,
                created_at
            FROM alertas;
        """)
        print("✅ Vista de compatibilidad creada")
        
        # 5. Verificar restricciones finales
        print("\n🔍 Verificando restricciones finales...")
        cursor.execute("""
            SELECT 
                tc.constraint_name,
                tc.constraint_type,
                kcu.column_name
            FROM information_schema.table_constraints tc
            JOIN information_schema.key_column_usage kcu
                ON tc.constraint_name = kcu.constraint_name
                AND tc.table_schema = kcu.table_schema
            WHERE tc.table_schema = 'public'
                AND tc.table_name = 'alertas'
                AND tc.constraint_type = 'NOT NULL'
                AND kcu.column_name LIKE '%fecha%';
        """)
        
        restricciones = cursor.fetchall()
        if restricciones:
            print("\nRestricciones NOT NULL en columnas de fecha:")
            for r in restricciones:
                print(f"  - {r[2]}: {r[1]}")
        
        conn.commit()
        
        # 6. Mostrar estructura final
        print("\n📊 ESTRUCTURA FINAL DE 'alertas':")
        print("="*60)
        cursor.execute("""
            SELECT column_name, data_type, is_nullable, column_default
            FROM information_schema.columns
            WHERE table_name = 'alertas'
            ORDER BY ordinal_position
        """)
        
        for col in cursor.fetchall():
            nullable = "NULL" if col[2] == 'YES' else "NOT NULL"
            default = f"DEFAULT {col[3]}" if col[3] else ""
            print(f"{col[0]:25} {col[1]:15} {nullable:10} {default}")
        
        print("="*60)
        
        return True
        
    except Exception as e:
        print(f"❌ Error: {e}")
        if conn:
            conn.rollback()
        return False
        
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Ejecutar corrección
print("🚀 Aplicando corrección final para tabla alertas...")
if corregir_tabla_alertas_final():
    print("\n✅ Corrección completada exitosamente")
    print("\n💡 SOLUCIÓN ALTERNATIVA:")
    print("Si el error persiste, modifica tu código de guardado de alertas para usar 'fecha_alerta'")
    print("en lugar de 'fecha_inicio', o viceversa según la estructura de tu tabla.")
else:
    print("\n❌ Error en la corrección")

# Código adicional para verificar qué columna usar
print("\n📝 RECOMENDACIÓN DE CÓDIGO:")
print("Modifica la función guardar_alertas() para manejar ambos casos:")
print("""
# En tu función guardar_alertas, cambia:
registro = {
    'fecha_inicio': alerta.get('fecha', datetime.now()),
    # ...
}

# Por:
registro = {
    'fecha_inicio': alerta.get('fecha', datetime.now()),
    'fecha_alerta': alerta.get('fecha', datetime.now()),  # Agregar esta línea
    # ...
}
""")

🚀 Aplicando corrección final para tabla alertas...
✅ Conectado a la base de datos

🔍 Analizando estructura de tabla 'alertas'...

Columnas de fecha encontradas:
  - fecha_alerta: nullable=NO, default=None
  - fecha_inicio: nullable=YES, default=CURRENT_TIMESTAMP
  - fecha_fin: nullable=YES, default=None

🔧 Columna 'fecha_alerta' encontrada. Aplicando correcciones...
✅ fecha_alerta ahora permite NULL y tiene default CURRENT_TIMESTAMP
✅ Valores NULL en fecha_alerta actualizados
✅ Default establecido para fecha_inicio

🔧 Creando vista de compatibilidad...
✅ Vista de compatibilidad creada

🔍 Verificando restricciones finales...

📊 ESTRUCTURA FINAL DE 'alertas':
id                        integer         NOT NULL   DEFAULT nextval('alertas_id_seq'::regclass)
tipo_alerta               character varying NOT NULL   
nivel                     character varying NULL       DEFAULT 'medio'::character varying
mensaje                   text            NULL       
fecha_alerta              timestamp w

In [21]:
# CÓDIGO COMPLETO CON CORRECCIONES DE BASE DE DATOS
# Ejecutar TODO este código en UNA SOLA CELDA

import psycopg2
from psycopg2 import sql
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text
from IPython.display import display, HTML
import sys
from io import StringIO
import os

# ==================== CONFIGURACIÓN ====================

DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'mip_quillota'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', '1478')
}

# ==================== FUNCIONES DE CORRECCIÓN DE BD ====================

def verificar_y_corregir_bd():
    """Verifica y corrige la estructura de la base de datos antes de guardar"""
    
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        print(f"✅ Conectado a la base de datos '{DB_CONFIG['database']}'")
        
        correcciones = []
        
        # 1. Crear tabla datos_meteorologicos si no existe
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS datos_meteorologicos (
                id SERIAL PRIMARY KEY,
                estacion_id INTEGER,
                estacion_nombre VARCHAR(255),
                fecha_hora TIMESTAMP NOT NULL,
                temperatura FLOAT,
                temperatura_aparente FLOAT,
                humedad_relativa FLOAT,
                precipitacion FLOAT,
                precipitacion_probabilidad FLOAT,
                velocidad_viento FLOAT,
                direccion_viento FLOAT,
                rafaga_viento FLOAT,
                presion_atmosferica FLOAT,
                radiacion_solar FLOAT,
                evapotranspiracion FLOAT,
                indice_uv FLOAT,
                punto_rocio FLOAT,
                temperatura_max FLOAT,
                temperatura_min FLOAT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        # 2. Crear tabla pronosticos si no existe
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS pronosticos (
                id SERIAL PRIMARY KEY,
                estacion_id INTEGER,
                fecha_pronostico TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                fecha_prediccion TIMESTAMP,
                tipo_pronostico VARCHAR(50),
                temperatura_pred FLOAT,
                temperatura_max_pred FLOAT,
                temperatura_min_pred FLOAT,
                precipitacion_pred FLOAT,
                humedad_pred FLOAT,
                probabilidad_helada FLOAT,
                modelo_usado VARCHAR(100),
                confianza FLOAT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        # 3. Crear tabla alertas si no existe
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS alertas (
                id SERIAL PRIMARY KEY,
                tipo_alerta VARCHAR(100),
                nivel_alerta VARCHAR(50) DEFAULT 'medio',
                mensaje TEXT,
                descripcion TEXT,
                fecha_inicio TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                fecha_fin TIMESTAMP,
                estacion_id INTEGER,
                estado VARCHAR(50) DEFAULT 'activa',
                cultivo_afectado VARCHAR(100),
                acciones_recomendadas TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        # 4. Crear tabla recomendaciones_cultivos si no existe
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS recomendaciones_cultivos (
                id SERIAL PRIMARY KEY,
                cultivo VARCHAR(100),
                fecha_analisis TIMESTAMP,
                estacion_id INTEGER,
                aptitud_siembra FLOAT,
                riesgo_helada FLOAT,
                necesidad_riego FLOAT,
                fase_fenologica VARCHAR(50),
                recomendacion TEXT,
                proxima_accion DATE,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        
        # 5. Crear tabla analisis_historicos si no existe
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS analisis_historicos (
                id SERIAL PRIMARY KEY,
                estacion_id INTEGER,
                mes INTEGER,
                año INTEGER,
                temp_promedio FLOAT,
                temp_max_promedio FLOAT,
                temp_min_promedio FLOAT,
                precipitacion_total FLOAT,
                dias_helada INTEGER,
                dias_lluvia INTEGER,
                humedad_promedio FLOAT,
                evapotranspiracion_total FLOAT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                UNIQUE(estacion_id, mes, año)
            )
        """)
        
        # 6. Verificar y agregar columnas faltantes
        # Para alertas - verificar nivel_alerta
        cursor.execute("""
            SELECT column_name FROM information_schema.columns 
            WHERE table_name = 'alertas' AND column_name = 'nivel_alerta'
        """)
        if not cursor.fetchone():
            cursor.execute("ALTER TABLE alertas ADD COLUMN nivel_alerta VARCHAR(50) DEFAULT 'medio'")
            correcciones.append("Agregada columna nivel_alerta a alertas")
        
        # Para pronosticos - verificar estacion_id
        cursor.execute("""
            SELECT column_name FROM information_schema.columns 
            WHERE table_name = 'pronosticos' AND column_name = 'estacion_id'
        """)
        if not cursor.fetchone():
            cursor.execute("ALTER TABLE pronosticos ADD COLUMN estacion_id INTEGER")
            correcciones.append("Agregada columna estacion_id a pronosticos")
        
        # Crear índices para mejorar rendimiento
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_datos_met_fecha ON datos_meteorologicos(fecha_hora)")
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_pronosticos_fecha ON pronosticos(fecha_pronostico)")
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_alertas_fecha ON alertas(fecha_inicio)")
        
        conn.commit()
        
        if correcciones:
            print("📋 Correcciones aplicadas:")
            for c in correcciones:
                print(f"  • {c}")
        
        return True
        
    except Exception as e:
        print(f"❌ Error en verificación/corrección de BD: {e}")
        if conn:
            conn.rollback()
        return False
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# ==================== FUNCIONES DE GUARDADO CORREGIDAS ====================

# Crear buffer para capturar salida
output_buffer = StringIO()
original_stdout = sys.stdout
sys.stdout = output_buffer

try:
    # Primero verificar y corregir la BD
    print("🔧 Verificando y corrigiendo estructura de base de datos...")
    if verificar_y_corregir_bd():
        print("✅ Base de datos lista para recibir datos")
    else:
        print("⚠️ Advertencia: Pueden ocurrir errores al guardar")

    def guardar_datos_meteorologicos(df_datos, engine):
        """Guardar datos meteorológicos en PostgreSQL"""
        
        try:
            # Preparar datos para inserción
            columnas_requeridas = [
                'estacion_id', 'estacion_nombre', 'fecha_hora', 'temperatura',
                'temperatura_aparente', 'humedad_relativa', 'precipitacion',
                'precipitacion_probabilidad', 'velocidad_viento', 'direccion_viento',
                'rafaga_viento', 'presion_atmosferica', 'radiacion_solar',
                'evapotranspiracion', 'indice_uv', 'punto_rocio'
            ]
            
            # Verificar que todas las columnas existan
            columnas_disponibles = [col for col in columnas_requeridas if col in df_datos.columns]
            df_guardar = df_datos[columnas_disponibles].copy()
            
            # Agregar temperaturas máximas y mínimas del día si no existen
            if 'temperatura' in df_guardar.columns:
                df_guardar['temperatura_max'] = df_guardar.groupby(
                    [df_guardar['fecha_hora'].dt.date, 'estacion_id']
                )['temperatura'].transform('max')
                
                df_guardar['temperatura_min'] = df_guardar.groupby(
                    [df_guardar['fecha_hora'].dt.date, 'estacion_id']
                )['temperatura'].transform('min')
            
            # Insertar en la base de datos
            df_guardar.to_sql(
                'datos_meteorologicos',
                engine,
                if_exists='append',
                index=False,
                method='multi',
                chunksize=1000
            )
            
            print(f"✅ {len(df_guardar)} registros guardados en datos_meteorologicos")
            return True
            
        except Exception as e:
            print(f"❌ Error guardando datos meteorológicos: {e}")
            return False

    def guardar_pronosticos(pronosticos_dict, estacion_id, engine):
        """Guardar pronósticos en PostgreSQL"""
        
        try:
            registros = []
            fecha_actual = datetime.now()
            
            for periodo, pronosticos in pronosticos_dict.items():
                if periodo == 'heladas' and pronosticos is not None:
                    # Guardar pronósticos de heladas
                    if isinstance(pronosticos, pd.DataFrame) and not pronosticos.empty:
                        for _, row in pronosticos.iterrows():
                            registros.append({
                                'estacion_id': estacion_id,
                                'fecha_pronostico': fecha_actual,
                                'fecha_prediccion': row.get('fecha_hora', fecha_actual),
                                'tipo_pronostico': 'helada',
                                'temperatura_pred': row.get('temperatura_esperada', 0),
                                'temperatura_min_pred': row.get('temperatura_minima', 0),
                                'probabilidad_helada': row.get('probabilidad', 0),
                                'modelo_usado': 'Prophet+ML',
                                'confianza': 0.85
                            })
                
                elif isinstance(pronosticos, dict):
                    # Guardar otros pronósticos
                    for variable, df_pronostico in pronosticos.items():
                        if df_pronostico is None or not isinstance(df_pronostico, pd.DataFrame):
                            continue
                        
                        # Filtrar solo predicciones futuras
                        if 'ds' in df_pronostico.columns:
                            df_futuro = df_pronostico[df_pronostico['ds'] > fecha_actual]
                            
                            # Determinar tipo de pronóstico
                            if '48_horas' in periodo:
                                tipo = 'corto_plazo'
                            elif '7_dias' in periodo:
                                tipo = 'mediano_plazo'
                            else:
                                tipo = 'largo_plazo'
                            
                            # Crear registros
                            for _, row in df_futuro.iterrows():
                                registro = {
                                    'estacion_id': estacion_id,
                                    'fecha_pronostico': fecha_actual,
                                    'fecha_prediccion': row['ds'],
                                    'tipo_pronostico': tipo,
                                    'modelo_usado': 'Prophet',
                                    'confianza': 0.95 - (row['ds'] - fecha_actual).days * 0.02
                                }
                                
                                # Agregar valores según variable
                                if variable == 'temperatura' and 'yhat' in row:
                                    registro.update({
                                        'temperatura_pred': row['yhat'],
                                        'temperatura_max_pred': row.get('yhat_upper', row['yhat']),
                                        'temperatura_min_pred': row.get('yhat_lower', row['yhat'])
                                    })
                                elif variable == 'precipitacion' and 'yhat' in row:
                                    registro['precipitacion_pred'] = max(0, row['yhat'])
                                elif variable == 'humedad_relativa' and 'yhat' in row:
                                    registro['humedad_pred'] = np.clip(row['yhat'], 0, 100)
                                
                                registros.append(registro)
            
            # Guardar en base de datos
            if registros:
                df_pronosticos = pd.DataFrame(registros)
                df_pronosticos.to_sql(
                    'pronosticos',
                    engine,
                    if_exists='append',
                    index=False,
                    method='multi'
                )
                print(f"✅ {len(registros)} pronósticos guardados")
                return True
            
            return False
            
        except Exception as e:
            print(f"❌ Error guardando pronósticos: {e}")
            return False

    def guardar_alertas(alertas, estacion_id, engine):
        """Guardar alertas en PostgreSQL"""
        
        try:
            registros = []
            
            for alerta in alertas:
                fecha_actual = alerta.get('fecha', datetime.now())
                
                registro = {
                    'tipo_alerta': alerta.get('tipo', 'general'),
                    'nivel': alerta.get('nivel', 'medio'),  # Para columna 'nivel'
                    'nivel_alerta': alerta.get('nivel', 'medio'),  # Para columna 'nivel_alerta'
                    'mensaje': '',
                    'descripcion': '',
                    'fecha_alerta': fecha_actual,  # AGREGAR ESTA LÍNEA
                    'fecha_inicio': fecha_actual,  # MANTENER ESTA LÍNEA
                    'estacion': estacion_id,  # Para columna 'estacion'
                    'estacion_id': estacion_id,  # Para columna 'estacion_id'
                    'estado': 'activa',
                    'activa': True  # Para columna 'activa' (boolean)
            }
                
                                # Personalizar según tipo de alerta con detalles meteorológicos
                if alerta.get('tipo') == 'helada':
                    registro['mensaje'] = f"Alerta de helada - Probabilidad: {alerta.get('probabilidad', 0)*100:.0f}%"
                    registro['descripcion'] = f"Tipo: {alerta.get('tipo_helada', 'general')}"
                    registro['nivel_alerta'] = 'alto' if alerta.get('probabilidad', 0) > 0.7 else 'medio'
                    registro['cultivo_afectado'] = 'todos'
                    
                    # Detalles meteorológicos específicos para heladas
                    detalles_met = []
                    if 'temperatura_minima' in alerta:
                        detalles_met.append(f"Temp. mínima esperada: {alerta['temperatura_minima']:.1f}°C")
                    if 'hora_critica' in alerta:
                        detalles_met.append(f"Hora crítica: {alerta['hora_critica']}")
                    if 'duracion_horas' in alerta:
                        detalles_met.append(f"Duración estimada: {alerta['duracion_horas']:.1f} horas")
                    if 'punto_rocio' in alerta:
                        detalles_met.append(f"Punto de rocío: {alerta['punto_rocio']:.1f}°C")
                    if 'velocidad_viento' in alerta:
                        detalles_met.append(f"Viento: {alerta['velocidad_viento']:.1f} km/h")
                    
                    registro['descripcion'] += '\n' + '\n'.join(detalles_met)
                    
                    if 'acciones' in alerta and alerta['acciones']:
                        registro['acciones_recomendadas'] = '\n'.join(alerta['acciones'][:3])
                    
                elif alerta.get('tipo') == 'ola_calor':
                    duracion = alerta.get('duracion_dias', 0)
                    temp_max = alerta.get('temp_maxima', 0)
                    
                    registro['mensaje'] = f"Ola de calor detectada - {duracion:.1f} días"
                    registro['nivel_alerta'] = 'alto' if temp_max > 35 else 'medio'
                    
                    # Detalles meteorológicos para ola de calor
                    detalles_met = [
                        f"Temperatura máxima esperada: {temp_max:.1f}°C",
                        f"Temperatura promedio: {alerta.get('temp_promedio', 0):.1f}°C"
                    ]
                    
                    if 'indice_calor' in alerta:
                        detalles_met.append(f"Índice de calor: {alerta['indice_calor']:.1f}°C")
                    if 'humedad_minima' in alerta:
                        detalles_met.append(f"Humedad mínima: {alerta['humedad_minima']:.0f}%")
                    if 'radiacion_maxima' in alerta:
                        detalles_met.append(f"Radiación UV máxima: {alerta['radiacion_maxima']:.1f}")
                    if 'evapotranspiracion_diaria' in alerta:
                        detalles_met.append(f"ET diaria: {alerta['evapotranspiracion_diaria']:.1f} mm")
                    
                    registro['descripcion'] = '\n'.join(detalles_met)
                    registro['fecha_fin'] = alerta.get('fecha_inicio', datetime.now()) + timedelta(days=duracion)
                    registro['acciones_recomendadas'] = """
                    - Aumentar frecuencia de riego (30-40% adicional)
                    - Implementar sombreado en cultivos sensibles
                    - Evitar labores entre 11:00-16:00 hrs
                    - Monitorear síntomas de estrés térmico
                    - Aplicar mulch para conservar humedad
                    """
                    
                elif alerta.get('tipo') == 'precipitacion':
                    cantidad = alerta.get('cantidad_mm', 0)
                    nivel = alerta.get('nivel', 'moderada')
                    
                    registro['mensaje'] = f"Precipitación {nivel} esperada: {cantidad:.1f}mm"
                    registro['nivel_alerta'] = 'alto' if nivel == 'extrema' or cantidad > 50 else 'medio'
                    
                    # Detalles meteorológicos para precipitación
                    detalles_met = [
                        f"Precipitación acumulada: {cantidad:.1f} mm",
                        f"Período: {alerta.get('duracion_horas', 24):.0f} horas"
                    ]
                    
                    if 'intensidad_maxima' in alerta:
                        detalles_met.append(f"Intensidad máxima: {alerta['intensidad_maxima']:.1f} mm/hr")
                    if 'probabilidad' in alerta:
                        detalles_met.append(f"Probabilidad: {alerta['probabilidad']*100:.0f}%")
                    if 'velocidad_viento' in alerta:
                        detalles_met.append(f"Viento asociado: {alerta['velocidad_viento']:.1f} km/h")
                    if 'riesgo_granizo' in alerta and alerta['riesgo_granizo']:
                        detalles_met.append("⚠️ Riesgo de granizo")
                    
                    registro['descripcion'] = '\n'.join(detalles_met)
                    registro['acciones_recomendadas'] = """
                    - Verificar y limpiar sistemas de drenaje
                    - Proteger cultivos sensibles a exceso de agua
                    - Posponer aplicaciones foliares 48-72 hrs
                    - Preparar zanjas de desvío si es necesario
                    - Cosechar productos maduros antes del evento
                    """
                
                elif alerta.get('tipo') == 'viento_fuerte':
                    velocidad = alerta.get('velocidad_maxima', 0)
                    
                    registro['mensaje'] = f"Alerta de viento fuerte - Ráfagas hasta {velocidad:.0f} km/h"
                    registro['nivel_alerta'] = 'alto' if velocidad > 60 else 'medio'
                    
                    # Detalles meteorológicos para viento
                    detalles_met = [
                        f"Velocidad sostenida: {alerta.get('velocidad_promedio', 0):.1f} km/h",
                        f"Ráfagas máximas: {velocidad:.0f} km/h",
                        f"Dirección predominante: {alerta.get('direccion', 'Variable')}"
                    ]
                    
                    if 'horas_criticas' in alerta:
                        detalles_met.append(f"Horas críticas: {alerta['horas_criticas']}")
                    if 'riesgo_volteo' in alerta:
                        detalles_met.append(f"Riesgo de volteo: {alerta['riesgo_volteo']}")
                    
                    registro['descripcion'] = '\n'.join(detalles_met)
                    registro['acciones_recomendadas'] = """
                    - Reforzar tutores y estructuras de soporte
                    - Posponer aplicaciones de agroquímicos
                    - Proteger invernaderos y túneles
                    - Amarrar o bajar cortinas cortavientos
                    - Cosechar frutos susceptibles a caída
                    """
                
                elif alerta.get('tipo') == 'sequia':
                    registro['mensaje'] = f"Condición de sequía - Déficit hídrico acumulado"
                    registro['nivel_alerta'] = alerta.get('nivel', 'medio')
                    
                    # Detalles meteorológicos para sequía
                    detalles_met = []
                    if 'deficit_mm' in alerta:
                        detalles_met.append(f"Déficit acumulado: {alerta['deficit_mm']:.1f} mm")
                    if 'dias_sin_lluvia' in alerta:
                        detalles_met.append(f"Días sin lluvia: {alerta['dias_sin_lluvia']}")
                    if 'humedad_suelo' in alerta:
                        detalles_met.append(f"Humedad del suelo: {alerta['humedad_suelo']:.0f}%")
                    if 'evapotranspiracion_acum' in alerta:
                        detalles_met.append(f"ET acumulada: {alerta['evapotranspiracion_acum']:.1f} mm")
                    
                    registro['descripcion'] = '\n'.join(detalles_met)
                    registro['acciones_recomendadas'] = """
                    - Implementar riego deficitario controlado
                    - Priorizar cultivos según valor económico
                    - Aplicar mulch orgánico
                    - Reducir densidad de plantación si es necesario
                    - Monitorear constantemente humedad del suelo
                    """
                
                elif alerta.get('tipo') == 'condiciones_fitosanitarias':
                    registro['mensaje'] = "Condiciones favorables para desarrollo de plagas/enfermedades"
                    registro['nivel_alerta'] = alerta.get('nivel', 'medio')
                    
                    # Detalles meteorológicos para condiciones fitosanitarias
                    detalles_met = []
                    if 'temperatura_optima' in alerta:
                        detalles_met.append(f"Temperatura: {alerta['temperatura_optima']:.1f}°C (óptima para patógenos)")
                    if 'humedad_relativa' in alerta:
                        detalles_met.append(f"Humedad relativa: {alerta['humedad_relativa']:.0f}%")
                    if 'horas_mojado_foliar' in alerta:
                        detalles_met.append(f"Horas de mojado foliar: {alerta['horas_mojado_foliar']:.1f}")
                    if 'patogenos_riesgo' in alerta:
                        detalles_met.append(f"Patógenos en riesgo: {', '.join(alerta['patogenos_riesgo'])}")
                    
                    registro['descripcion'] = '\n'.join(detalles_met)
                    registro['acciones_recomendadas'] = """
                    - Realizar monitoreo fitosanitario intensivo
                    - Aplicar fungicidas preventivos según umbral
                    - Mejorar ventilación en invernaderos
                    - Eliminar restos vegetales infectados
                    - Evitar riego por aspersión en horas críticas
                    """
                
                registros.append(registro)
            
            # Guardar en base de datos
            if registros:
                df_alertas = pd.DataFrame(registros)
                df_alertas.to_sql(
                    'alertas',
                    engine,
                    if_exists='append',
                    index=False
                )
                print(f"✅ {len(registros)} alertas guardadas")
                return True
            
            return False
            
        except Exception as e:
            print(f"❌ Error guardando alertas: {e}")
            return False

    def guardar_recomendaciones_cultivos(analisis_cultivos, estacion_id, engine):
        """Guardar recomendaciones de cultivos en PostgreSQL con detalles meteorológicos"""
        
        try:
            registros = []
            
            for cultivo, analisis in analisis_cultivos.items():
                registro = {
                    'cultivo': cultivo,
                    'fecha_analisis': analisis.get('fecha_analisis', datetime.now()),
                    'estacion_id': estacion_id,
                    'aptitud_siembra': 0.0,
                    'riesgo_helada': 0.0,
                    'necesidad_riego': 0.0,
                    'fase_fenologica': '',
                    'recomendacion': '',
                    'proxima_accion': None
                }
                
                # Calcular aptitud de siembra basada en condiciones meteorológicas
                if 'condiciones' in analisis:
                    condiciones = analisis['condiciones']
                    
                    # Temperatura
                    if 'temperatura' in condiciones:
                        temp_actual = condiciones['temperatura'].get('actual', 20)
                        temp_min_7d = condiciones['temperatura'].get('min_7_dias', 15)
                        temp_max_7d = condiciones['temperatura'].get('max_7_dias', 25)
                        
                        # Obtener configuración del cultivo
                        if 'CULTIVOS_CONFIG' in globals() and cultivo in CULTIVOS_CONFIG:
                            config = CULTIVOS_CONFIG[cultivo]
                            
                            # Calcular aptitud por temperatura
                            if config['temp_min'] <= temp_actual <= config['temp_max']:
                                aptitud_temp = 1.0 - abs(temp_actual - config['temp_optima']) / 10
                            else:
                                aptitud_temp = 0.0
                            
                            # Ajustar por extremos
                            if temp_min_7d < config['temp_min'] - 5:
                                aptitud_temp *= 0.5
                            if temp_max_7d > config['temp_max'] + 5:
                                aptitud_temp *= 0.7
                        else:
                            aptitud_temp = 0.5
                    
                                        # Humedad
                    if 'humedad' in condiciones:
                        humedad_actual = condiciones['humedad'].get('actual', 60)
                        humedad_prom = condiciones['humedad'].get('promedio_7_dias', 65)
                        
                        if 'CULTIVOS_CONFIG' in globals() and cultivo in CULTIVOS_CONFIG:
                            config = CULTIVOS_CONFIG[cultivo]
                            if config['humedad_min'] <= humedad_actual <= config['humedad_max']:
                                aptitud_humedad = 1.0 - abs(humedad_actual - config['humedad_optima']) / 20
                            else:
                                aptitud_humedad = 0.0
                        else:
                            aptitud_humedad = 0.5
                    
                    # Precipitación y riego
                    if 'precipitacion' in condiciones:
                        precip_acum = condiciones['precipitacion'].get('acumulada_7_dias', 0)
                        precip_prox = condiciones['precipitacion'].get('pronostico_7_dias', 0)
                        
                        # Calcular necesidad de riego basada en balance hídrico
                        if 'evapotranspiracion' in condiciones:
                            et_acum = condiciones['evapotranspiracion'].get('acumulada_7_dias', 35)
                            deficit_hidrico = et_acum - precip_acum
                            
                            # Normalizar necesidad de riego (0-1)
                            registro['necesidad_riego'] = min(1.0, max(0, deficit_hidrico / 50))
                            
                            # Ajustar por pronóstico
                            if precip_prox > 20:
                                registro['necesidad_riego'] *= 0.5
                    
                    # Calcular aptitud final
                    registro['aptitud_siembra'] = (aptitud_temp * 0.6 + aptitud_humedad * 0.4)
                    
                    # Ajustes por condiciones extremas
                    if 'viento' in condiciones:
                        viento_max = condiciones['viento'].get('maximo_7_dias', 0)
                        if viento_max > 40:
                            registro['aptitud_siembra'] *= 0.8
                    
                    if 'radiacion' in condiciones:
                        rad_prom = condiciones['radiacion'].get('promedio_7_dias', 0)
                        if rad_prom > 800:  # Alta radiación
                            registro['aptitud_siembra'] *= 0.9
                
                # Riesgo de helada
                if 'alertas' in analisis and analisis['alertas']:
                    heladas = [a for a in analisis['alertas'] if 'helada' in str(a.get('tipo', ''))]
                    if heladas:
                        # Calcular riesgo ponderado
                        max_prob = max([h.get('probabilidad', 0) for h in heladas])
                        registro['riesgo_helada'] = max_prob
                        
                        # Ajustar aptitud si hay riesgo de helada
                        if registro['riesgo_helada'] > 0.5:
                            registro['aptitud_siembra'] *= (1 - registro['riesgo_helada'] * 0.5)
                
                # Determinar fase fenológica basada en GDD (Grados Día de Desarrollo)
                if 'indices' in analisis and 'gdd_acumulado' in analisis['indices']:
                    gdd = analisis['indices']['gdd_acumulado']
                    mes_actual = analisis.get('fecha_analisis', datetime.now()).month
                    
                    if 'CULTIVOS_CONFIG' in globals() and cultivo in CULTIVOS_CONFIG:
                        config = CULTIVOS_CONFIG[cultivo]
                        
                        # Determinar fase según GDD y calendario
                        if mes_actual in config.get('meses_siembra', []):
                            registro['fase_fenologica'] = 'siembra'
                        elif gdd < config.get('gdd_emergencia', 100):
                            registro['fase_fenologica'] = 'germinacion'
                        elif gdd < config.get('gdd_floracion', 600):
                            registro['fase_fenologica'] = 'vegetativo'
                        elif gdd < config.get('gdd_fructificacion', 900):
                            registro['fase_fenologica'] = 'floracion'
                        elif gdd < config.get('gdd_maduracion', 1200):
                            registro['fase_fenologica'] = 'fructificacion'
                        elif mes_actual in config.get('meses_cosecha', []):
                            registro['fase_fenologica'] = 'maduracion'
                        else:
                            registro['fase_fenologica'] = 'cosecha'
                    else:
                        registro['fase_fenologica'] = 'indeterminado'
                
                # Generar recomendación principal basada en análisis meteorológico
                recomendaciones_met = []
                
                # Recomendación por temperatura
                if 'condiciones' in analisis and 'temperatura' in analisis['condiciones']:
                    temp = analisis['condiciones']['temperatura']
                    if temp.get('actual', 20) > 30:
                        recomendaciones_met.append({
                            'prioridad': 1,
                            'tipo': 'temperatura',
                            'texto': 'Implementar medidas de protección contra estrés térmico'
                        })
                    elif temp.get('min_7_dias', 10) < 5:
                        recomendaciones_met.append({
                            'prioridad': 1,
                            'tipo': 'temperatura',
                            'texto': 'Preparar protección antiheladas'
                        })
                
                # Recomendación por agua
                if registro['necesidad_riego'] > 0.7:
                    recomendaciones_met.append({
                        'prioridad': 1,
                        'tipo': 'riego',
                        'texto': f'Riego urgente requerido - Déficit: {registro["necesidad_riego"]*100:.0f}%'
                    })
                elif registro['necesidad_riego'] > 0.4:
                    recomendaciones_met.append({
                        'prioridad': 2,
                        'tipo': 'riego',
                        'texto': 'Programar riego en próximos 2-3 días'
                    })
                
                # Recomendación por fase fenológica
                recom_fase = {
                    'siembra': 'Condiciones aptas para siembra - Preparar terreno',
                    'germinacion': 'Mantener humedad constante para germinación',
                    'vegetativo': 'Aplicar fertilización nitrogenada',
                    'floracion': 'Reducir nitrógeno, aumentar fósforo y potasio',
                    'fructificacion': 'Monitorear desarrollo de frutos, ajustar riego',
                    'maduracion': 'Reducir riego, preparar cosecha',
                    'cosecha': 'Cosechar en horas frescas, evitar rocío'
                }
                
                if registro['fase_fenologica'] in recom_fase:
                    recomendaciones_met.append({
                        'prioridad': 2,
                        'tipo': 'manejo',
                        'texto': recom_fase[registro['fase_fenologica']]
                    })
                
                # Seleccionar recomendación principal
                if recomendaciones_met:
                    recom_principal = sorted(recomendaciones_met, key=lambda x: x['prioridad'])[0]
                    registro['recomendacion'] = recom_principal['texto']
                elif 'recomendaciones' in analisis and analisis['recomendaciones']:
                    rec = analisis['recomendaciones'][0]
                    registro['recomendacion'] = f"{rec.get('categoria', 'General')}: {rec.get('accion', 'Monitorear cultivo')}"
                else:
                    registro['recomendacion'] = 'Mantener monitoreo regular del cultivo'
                
                # Programar próxima acción
                dias_prox_accion = 7
                if registro['necesidad_riego'] > 0.7:
                    dias_prox_accion = 1
                elif registro['riesgo_helada'] > 0.5:
                    dias_prox_accion = 1
                elif registro['fase_fenologica'] in ['floracion', 'fructificacion']:
                    dias_prox_accion = 3
                
                registro['proxima_accion'] = (
                    analisis.get('fecha_analisis', datetime.now()) + timedelta(days=dias_prox_accion)
                ).date()
                
                registros.append(registro)
            
            # Guardar en base de datos
            if registros:
                df_recomendaciones = pd.DataFrame(registros)
                df_recomendaciones.to_sql(
                    'recomendaciones_cultivos',
                    engine,
                    if_exists='append',
                    index=False
                )
                print(f"✅ {len(registros)} recomendaciones de cultivos guardadas")
                return True
            
            return False
            
        except Exception as e:
            print(f"❌ Error guardando recomendaciones: {e}")
            return False

    def actualizar_analisis_historico(engine):
        """Actualizar análisis histórico con métricas meteorológicas detalladas"""
        
        try:
            with engine.connect() as conn:
                # Actualizar análisis del último mes con más detalles meteorológicos
                query = text("""
                    INSERT INTO analisis_historicos 
                    (estacion_id, mes, año, temp_promedio, temp_max_promedio, 
                     temp_min_promedio, precipitacion_total, dias_helada, 
                     dias_lluvia, humedad_promedio, evapotranspiracion_total)
                    SELECT 
                        estacion_id,
                        EXTRACT(MONTH FROM fecha_hora) as mes,
                        EXTRACT(YEAR FROM fecha_hora) as año,
                        AVG(temperatura) as temp_promedio,
                        AVG(temperatura_max) as temp_max_promedio,
                        AVG(temperatura_min) as temp_min_promedio,
                        SUM(precipitacion) as precipitacion_total,
                        COUNT(CASE WHEN temperatura_min <= 0 THEN 1 END) as dias_helada,
                        COUNT(DISTINCT CASE WHEN precipitacion > 0.1 THEN DATE(fecha_hora) END) as dias_lluvia,
                        AVG(humedad_relativa) as humedad_promedio,
                        SUM(evapotranspiracion) as evapotranspiracion_total
                    FROM datos_meteorologicos
                    WHERE fecha_hora >= CURRENT_DATE - INTERVAL '30 days'
                    GROUP BY estacion_id, EXTRACT(MONTH FROM fecha_hora), EXTRACT(YEAR FROM fecha_hora)
                    ON CONFLICT (estacion_id, mes, año) 
                    DO UPDATE SET
                        temp_promedio = EXCLUDED.temp_promedio,
                        temp_max_promedio = EXCLUDED.temp_max_promedio,
                        temp_min_promedio = EXCLUDED.temp_min_promedio,
                        precipitacion_total = EXCLUDED.precipitacion_total,
                        dias_helada = EXCLUDED.dias_helada,
                        dias_lluvia = EXCLUDED.dias_lluvia,
                        humedad_promedio = EXCLUDED.humedad_promedio,
                        evapotranspiracion_total = EXCLUDED.evapotranspiracion_total,
                        updated_at = CURRENT_TIMESTAMP
                """)
                
                result = conn.execute(query)
                conn.commit()
                
                print(f"✅ Análisis histórico actualizado")
                return True
                
        except Exception as e:
            print(f"❌ Error actualizando análisis histórico: {e}")
            return False

    # Función principal de guardado
    def guardar_todo_en_db(datos_actuales, df_horarios, df_diarios, pronosticos_estaciones, analisis_cultivos, engine):
        """Función principal para guardar todos los datos en PostgreSQL"""
        
        print("\n💾 Iniciando guardado en base de datos...")
        
        resultados = {
            'datos_meteorologicos': False,
            'pronosticos': False,
            'alertas': False,
            'recomendaciones': False,
            'analisis_historico': False
        }
        
        try:
            # 1. Guardar datos meteorológicos
            if 'df_horarios' in locals() and not df_horarios.empty:
                resultados['datos_meteorologicos'] = guardar_datos_meteorologicos(df_horarios, engine)
            
            # 2. Guardar pronósticos y alertas por estación
            for estacion_id, pronostico_data in pronosticos_estaciones.items():
                if pronostico_data:
                    # Guardar pronósticos
                    if 'pronosticos' in pronostico_data:
                        resultados['pronosticos'] = guardar_pronosticos(
                            pronostico_data['pronosticos'], 
                            estacion_id, 
                            engine
                        )
                    
                    # Guardar alertas
                    if 'alertas' in pronostico_data and pronostico_data['alertas']:
                        resultados['alertas'] = guardar_alertas(
                            pronostico_data['alertas'], 
                            estacion_id, 
                            engine
                        )
            
            # 3. Guardar recomendaciones de cultivos
            for estacion_id, cultivos_analisis in analisis_cultivos.items():
                if cultivos_analisis:
                    resultados['recomendaciones'] = guardar_recomendaciones_cultivos(
                        cultivos_analisis, 
                        estacion_id, 
                        engine
                    )
            
                        # 4. Actualizar análisis histórico
            resultados['analisis_historico'] = actualizar_analisis_historico(engine)
            
            # 5. Guardar métricas adicionales si están disponibles
            if 'metricas_adicionales' in locals():
                guardar_metricas_adicionales(metricas_adicionales, engine)
            
            # Resumen de guardado
            print("\n📊 Resumen de guardado:")
            for tabla, exito in resultados.items():
                estado = "✅" if exito else "❌"
                print(f"  {estado} {tabla}")
            
            return resultados
            
        except Exception as e:
            print(f"❌ Error general en guardado: {e}")
            return resultados

    # ========== CREAR CONEXIÓN A BASE DE DATOS ==========
    def crear_conexion_db():
        """Crear conexión SQLAlchemy a PostgreSQL"""
        try:
            connection_string = (
                f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
                f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
            )
            engine = create_engine(connection_string)
            print("✅ Conexión a base de datos establecida")
            return engine
        except Exception as e:
            print(f"❌ Error conectando a base de datos: {e}")
            return None

    # ========== EJECUTAR EL GUARDADO ==========
    
    # Verificar si hay datos para guardar
    if 'df_horarios' in locals() and not df_horarios.empty:
        engine = crear_conexion_db()
        if engine:
            # Generar pronósticos para cada estación si no existen
            if 'pronosticos_estaciones' not in locals():
                pronosticos_estaciones = {}
                if 'QUILLOTA_CONFIG' in globals():
                    for estacion in QUILLOTA_CONFIG['estaciones']:
                        if 'generar_pronostico_integrado' in globals():
                            pronostico = generar_pronostico_integrado(df_horarios, estacion)
                            if pronostico:
                                pronosticos_estaciones[estacion['id']] = pronostico
            
            # Analizar cultivos para cada estación si no existe
            if 'analisis_cultivos' not in locals():
                analisis_cultivos = {}
                if 'QUILLOTA_CONFIG' in globals() and 'CULTIVOS_CONFIG' in globals():
                    for estacion in QUILLOTA_CONFIG['estaciones']:
                        df_estacion = df_horarios[df_horarios['estacion_id'] == estacion['id']]
                        if not df_estacion.empty:
                            analisis_estacion = {}
                            for cultivo in CULTIVOS_CONFIG.keys():
                                if 'analizar_condiciones_cultivo' in globals():
                                    analisis = analizar_condiciones_cultivo(df_estacion, cultivo)
                                    analisis_estacion[cultivo] = analisis
                            analisis_cultivos[estacion['id']] = analisis_estacion
            
            # Guardar todo
            resultados_guardado = guardar_todo_en_db(
                datos_actuales if 'datos_actuales' in locals() else None, 
                df_horarios, 
                df_diarios if 'df_diarios' in locals() else pd.DataFrame(),
                pronosticos_estaciones if 'pronosticos_estaciones' in locals() else {},
                analisis_cultivos if 'analisis_cultivos' in locals() else {},
                engine
            )
            
            # Cerrar conexión
            engine.dispose()
    else:
        print("⚠️ No hay datos para guardar")
        resultados_guardado = {
            'datos_meteorologicos': False,
            'pronosticos': False,
            'alertas': False,
            'recomendaciones': False,
            'analisis_historico': False
        }

except Exception as e:
    print(f"❌ Error en el proceso principal: {e}")
    import traceback
    traceback.print_exc()

finally:
    # 3. Restaurar stdout y procesar la salida
    sys.stdout = original_stdout
    
    # 4. Obtener todo el contenido del buffer
    output_content = output_buffer.getvalue()
    
    # 5. Contar líneas y caracteres
    lines = output_content.split('\n')
    total_lines = len(lines)
    total_chars = len(output_content)
    
    # 6. Mostrar resumen conciso
    print("="*60)
    print("📊 PROCESO COMPLETADO - RESUMEN")
    print("="*60)
    print(f"Total de líneas procesadas: {total_lines}")
    print(f"Total de caracteres: {total_chars}")
    
    # 7. Mostrar solo las últimas líneas relevantes
    print("\n📋 Últimas operaciones realizadas:")
    print("-"*60)
    
    # Filtrar solo líneas con ✅ o ❌
    important_lines = [line for line in lines if '✅' in line or '❌' in line or '📊' in line]
    
    # Mostrar máximo las últimas 20 líneas importantes
    for line in important_lines[-20:]:
        if line.strip():
            print(line)
    
    # 8. Mostrar estadísticas finales si existen
    if 'resultados_guardado' in locals():
        print("\n📈 Estado final de guardado:")
        print("-"*60)
        for tabla, exito in resultados_guardado.items():
            estado = "✅ Éxito" if exito else "❌ Fallo"
            print(f"{tabla.ljust(25)}: {estado}")
    
    print("="*60)
    
    # 9. Guardar log completo en archivo con encoding UTF-8
    try:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'log_guardado_{timestamp}.txt'
        
        # Usar encoding UTF-8 para manejar caracteres especiales
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(output_content)
        print(f"\n💾 Log completo guardado en: {filename}")
    except Exception as e:
        print(f"\n❌ Error guardando log con UTF-8: {e}")
        
        # Intento alternativo: guardar sin caracteres especiales
        try:
            # Remover emojis y caracteres especiales
            output_limpio = output_content.encode('ascii', 'ignore').decode('ascii')
            filename_alt = f'log_guardado_{timestamp}_ascii.txt'
            
            with open(filename_alt, 'w') as f:
                f.write(output_limpio)
            print(f"💾 Log alternativo (sin emojis) guardado en: {filename_alt}")
        except Exception as e2:
            print(f"❌ Error en guardado alternativo: {e2}")

# ========== FUNCIONES ADICIONALES DE ANÁLISIS METEOROLÓGICO ==========

def generar_resumen_meteorologico_detallado(df_datos, estacion_id):
    """Genera un resumen meteorológico detallado para informes"""
    
    try:
        resumen = {
            'estacion_id': estacion_id,
            'periodo': {
                'inicio': df_datos['fecha_hora'].min(),
                'fin': df_datos['fecha_hora'].max(),
                'dias': (df_datos['fecha_hora'].max() - df_datos['fecha_hora'].min()).days
            },
            'temperatura': {
                'promedio': df_datos['temperatura'].mean(),
                'maxima_absoluta': df_datos['temperatura'].max(),
                'minima_absoluta': df_datos['temperatura'].min(),
                'amplitud_termica_promedio': df_datos.groupby(df_datos['fecha_hora'].dt.date)['temperatura'].apply(lambda x: x.max() - x.min()).mean(),
                'dias_helada': (df_datos.groupby(df_datos['fecha_hora'].dt.date)['temperatura'].min() <= 0).sum(),
                'dias_sobre_30': (df_datos.groupby(df_datos['fecha_hora'].dt.date)['temperatura'].max() >= 30).sum(),
                'grados_dia_acumulados': calcular_gdd(df_datos, 10, 30)  # Base 10°C, tope 30°C
            },
            'precipitacion': {
                'total': df_datos['precipitacion'].sum(),
                'dias_lluvia': (df_datos.groupby(df_datos['fecha_hora'].dt.date)['precipitacion'].sum() > 0.1).sum(),
                'maxima_diaria': df_datos.groupby(df_datos['fecha_hora'].dt.date)['precipitacion'].sum().max(),
                'intensidad_maxima_horaria': df_datos['precipitacion'].max()
            },
            'humedad': {
                'promedio': df_datos['humedad_relativa'].mean(),
                'minima': df_datos['humedad_relativa'].min(),
                'horas_hr_mayor_90': (df_datos['humedad_relativa'] > 90).sum(),
                'horas_hr_menor_30': (df_datos['humedad_relativa'] < 30).sum()
            },
            'viento': {
                'velocidad_promedio': df_datos['velocidad_viento'].mean(),
                'velocidad_maxima': df_datos['velocidad_viento'].max(),
                'rafaga_maxima': df_datos['rafaga_viento'].max() if 'rafaga_viento' in df_datos else None,
                'direccion_predominante': df_datos['direccion_viento'].mode()[0] if not df_datos['direccion_viento'].empty else None
            },
            'radiacion': {
                'promedio_diario': df_datos.groupby(df_datos['fecha_hora'].dt.date)['radiacion_solar'].sum().mean() if 'radiacion_solar' in df_datos else None,
                'maxima': df_datos['radiacion_solar'].max() if 'radiacion_solar' in df_datos else None,
                'dias_nublados': (df_datos.groupby(df_datos['fecha_hora'].dt.date)['radiacion_solar'].max() < 200).sum() if 'radiacion_solar' in df_datos else None
            },
            'evapotranspiracion': {
                'total': df_datos['evapotranspiracion'].sum() if 'evapotranspiracion' in df_datos else None,
                'promedio_diario': df_datos.groupby(df_datos['fecha_hora'].dt.date)['evapotranspiracion'].sum().mean() if 'evapotranspiracion' in df_datos else None,
                'maxima_diaria': df_datos.groupby(df_datos['fecha_hora'].dt.date)['evapotranspiracion'].sum().max() if 'evapotranspiracion' in df_datos else None
            }
        }
        
        return resumen
        
    except Exception as e:
        print(f"Error generando resumen meteorológico: {e}")
        return None

def calcular_gdd(df, temp_base=10, temp_max=30):
    """Calcula los Grados Día de Desarrollo acumulados"""
    try:
        df_diario = df.groupby(df['fecha_hora'].dt.date).agg({
            'temperatura': ['max', 'min']
        })
        df_diario.columns = ['temp_max', 'temp_min']
        
        # Método estándar: (Tmax + Tmin) / 2 - Tbase
        df_diario['temp_media'] = (df_diario['temp_max'] + df_diario['temp_min']) / 2
        
        # Ajustar por límites
        df_diario['temp_media'] = df_diario['temp_media'].clip(lower=temp_base, upper=temp_max)
        
        # Calcular GDD diarios
        df_diario['gdd'] = np.maximum(0, df_diario['temp_media'] - temp_base)
        
        # Retornar GDD acumulados
        return df_diario['gdd'].sum()
        
    except Exception as e:
        print(f"Error calculando GDD: {e}")
        return 0

print("\n✅ Código completo ejecutado. Revise los resultados arriba.")

📊 PROCESO COMPLETADO - RESUMEN
Total de líneas procesadas: 34
Total de caracteres: 1000

📋 Últimas operaciones realizadas:
------------------------------------------------------------
✅ 11 alertas guardadas
✅ 10697 pronósticos guardados
✅ 10 alertas guardadas
✅ 10697 pronósticos guardados
✅ 17 alertas guardadas
✅ 10697 pronósticos guardados
✅ 10 alertas guardadas
✅ 4 recomendaciones de cultivos guardadas
✅ 4 recomendaciones de cultivos guardadas
✅ 4 recomendaciones de cultivos guardadas
✅ 4 recomendaciones de cultivos guardadas
✅ 4 recomendaciones de cultivos guardadas
✅ 4 recomendaciones de cultivos guardadas
✅ Análisis histórico actualizado
📊 Resumen de guardado:
  ✅ datos_meteorologicos
  ✅ pronosticos
  ✅ alertas
  ✅ recomendaciones
  ✅ analisis_historico

📈 Estado final de guardado:
------------------------------------------------------------
datos_meteorologicos     : ✅ Éxito
pronosticos              : ✅ Éxito
alertas                  : ✅ Éxito
recomendaciones          : ✅ Éxito


In [24]:
import sys
print(sys.executable)

D:\Miguel\Anaconda_AIEP\python.exe


In [26]:
# Verificar versión actual
import dash_bootstrap_components as dbc
print(f"Versión actual de dash-bootstrap-components: {dbc.__version__}")

# Si la versión es menor a 1.6.0, actualizar
import subprocess
import sys

# Actualizar a la última versión
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "dash-bootstrap-components"])

Versión actual de dash-bootstrap-components: 1.5.0


0

In [30]:
# ============================================================================
# SISTEMA DE MONITOREO INTEGRADO DE PLAGAS (MIP) - QUILLOTA
# Dashboard Web Interactivo con Análisis Meteorológico y Agrícola
# Versión: 2.0 - Alto Detalle
# ============================================================================

import dash
from dash import dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import folium
from folium import plugins
import calendar
import schedule
import threading
import time
from sqlalchemy import create_engine, text
import os
import warnings
import json
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

# Configuración de base de datos
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5432'),
    'database': os.getenv('DB_NAME', 'mip_quillota'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', '1478')
}

# Configuración de Quillota
QUILLOTA_CONFIG = {
    'centro': {'lat': -32.8794, 'lon': -71.2491},
    'estaciones': [
        {'id': 'LCL01', 'nombre': 'La Cruz-La Cantera', 'lat': -32.8247, 'lon': -71.2274, 
         'altitud': 150, 'tipo_suelo': 'Franco-arcilloso'},
        {'id': 'HJL01', 'nombre': 'Hijuelas-La Peña', 'lat': -32.8020, 'lon': -71.1640,
         'altitud': 220, 'tipo_suelo': 'Franco'},
        {'id': 'LMC01', 'nombre': 'Limache-Campo', 'lat': -33.0069, 'lon': -71.2629,
         'altitud': 180, 'tipo_suelo': 'Franco-limoso'},
        {'id': 'NGL01', 'nombre': 'Nogales-El Melón', 'lat': -32.7391, 'lon': -71.2045,
         'altitud': 250, 'tipo_suelo': 'Franco-arenoso'}
    ]
}

# Configuración de cultivos mejorada
CULTIVOS_CONFIG = {
    'palta': {
        'nombre': 'Palta (Aguacate)',
        'temp_min': 10, 'temp_max': 30, 'temp_optima': 20,
        'temp_critica_helada': 0, 'temp_daño_helada': -2,
        'humedad_min': 60, 'humedad_max': 85, 'humedad_optima': 70,
        'ph_suelo': [5.5, 7.0],
        'meses_siembra': [3, 4, 5, 9, 10],
        'meses_floracion': [8, 9, 10, 11],
        'meses_cosecha': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'requerimiento_agua': 800,  # mm/año
        'umbral_et': 5.5,  # mm/día
        'color_theme': '#4CAF50'
    },
    'limonero': {
        'nombre': 'Limón',
        'temp_min': 12, 'temp_max': 35, 'temp_optima': 25,
        'temp_critica_helada': -2, 'temp_daño_helada': -4,
        'humedad_min': 60, 'humedad_max': 80, 'humedad_optima': 70,
        'ph_suelo': [5.5, 6.5],
        'meses_siembra': [3, 4, 5, 9, 10],
        'meses_floracion': [3, 4, 5, 8, 9, 10],
        'meses_cosecha': [5, 6, 7, 8, 9, 10, 11],
        'requerimiento_agua': 900,
        'umbral_et': 6.0,
        'color_theme': '#FFD700'
    },
    'tomate': {
        'nombre': 'Tomate',
        'temp_min': 10, 'temp_max': 30, 'temp_optima': 22,
        'temp_critica_helada': 2, 'temp_daño_helada': 0,
        'humedad_min': 50, 'humedad_max': 70, 'humedad_optima': 60,
        'ph_suelo': [6.0, 7.0],
        'meses_siembra': [8, 9, 10, 2, 3, 4],
        'meses_floracion': [10, 11, 12, 4, 5, 6],
        'meses_cosecha': [12, 1, 2, 3, 6, 7, 8],
        'requerimiento_agua': 600,
        'umbral_et': 5.0,
        'color_theme': '#FF6347'
    },
    'uva_mesa': {
        'nombre': 'Uva de Mesa',
        'temp_min': 7, 'temp_max': 35, 'temp_optima': 25,
        'temp_critica_helada': -1, 'temp_daño_helada': -3,
        'humedad_min': 40, 'humedad_max': 70, 'humedad_optima': 55,
        'ph_suelo': [6.0, 7.5],
        'meses_siembra': [7, 8],
        'meses_floracion': [10, 11],
        'meses_cosecha': [2, 3, 4],
        'requerimiento_agua': 700,
        'umbral_et': 5.5,
        'color_theme': '#9C27B0'
    }
}

# Niveles de alerta para heladas con rangos detallados
NIVELES_HELADA = {
    'sin_riesgo': {
        'rango': [5, float('inf')],
        'color': '#4CAF50',
        'icono': '✅',
        'descripcion': 'Sin riesgo de helada',
        'acciones': ['Mantener monitoreo regular']
    },
    'advertencia': {
        'rango': [2, 5],
        'color': '#FF9800',
        'icono': '⚠️',
        'descripcion': 'Advertencia de helada',
        'acciones': [
            'Preparar sistemas de protección',
            'Verificar pronósticos cada 6 horas',
            'Revisar estado de cultivos sensibles'
        ]
    },
    'alerta': {
        'rango': [0, 2],
        'color': '#FF5722',
        'icono': '🟡',
        'descripcion': 'Alerta de helada',
        'acciones': [
            'Activar sistemas de protección pasiva',
            'Riego preventivo al atardecer',
            'Preparar calefactores si disponibles',
            'Monitoreo cada 3 horas'
        ]
    },
    'peligro': {
        'rango': [-2, 0],
        'color': '#F44336',
        'icono': '🔴',
        'descripcion': 'Peligro de helada severa',
        'acciones': [
            'Activar todos los sistemas de protección',
            'Riego continuo por aspersión',
            'Encender calefactores',
            'Vigilancia permanente',
            'Cosechar frutos maduros urgentemente'
        ]
    },
    'extremo': {
        'rango': [float('-inf'), -2],
        'color': '#B71C1C',
        'icono': '🚨',
        'descripcion': 'Helada extrema',
        'acciones': [
            'Máxima protección activa',
            'Evacuación de productos sensibles',
            'Preparar evaluación de daños',
            'Contactar seguros agrícolas'
        ]
    }
}

# Colores para gráficos meteorológicos
COLORES_GRAFICOS = {
    'temperatura': '#FF5252',
    'temperatura_max': '#D32F2F',
    'temperatura_min': '#1976D2',
    'humedad': '#4CAF50',
    'precipitacion': {
        'leve': '#81C784',      # 0-2 mm
        'moderada': '#4CAF50',   # 2-10 mm
        'fuerte': '#2E7D32',     # 10-25 mm
        'intensa': '#1B5E20',    # 25-50 mm
        'torrencial': '#0D47A1'  # >50 mm
    },
    'viento': '#FFA726',
    'radiacion': '#FFD54F',
    'evapotranspiracion': '#7E57C2',
    'presion': '#607D8B'
}

# ============================================================================
# FUNCIONES AUXILIARES
# ============================================================================

def crear_conexion_db():
    """Crear conexión SQLAlchemy a PostgreSQL con manejo de errores"""
    try:
        connection_string = (
            f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
            f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        )
        engine = create_engine(connection_string, pool_pre_ping=True)
        # Probar la conexión
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        return engine
    except Exception as e:
        print(f"❌ Error conectando a base de datos: {e}")
        return None

def clasificar_nivel_helada(temperatura):
    """Clasifica el nivel de riesgo de helada según la temperatura"""
    for nivel, config in NIVELES_HELADA.items():
        if config['rango'][0] <= temperatura < config['rango'][1]:
            return nivel, config
    return 'sin_riesgo', NIVELES_HELADA['sin_riesgo']

def get_color_precipitacion(valor):
    """Obtiene el color según la intensidad de precipitación"""
    if valor <= 2:
        return COLORES_GRAFICOS['precipitacion']['leve']
    elif valor <= 10:
        return COLORES_GRAFICOS['precipitacion']['moderada']
    elif valor <= 25:
        return COLORES_GRAFICOS['precipitacion']['fuerte']
    elif valor <= 50:
        return COLORES_GRAFICOS['precipitacion']['intensa']
    else:
        return COLORES_GRAFICOS['precipitacion']['torrencial']

def calcular_indice_confort(temp, humedad, viento):
    """Calcula índice de confort térmico"""
    # Índice de calor si temp > 27°C
    if temp > 27:
        hi = -8.78469475556 + 1.61139411*temp + 2.33854883889*humedad - \
             0.14611605*temp*humedad - 0.012308094*(temp**2) - \
             0.0164248277778*(humedad**2) + 0.002211732*(temp**2)*humedad + \
             0.00072546*temp*(humedad**2) - 0.000003582*(temp**2)*(humedad**2)
        return hi
    # Sensación térmica si temp < 10°C y viento > 5 km/h
    elif temp < 10 and viento > 5:
        wc = 13.12 + 0.6215*temp - 11.37*(viento**0.16) + 0.3965*temp*(viento**0.16)
        return wc
    else:
        return temp

# ============================================================================
# INICIALIZACIÓN DE LA APLICACIÓN
# ============================================================================

# Crear aplicación Dash con tema personalizado
app = dash.Dash(
    __name__, 
    external_stylesheets=[
        dbc.themes.BOOTSTRAP,
        "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css"
    ],
    suppress_callback_exceptions=True,
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1"}
    ]
)

# Título de la aplicación
app.title = "Sistema MIP Quillota"

# Servidor para deployment
server = app.server

# ============================================================================
# ESTILOS CSS PERSONALIZADOS
# ============================================================================

app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            body {
                font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                background-color: #f5f5f5;
            }
            .navbar-custom {
                background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
                box-shadow: 0 2px 4px rgba(0,0,0,.1);
            }
            .card {
                border: none;
                border-radius: 10px;
                box-shadow: 0 2px 10px rgba(0,0,0,.08);
                transition: all 0.3s ease;
            }
            .card:hover {
                transform: translateY(-2px);
                box-shadow: 0 4px 20px rgba(0,0,0,.12);
            }
            .metric-card {
                background: white;
                padding: 20px;
                border-radius: 10px;
                text-align: center;
            }
            .metric-value {
                font-size: 2.5em;
                font-weight: bold;
                margin: 10px 0;
            }
            .metric-label {
                color: #666;
                font-size: 0.9em;
                text-transform: uppercase;
                letter-spacing: 1px;
            }
            .alert-card {
                border-left: 4px solid;
                margin-bottom: 15px;
            }
            .tab-content {
                background: white;
                padding: 20px;
                border-radius: 0 0 10px 10px;
            }
            .weather-icon {
                font-size: 3em;
                margin-bottom: 10px;
            }
            .gradient-text {
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                -webkit-background-clip: text;
                -webkit-text-fill-color: transparent;
                background-clip: text;
            }
            .status-indicator {
                width: 10px;
                height: 10px;
                border-radius: 50%;
                display: inline-block;
                margin-right: 5px;
                animation: pulse 2s infinite;
            }
            @keyframes pulse {
                0% { opacity: 1; }
                50% { opacity: 0.5; }
                100% { opacity: 1; }
            }
            .graph-container {
                background: white;
                padding: 20px;
                border-radius: 10px;
                margin-bottom: 20px;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# ============================================================================
# LAYOUT DE LA APLICACIÓN
# ============================================================================

app.layout = dbc.Container([
    # Header con navegación
    dbc.Navbar(
        dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.Div([
                        html.I(className="fas fa-seedling fa-2x text-white me-3"),
                        dbc.NavbarBrand("Sistema MIP Quillota", className="ms-2 fs-3 text-white")
                    ], className="d-flex align-items-center")
                ], width="auto"),
                dbc.Col([
                    html.Div([
                        html.Span(id="live-clock", className="text-white me-3"),
                        html.Span(id="connection-status", className="text-white")
                    ], className="d-flex align-items-center justify-content-end")
                ])
            ], className="w-100")
        ], fluid=True),
        className="navbar-custom mb-4",
        dark=True
    ),
    
    # Fila de controles principales
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className="fas fa-map-marker-alt me-2"),
                    "Estación Meteorológica"
                ], className="fw-bold"),
                dbc.CardBody([
                    dcc.Dropdown(
                        id='estacion-dropdown',
                        options=[
                            {
                                'label': html.Div([
                                    html.Span(est['nombre'], className="fw-bold"),
                                    html.Br(),
                                    html.Small(f"Alt: {est['altitud']}m | {est['tipo_suelo']}", 
                                             className="text-muted")
                                ], style={'lineHeight': '1.2'}),
                                'value': est['id']
                            } 
                            for est in QUILLOTA_CONFIG['estaciones']
                        ],
                        value=QUILLOTA_CONFIG['estaciones'][0]['id'],
                        clearable=False,
                        style={'minHeight': '60px'}
                    ),
                    html.Div(id="estacion-info", className="mt-2")
                ])
            ], className="h-100")
        ], lg=3, md=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className="fas fa-leaf me-2"),
                    "Cultivo"
                ], className="fw-bold"),
                dbc.CardBody([
                    dcc.Dropdown(
                        id='cultivo-dropdown',
                        options=[
                            {
                                'label': html.Div([
                                    html.Span(config['nombre'], className="fw-bold"),
                                    html.Br(),
                                    html.Small(f"T° óptima: {config['temp_optima']}°C", 
                                             className="text-muted")
                                ], style={'lineHeight': '1.2'}),
                                'value': cultivo
                            } 
                            for cultivo, config in CULTIVOS_CONFIG.items()
                        ],
                        value='palta',
                        clearable=False,
                        style={'minHeight': '60px'}
                    ),
                    html.Div(id="cultivo-info", className="mt-2")
                ])
            ], className="h-100")
        ], lg=3, md=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className="fas fa-calendar-alt me-2"),
                    "Período de Análisis"
                ], className="fw-bold"),
                dbc.CardBody([
                    dcc.DatePickerRange(
                        id='rango-fechas',
                        start_date=(datetime.now() - timedelta(days=7)).date(),
                        end_date=(datetime.now() + timedelta(days=7)).date(),
                        display_format='DD/MM/YYYY',
                        month_format='MMMM YYYY',
                        start_date_placeholder_text="Inicio",
                        end_date_placeholder_text="Fin",
                        style={'width': '100%'}
                    ),
                    html.Div([
                        dbc.ButtonGroup([
                            dbc.Button("7 días", id="btn-7d", size="sm", outline=True, color="primary"),
                            dbc.Button("15 días", id="btn-15d", size="sm", outline=True, color="primary"),
                            dbc.Button("30 días", id="btn-30d", size="sm", outline=True, color="primary"),
                        ], className="mt-2 w-100")
                    ])
                ])
            ], className="h-100")
        ], lg=3, md=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className="fas fa-cog me-2"),
                    "Acciones Rápidas"
                ], className="fw-bold"),
                dbc.CardBody([
                    dbc.ButtonGroup([
                        dbc.Button([
                            html.I(className="fas fa-sync-alt me-1"),
                            "Actualizar"
                        ], id="btn-refresh", color="primary", outline=True),
                        dbc.Button([
                            html.I(className="fas fa-download me-1"),
                            "Exportar"
                        ], id="btn-export", color="success", outline=True),
                        dbc.Button([
                            html.I(className="fas fa-bell me-1"),
                            "Alertas"
                        ], id="btn-alerts", color="warning", outline=True)
                    ], vertical=True, className="w-100")
                ])
            ], className="h-100")
        ], lg=3, md=6, className="mb-3")
    ]),
    
    # Sección de alertas activas
    dbc.Row([
        dbc.Col([
            html.Div(id='alertas-activas', className="mb-4")
        ])
    ]),
    
    # Panel de métricas rápidas
    html.Div(id='metricas-rapidas', className="mb-4"),
    
    # Tabs principales con branding METGO_3D®
dbc.Tabs([
    dbc.Tab(
        label="🌡️ Monitor Actual | METGO_3D", 
        tab_id="tab-actual",
        label_style={"padding": "10px 20px", "fontWeight": "500"}
    ),
    dbc.Tab(
        label="📊 Pronóstico 3D | METGO_3D", 
        tab_id="tab-pronostico",
        label_style={"padding": "10px 20px", "fontWeight": "500"}
    ),
    dbc.Tab(
        label="🌱 AgroCultivos | METGO_3D", 
        tab_id="tab-cultivos",
        label_style={"padding": "10px 20px", "fontWeight": "500"}
    ),
    dbc.Tab(
        label="📈 DataHistory | METGO_3D", 
        tab_id="tab-historico",
        label_style={"padding": "10px 20px", "fontWeight": "500"}
    ),
    dbc.Tab(
        label="🗺️ GeoMap 3D | METGO_3D", 
        tab_id="tab-mapa",
        label_style={"padding": "10px 20px", "fontWeight": "500"}
    ),
    dbc.Tab(
        label="🤖 AI Analytics | METGO_3D", 
        tab_id="tab-analisis",
        label_style={"padding": "10px 20px", "fontWeight": "500"}
    )
], id="tabs", active_tab="tab-actual", className="mb-4"),
    
    # Contenido de las tabs
    html.Div(id='tab-content', className="tab-content"),
    
    # Footer con información adicional
    dbc.Row([
        dbc.Col([
            html.Hr(),
            html.Div([
                html.Small([
                    html.I(className="fas fa-info-circle me-2"),
                    "Sistema MIP Quillota v2.0 | ",
                    html.A("Documentación", href="#", className="text-decoration-none"),
                    " | ",
                    html.A("Soporte", href="#", className="text-decoration-none"),
                    " | Última actualización: ",
                    html.Span(id="ultima-actualizacion", children=datetime.now().strftime("%d/%m/%Y %H:%M"))
                ], className="text-muted")
            ], className="text-center mb-3")
        ])
    ]),
    
    # Componentes auxiliares
    dcc.Interval(
        id='interval-component',
        interval=300*1000,  # Actualizar cada 5 minutos
        n_intervals=0
    ),
    
    dcc.Interval(
        id='clock-interval',
        interval=1000,  # Actualizar cada segundo
        n_intervals=0
    ),
    
    # Stores para datos
    dcc.Store(id='datos-store'),
    dcc.Store(id='alertas-store'),
    dcc.Store(id='pronosticos-store'),
    
    # Modales
    dbc.Modal([
        dbc.ModalHeader(dbc.ModalTitle("Exportar Datos")),
        dbc.ModalBody([
            html.P("Seleccione el formato de exportación:"),
            dbc.RadioItems(
                id="export-format",
                options=[
                    {"label": "Excel (.xlsx)", "value": "excel"},
                    {"label": "CSV (.csv)", "value": "csv"},
                    {"label": "PDF (Reporte)", "value": "pdf"}
                ],
                value="excel"
            ),
            html.Hr(),
            html.P("Incluir en la exportación:"),
            dbc.Checklist(
                id="export-options",
                options=[
                    {"label": "Datos meteorológicos", "value": "meteo"},
                    {"label": "Pronósticos", "value": "pronosticos"},
                    {"label": "Análisis de cultivos", "value": "cultivos"},
                    {"label": "Alertas", "value": "alertas"}
                ],
                value=["meteo", "pronosticos"]
            )
        ]),
        dbc.ModalFooter([
            dbc.Button("Cancelar", id="close-export", className="ms-auto", n_clicks=0),
            dbc.Button("Exportar", id="confirm-export", color="primary", n_clicks=0)
        ])
    ], id="modal-export", is_open=False),
    
        # Toast para notificaciones
    dbc.Toast(
        id="notification-toast",
        header="Notificación",
        is_open=False,
        dismissable=True,
        duration=4000,
        icon="info",
        style={"position": "fixed", "top": 66, "right": 10, "width": 350},
    ),
    
], fluid=True, className="px-4")

# ============================================================================
# CALLBACKS - ACTUALIZACIÓN DE INTERFAZ
# ============================================================================

# Callback para actualizar el reloj
@app.callback(
    Output('live-clock', 'children'),
    Input('clock-interval', 'n_intervals')
)
def update_clock(n):
    return datetime.now().strftime('%d/%m/%Y %H:%M:%S')

# Callback para estado de conexión
@app.callback(
    Output('connection-status', 'children'),
    Input('interval-component', 'n_intervals')
)
def update_connection_status(n):
    engine = crear_conexion_db()
    if engine:
        engine.dispose()
        return html.Span([
            html.Span(className="status-indicator bg-success"),
            "Conectado"
        ])
    else:
        return html.Span([
            html.Span(className="status-indicator bg-danger"),
            "Desconectado"
        ])

# Callback para información de estación
@app.callback(
    Output('estacion-info', 'children'),
    Input('estacion-dropdown', 'value')
)
def mostrar_info_estacion(estacion_id):
    if not estacion_id:
        return ""
    
    estacion = next((e for e in QUILLOTA_CONFIG['estaciones'] if e['id'] == estacion_id), None)
    if estacion:
        return html.Small([
            html.I(className="fas fa-mountain me-1"),
            f"Altitud: {estacion['altitud']}m | ",
            html.I(className="fas fa-layer-group me-1"),
            f"Suelo: {estacion['tipo_suelo']}"
        ], className="text-muted")
    return ""

# Callback para información de cultivo
@app.callback(
    Output('cultivo-info', 'children'),
    Input('cultivo-dropdown', 'value')
)
def mostrar_info_cultivo(cultivo):
    if not cultivo or cultivo not in CULTIVOS_CONFIG:
        return ""
    
    config = CULTIVOS_CONFIG[cultivo]
    return html.Small([
        html.I(className="fas fa-tint me-1"),
        f"Riego: {config['requerimiento_agua']}mm/año | ",
        html.I(className="fas fa-snowflake me-1"),
        f"T° crítica: {config['temp_critica_helada']}°C"
    ], className="text-muted")

# Callback para botones de período rápido
@app.callback(
    [Output('rango-fechas', 'start_date'),
     Output('rango-fechas', 'end_date')],
    [Input('btn-7d', 'n_clicks'),
     Input('btn-15d', 'n_clicks'),
     Input('btn-30d', 'n_clicks')],
    prevent_initial_call=True
)
def set_date_range(n7, n15, n30):
    ctx = dash.callback_context
    if not ctx.triggered:
        return dash.no_update, dash.no_update
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    end_date = datetime.now().date()
    
    if button_id == 'btn-7d':
        start_date = end_date - timedelta(days=7)
    elif button_id == 'btn-15d':
        start_date = end_date - timedelta(days=15)
    elif button_id == 'btn-30d':
        start_date = end_date - timedelta(days=30)
    else:
        return dash.no_update, dash.no_update
    
    return start_date, end_date

# Callback para alertas activas mejorado
@app.callback(
    Output('alertas-activas', 'children'),
    [Input('estacion-dropdown', 'value'),
     Input('interval-component', 'n_intervals')]
)
def actualizar_alertas(estacion_id, n):
    """Mostrar alertas activas con detalles de niveles de helada"""
    
    if not estacion_id:
        return None
    
    engine = crear_conexion_db()
    if not engine:
        return dbc.Alert("No hay conexión a la base de datos", color="warning", dismissable=True)
    
    try:
        # Obtener alertas activas
        query = """
            SELECT * FROM alertas 
            WHERE estacion_id = %(estacion_id)s 
            AND estado = 'activa'
            AND fecha_inicio <= NOW()
            AND (fecha_fin IS NULL OR fecha_fin >= NOW())
            ORDER BY 
                CASE nivel_alerta 
                    WHEN 'extremo' THEN 1
                    WHEN 'alto' THEN 2
                    WHEN 'medio' THEN 3
                    ELSE 4
                END,
                fecha_inicio DESC
            LIMIT 10
        """
        
        df_alertas = pd.read_sql(query, engine, params={'estacion_id': estacion_id})
        engine.dispose()
        
        if df_alertas.empty:
            return dbc.Alert([
                html.I(className="fas fa-check-circle me-2"),
                "No hay alertas activas"
            ], color="success", className="d-flex align-items-center")
        
        alertas_cards = []
        
        for _, alerta in df_alertas.iterrows():
            # Determinar configuración según tipo de alerta
            if alerta['tipo_alerta'] == 'helada':
                # Extraer temperatura mínima del mensaje o descripción
                temp_min = -2  # Valor por defecto
                try:
                    import re
                    # Buscar temperatura en el mensaje
                    match = re.search(r'[-]?\d+\.?\d*°C', str(alerta['descripcion']))
                    if match:
                        temp_min = float(match.group().replace('°C', ''))
                except:
                    pass
                
                nivel, config_nivel = clasificar_nivel_helada(temp_min)
                color_alerta = config_nivel['color']
                icono = config_nivel['icono']
                
                # Crear contenido detallado para heladas
                contenido_alerta = [
                    html.Div([
                        html.H5([
                            html.Span(icono, className="me-2"),
                            f"ALERTA DE HELADA - {config_nivel['descripcion'].upper()}"
                        ], className="mb-3 fw-bold"),
                        
                        # Panel de información crítica
                        dbc.Row([
                            dbc.Col([
                                html.Div([
                                    html.I(className="fas fa-thermometer-quarter fa-2x mb-2", 
                                          style={'color': color_alerta}),
                                    html.H6("Temperatura Mínima"),
                                    html.H4(f"{temp_min}°C", className="fw-bold")
                                ], className="text-center")
                            ], md=3),
                            dbc.Col([
                                html.Div([
                                    html.I(className="fas fa-clock fa-2x mb-2", 
                                          style={'color': color_alerta}),
                                    html.H6("Hora Crítica"),
                                    html.H4("04:00-06:00", className="fw-bold")
                                ], className="text-center")
                            ], md=3),
                            dbc.Col([
                                html.Div([
                                    html.I(className="fas fa-exclamation-triangle fa-2x mb-2", 
                                          style={'color': color_alerta}),
                                    html.H6("Probabilidad"),
                                    html.H4("85%", className="fw-bold")
                                ], className="text-center")
                            ], md=3),
                            dbc.Col([
                                html.Div([
                                    html.I(className="fas fa-shield-alt fa-2x mb-2", 
                                          style={'color': color_alerta}),
                                    html.H6("Acción Requerida"),
                                    html.H4("INMEDIATA", className="fw-bold")
                                ], className="text-center")
                            ], md=3)
                        ], className="mb-3"),
                        
                        html.Hr(),
                        
                        # Acciones recomendadas
                        html.H6([
                            html.I(className="fas fa-tasks me-2"),
                            "Acciones Recomendadas:"
                        ], className="mb-2"),
                        html.Ul([
                            html.Li(accion) for accion in config_nivel['acciones']
                        ], className="mb-0")
                    ])
                ]
                
                alertas_cards.append(
                    dbc.Alert(
                        contenido_alerta,
                        color="light",
                        className="alert-card",
                        style={'borderLeftColor': color_alerta}
                    )
                )
                
            else:
                # Otras alertas
                nivel = alerta.get('nivel_alerta', alerta.get('nivel', 'medio'))
                color = "danger" if nivel == 'alto' else "warning" if nivel == 'medio' else "info"
                icon = "🔴" if nivel == 'alto' else "🟡" if nivel == 'medio' else "🔵"
                
                alertas_cards.append(
                    dbc.Alert([
                        html.H5([icon, f" {alerta['tipo_alerta'].upper()}"], className="alert-heading"),
                        html.P(alerta['mensaje'] if pd.notna(alerta['mensaje']) else "", className="mb-1"),
                        html.Small(alerta['descripcion'] if pd.notna(alerta['descripcion']) else "", className="d-block"),
                        html.Hr(),
                        html.P(alerta['acciones_recomendadas'] if pd.notna(alerta['acciones_recomendadas']) else "", 
                              className="mb-0 small")
                    ], color=color, dismissable=True)
                )
        
        return html.Div([
            html.H5([
                html.I(className="fas fa-exclamation-triangle me-2"),
                f"Alertas Activas ({len(df_alertas)})"
            ], className="mb-3"),
            html.Div(alertas_cards)
        ])
        
    except Exception as e:
        return dbc.Alert(f"Error obteniendo alertas: {str(e)}", color="danger", dismissable=True)

# Callback para métricas rápidas
@app.callback(
    Output('metricas-rapidas', 'children'),
    [Input('estacion-dropdown', 'value'),
     Input('interval-component', 'n_intervals')]
)
def actualizar_metricas_rapidas(estacion_id, n):
    """Mostrar panel de métricas rápidas"""
    
    if not estacion_id:
        return None
    
    engine = crear_conexion_db()
    if not engine:
        return None
    
    try:
        # Obtener últimos datos
        query = """
            SELECT * FROM datos_meteorologicos 
            WHERE estacion_id = %(estacion_id)s 
            AND fecha_hora >= NOW() - INTERVAL '1 hour'
            ORDER BY fecha_hora DESC
            LIMIT 1
        """
        
        df = pd.read_sql(query, engine, params={'estacion_id': estacion_id})
        
        if df.empty:
            engine.dispose()
            return None
        
        ultimo = df.iloc[0]
        
        # Calcular tendencias
        query_tendencia = """
            SELECT 
                AVG(temperatura) as temp_promedio,
                AVG(humedad_relativa) as humedad_promedio,
                SUM(precipitacion) as precip_total
            FROM datos_meteorologicos 
            WHERE estacion_id = %(estacion_id)s 
            AND fecha_hora >= NOW() - INTERVAL '24 hours'
        """
        
        df_tendencia = pd.read_sql(query_tendencia, engine, params={'estacion_id': estacion_id})
        engine.dispose()
        
        tendencia = df_tendencia.iloc[0]
        
        # Crear métricas
        return dbc.Row([
            dbc.Col([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-thermometer-half weather-icon", 
                              style={'color': COLORES_GRAFICOS['temperatura']}),
                        html.H2(f"{ultimo['temperatura']:.1f}°C", className="metric-value"),
                        html.P("Temperatura", className="metric-label"),
                        html.Small([
                            html.I(className="fas fa-arrow-up text-danger" 
                                  if ultimo['temperatura'] > tendencia['temp_promedio'] 
                                  else "fas fa-arrow-down text-info"),
                            f" {abs(ultimo['temperatura'] - tendencia['temp_promedio']):.1f}°C vs promedio 24h"
                        ])
                    ], className="metric-card")
                ])
            ], lg=2, md=4, sm=6, className="mb-3"),
            
            dbc.Col([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-tint weather-icon", 
                              style={'color': COLORES_GRAFICOS['humedad']}),
                        html.H2(f"{ultimo['humedad_relativa']:.0f}%", className="metric-value"),
                        html.P("Humedad", className="metric-label"),
                        html.Small(f"Punto rocío: {ultimo['punto_rocio']:.1f}°C")
                    ], className="metric-card")
                ])
            ], lg=2, md=4, sm=6, className="mb-3"),
            
            dbc.Col([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-cloud-rain weather-icon", 
                              style={'color': get_color_precipitacion(ultimo['precipitacion'])}),
                        html.H2(f"{ultimo['precipitacion']:.1f} mm", className="metric-value"),
                        html.P("Precipitación", className="metric-label"),
                        html.Small(f"Acumulado 24h: {tendencia['precip_total']:.1f} mm")
                    ], className="metric-card")
                ])
            ], lg=2, md=4, sm=6, className="mb-3"),
            
            dbc.Col([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-wind weather-icon", 
                              style={'color': COLORES_GRAFICOS['viento']}),
                        html.H2(f"{ultimo['velocidad_viento']:.1f} km/h", className="metric-value"),
                        html.P("Viento", className="metric-label"),
                        html.Small(f"Ráfagas: {ultimo['rafaga_viento']:.1f} km/h | {ultimo['direccion_viento']:.0f}°")
                    ], className="metric-card")
                ])
            ], lg=2, md=4, sm=6, className="mb-3"),
            
            dbc.Col([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-sun weather-icon", 
                              style={'color': COLORES_GRAFICOS['radiacion']}),
                        html.H2(f"{ultimo['radiacion_solar']:.0f} W/m²", className="metric-value"),
                        html.P("Radiación Solar", className="metric-label"),
                        html.Small(f"UV: {ultimo['indice_uv']:.1f}")
                    ], className="metric-card")
                ])
            ], lg=2, md=4, sm=6, className="mb-3"),
            
            dbc.Col([
                html.Div([
                    html.Div([
                        html.I(className="fas fa-water weather-icon", 
                              style={'color': COLORES_GRAFICOS['evapotranspiracion']}),
                        html.H2(f"{ultimo['evapotranspiracion']:.1f} mm", className="metric-value"),
                        html.P("Evapotranspiración", className="metric-label"),
                        html.Small(f"Balance hídrico: {ultimo['precipitacion'] - ultimo['evapotranspiracion']:.1f} mm")
                    ], className="metric-card")
                ])
            ], lg=2, md=4, sm=6, className="mb-3")
        ])
        
    except Exception as e:
        return dbc.Alert(f"Error cargando métricas: {str(e)}", color="warning", dismissable=True)

# Callback principal para contenido de tabs
@app.callback(
    Output('tab-content', 'children'),
    [Input('tabs', 'active_tab'),
     Input('estacion-dropdown', 'value'),
     Input('cultivo-dropdown', 'value'),
     Input('rango-fechas', 'start_date'),
     Input('rango-fechas', 'end_date')]
)
def render_tab_content(active_tab, estacion_id, cultivo, fecha_inicio, fecha_fin):
    """Renderizar contenido según tab activa"""
    
    if not active_tab or not estacion_id:
        return html.Div()
    
    if active_tab == "tab-actual":
        return render_condiciones_actuales(estacion_id)
    elif active_tab == "tab-pronostico":
        return render_pronosticos(estacion_id, fecha_inicio, fecha_fin, cultivo)
    elif active_tab == "tab-cultivos":
        return render_analisis_cultivos(estacion_id, cultivo)
    elif active_tab == "tab-historico":
        return render_historicos(estacion_id, fecha_inicio, fecha_fin)
    elif active_tab == "tab-mapa":
        return render_mapa()
    elif active_tab == "tab-analisis":
        return render_analisis_avanzado(estacion_id, cultivo, fecha_inicio, fecha_fin)
    
    return html.Div()

# ============================================================================
# FUNCIONES DE RENDERIZADO - CONDICIONES ACTUALES
# ============================================================================

def render_condiciones_actuales(estacion_id):
    """Renderizar condiciones meteorológicas actuales con sub-pestañas"""
    
    engine = crear_conexion_db()
    if not engine:
        return dbc.Alert("No hay conexión a la base de datos", color="warning")
    
    try:
        # Obtener datos de las últimas 48 horas
        query = """
            SELECT * FROM datos_meteorologicos 
            WHERE estacion_id = %(estacion_id)s 
            AND fecha_hora >= NOW() - INTERVAL '48 hours'
            ORDER BY fecha_hora DESC
        """
        
        df_actual = pd.read_sql(query, engine, params={'estacion_id': estacion_id})
        engine.dispose()
        
        if df_actual.empty:
            return dbc.Alert("No hay datos disponibles para esta estación", color="info")
        
        # Datos de las últimas 24 horas para gráficos
        df_24h = df_actual[df_actual['fecha_hora'] >= datetime.now() - timedelta(hours=24)]
        ultimo_registro = df_actual.iloc[0]
        
        # Calcular índice de confort
        indice_confort = calcular_indice_confort(
            ultimo_registro['temperatura'],
            ultimo_registro['humedad_relativa'],
            ultimo_registro['velocidad_viento']
        )
        
        # Crear sub-tabs para mejor organización
        return html.Div([
            # Información de última actualización
            dbc.Alert([
                html.I(className="fas fa-info-circle me-2"),
                f"Última actualización: {ultimo_registro['fecha_hora'].strftime('%d/%m/%Y %H:%M:%S')} | ",
                f"Estación: {next((e['nombre'] for e in QUILLOTA_CONFIG['estaciones'] if e['id'] == estacion_id), 'Desconocida')}"
            ], color="info", className="mb-3"),
            
            # Panel de métricas actuales (siempre visible)
            render_panel_metricas_actuales(ultimo_registro, df_24h),
            
            # Sub-tabs para los diferentes análisis
            dbc.Tabs([
                dbc.Tab(
                    label="📊 Temperatura",
                    tab_id="sub-temperatura",
                    children=[
                        html.Div([
                            html.H5([
                                html.I(className="fas fa-temperature-high me-2"),
                                "Análisis de Temperatura"
                            ], className="mt-3 mb-3"),
                            dcc.Graph(
                                figure=crear_grafico_temperatura_detallado(df_24h, indice_confort),
                                config={'displayModeBar': True, 'displaylogo': False},
                                style={'height': '600px'}
                            )
                        ], className="p-3")
                    ]
                ),
                dbc.Tab(
                    label="💧 Precipitación",
                    tab_id="sub-precipitacion",
                    children=[
                        html.Div([
                            html.H5([
                                html.I(className="fas fa-cloud-rain me-2"),
                                "Análisis de Precipitación y Humedad"
                            ], className="mt-3 mb-3"),
                            # Alerta si hay precipitación significativa
                            render_alerta_precipitacion(df_24h),
                            dcc.Graph(
                                figure=crear_grafico_precipitacion_humedad(df_24h),
                                config={'displayModeBar': True, 'displaylogo': False},
                                style={'height': '600px'}
                            )
                        ], className="p-3")
                    ]
                ),
                dbc.Tab(
                    label="💨 Viento",
                    tab_id="sub-viento",
                    children=[
                        html.Div([
                            html.H5([
                                html.I(className="fas fa-wind me-2"),
                                "Análisis Completo del Viento"
                            ], className="mt-3 mb-3"),
                            dcc.Graph(
                                figure=crear_grafico_viento(df_24h),
                                config={'displayModeBar': True, 'displaylogo': False},
                                style={'height': '800px'}
                            )
                        ], className="p-3")
                    ]
                ),
                dbc.Tab(
                    label="☀️ Radiación/ET",
                    tab_id="sub-radiacion",
                    children=[
                        html.Div([
                            html.H5([
                                html.I(className="fas fa-sun me-2"),
                                "Radiación Solar y Evapotranspiración"
                            ], className="mt-3 mb-3"),
                            dcc.Graph(
                                figure=crear_grafico_radiacion_et(df_24h),
                                config={'displayModeBar': True, 'displaylogo': False},
                                style={'height': '600px'}
                            )
                        ], className="p-3")
                    ]
                ),
                dbc.Tab(
                    label="📈 Resumen",
                    tab_id="sub-resumen",
                    children=[
                        html.Div([
                            html.H5([
                                html.I(className="fas fa-chart-bar me-2"),
                                "Resumen Estadístico Detallado"
                            ], className="mt-3 mb-3"),
                            crear_tabla_resumen_estadistico(df_24h),
                            html.Div(className="mt-4"),
                            render_graficos_resumen_adicionales(df_24h)
                        ], className="p-3")
                    ]
                )
            ], id="subtabs-actual", active_tab="sub-temperatura", className="mt-3")
        ])
        
    except Exception as e:
        return dbc.Alert(f"Error cargando datos: {str(e)}", color="danger")

def render_panel_metricas_actuales(ultimo_registro, df_24h):
    """Panel de métricas actuales mejorado"""
    
    # Calcular acumulados correctamente
    precip_24h = df_24h['precipitacion'].sum()
    et_24h = df_24h['evapotranspiracion'].sum()
    
    return dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("🌡️ Temperatura", className="text-muted mb-2"),
                    html.H3(f"{ultimo_registro['temperatura']:.1f}°C"),
                    html.Small([
                        f"Máx: {df_24h['temperatura'].max():.1f}°C | ",
                        f"Mín: {df_24h['temperatura'].min():.1f}°C"
                    ], className="text-muted")
                ])
            ], className="text-center h-100")
        ], lg=2, md=4, sm=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("💧 Humedad", className="text-muted mb-2"),
                    html.H3(f"{ultimo_registro['humedad_relativa']:.0f}%"),
                    html.Small(f"Punto rocío: {ultimo_registro['punto_rocio']:.1f}°C", 
                              className="text-muted")
                ])
            ], className="text-center h-100")
        ], lg=2, md=4, sm=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("🌧️ Precipitación", className="text-muted mb-2"),
                    html.H3(f"{ultimo_registro['precipitacion']:.1f} mm/h"),
                    html.Small(f"Acum. 24h: {precip_24h:.1f} mm", 
                              className="text-muted"),
                    # Indicador visual si hay lluvia
                    html.Div([
                        dbc.Badge(
                            "LLUVIA ACTIVA" if ultimo_registro['precipitacion'] > 0 else "SIN LLUVIA",
                            color="primary" if ultimo_registro['precipitacion'] > 0 else "secondary",
                            className="mt-2"
                        )
                    ])
                ])
            ], className="text-center h-100")
        ], lg=2, md=4, sm=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("💨 Viento", className="text-muted mb-2"),
                    html.H3(f"{ultimo_registro['velocidad_viento']:.1f} km/h"),
                    html.Small([
                        f"Dir: {ultimo_registro['direccion_viento']:.0f}° | ",
                        f"Ráf: {ultimo_registro['rafaga_viento']:.1f} km/h"
                    ], className="text-muted")
                ])
            ], className="text-center h-100")
        ], lg=2, md=4, sm=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("☀️ Radiación", className="text-muted mb-2"),
                    html.H3(f"{ultimo_registro['radiacion_solar']:.0f} W/m²"),
                    html.Small(f"UV: {ultimo_registro['indice_uv']:.1f}", 
                              className="text-muted")
                ])
            ], className="text-center h-100")
        ], lg=2, md=4, sm=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("💧 Balance Hídrico", className="text-muted mb-2"),
                    html.H3(f"{precip_24h - et_24h:.1f} mm"),
                    html.Small(f"ET 24h: {et_24h:.1f} mm", 
                              className="text-muted"),
                    # Indicador de balance
                    html.Div([
                        dbc.Badge(
                            "DÉFICIT" if (precip_24h - et_24h) < 0 else "SUPERÁVIT",
                            color="warning" if (precip_24h - et_24h) < 0 else "success",
                            className="mt-2"
                        )
                    ])
                ])
            ], className="text-center h-100")
        ], lg=2, md=4, sm=6, className="mb-3")
    ])

def render_alerta_precipitacion(df_24h):
    """
    Renderiza alertas de precipitación con análisis de intensidad y riesgos
    """
    try:
        if df_24h.empty:
            return html.Div([
                html.H4("Sistema de Alertas de Precipitación", className="text-primary mb-3"),
                html.P("No hay datos disponibles para generar alertas", className="text-muted")
            ])
        
        # Debug: mostrar columnas disponibles
        print(f"📊 Columnas disponibles en df_24h: {df_24h.columns.tolist()}")
        print(f"📊 Primeras filas del DataFrame:")
        print(df_24h.head())
        
        # Verificar que la columna Precipitacion existe
        if 'Precipitacion' not in df_24h.columns:
            return html.Div([
                html.H4("Sistema de Alertas de Precipitación", className="text-primary mb-3"),
                dbc.Alert([
                    html.I(className="fas fa-exclamation-triangle me-2"),
                    f"Error: Columna 'Precipitacion' no encontrada. Columnas disponibles: {', '.join(df_24h.columns)}"
                ], color="warning")
            ])
        
        # Convertir a numérico y manejar valores nulos
        df_24h['Precipitacion'] = pd.to_numeric(df_24h['Precipitacion'], errors='coerce').fillna(0)
        
        # Calcular precipitación acumulada
        precip_total = df_24h['Precipitacion'].sum()
        precip_max_hora = df_24h['Precipitacion'].max()
        
        # Encontrar hora de máxima precipitación
        if precip_max_hora > 0 and not df_24h.empty:
            idx_max = df_24h['Precipitacion'].idxmax()
            if 'Fecha' in df_24h.columns:
                hora_max_precip = df_24h.loc[idx_max, 'Fecha']
            else:
                hora_max_precip = pd.Timestamp.now()
        else:
            hora_max_precip = pd.Timestamp.now()
        
        # Determinar nivel de alerta
        nivel_alerta = "normal"
        color_alerta = "success"
        icono_alerta = "fa-check-circle"
        mensaje_principal = "Condiciones normales"
        
        if precip_total > 50:  # mm en 24h
            nivel_alerta = "extremo"
            color_alerta = "danger"
            icono_alerta = "fa-exclamation-triangle"
            mensaje_principal = "ALERTA EXTREMA: Precipitación muy intensa"
        elif precip_total > 30:
            nivel_alerta = "alto"
            color_alerta = "warning"
            icono_alerta = "fa-exclamation-circle"
            mensaje_principal = "ALERTA ALTA: Precipitación intensa"
        elif precip_total > 10:
            nivel_alerta = "moderado"
            color_alerta = "info"
            icono_alerta = "fa-info-circle"
            mensaje_principal = "ALERTA MODERADA: Precipitación moderada"
        
        # Calcular intensidad de precipitación
        intensidades = []
        if len(df_24h) > 1:
            for i in range(len(df_24h)):
                if df_24h.iloc[i]['Precipitacion'] > 0:
                    intensidad = df_24h.iloc[i]['Precipitacion']  # mm/hora
                    if 'Fecha' in df_24h.columns:
                        hora = df_24h.iloc[i]['Fecha']
                    else:
                        hora = pd.Timestamp.now() - pd.Timedelta(hours=i)
                    
                    intensidades.append({
                        'hora': hora,
                        'intensidad': intensidad,
                        'categoria': categorizar_intensidad_lluvia(intensidad)
                    })
        
        # Crear gráfico de precipitación acumulada
        fig_acumulada = go.Figure()
        
        # Determinar eje X (fecha/hora)
        if 'Fecha' in df_24h.columns:
            x_axis = df_24h['Fecha']
        else:
            x_axis = list(range(len(df_24h)))
        
        # Precipitación horaria
        fig_acumulada.add_trace(go.Bar(
            x=x_axis,
            y=df_24h['Precipitacion'],
            name='Precipitación horaria',
            marker_color='lightblue',
            text=df_24h['Precipitacion'].round(1),
            textposition='auto',
            yaxis='y'
        ))
        
        # Precipitación acumulada
        precip_acum = df_24h['Precipitacion'].cumsum()
        fig_acumulada.add_trace(go.Scatter(
            x=x_axis,
            y=precip_acum,
            name='Precipitación acumulada',
            mode='lines+markers',
            line=dict(color='darkblue', width=3),
            yaxis='y2'
        ))
        
        # Líneas de referencia para alertas
        fig_acumulada.add_hline(y=10, line_dash="dash", line_color="yellow", 
                               annotation_text="Alerta Moderada (10mm)", yref='y2')
        fig_acumulada.add_hline(y=30, line_dash="dash", line_color="orange", 
                               annotation_text="Alerta Alta (30mm)", yref='y2')
        fig_acumulada.add_hline(y=50, line_dash="dash", line_color="red", 
                               annotation_text="Alerta Extrema (50mm)", yref='y2')
        
        fig_acumulada.update_layout(
            title="Precipitación: Análisis 24 horas",
            xaxis_title="Hora",
            yaxis=dict(title="Precipitación horaria (mm)", side="left"),
            yaxis2=dict(title="Precipitación acumulada (mm)", side="right", overlaying="y"),
            hovermode='x unified',
            height=400,
            showlegend=True,
            template="plotly_white"
        )
        
        # Análisis de riesgos agrícolas
        riesgos = analizar_riesgos_precipitacion(precip_total, precip_max_hora, intensidades)
        
        # Crear componente de alerta
        alerta_component = dbc.Alert([
            html.Div([
                html.I(className=f"fas {icono_alerta} fa-2x me-3"),
                html.Div([
                    html.H5(mensaje_principal, className="alert-heading mb-1"),
                    html.P(f"Precipitación acumulada: {precip_total:.1f} mm", className="mb-0")
                ])
            ], className="d-flex align-items-center")
        ], color=color_alerta, className="mb-3")
        
        # Estadísticas detalladas
        stats_cards = dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H6("Acumulado 24h", className="text-muted"),
                        html.H3(f"{precip_total:.1f} mm"),
                        html.P(f"Nivel: {nivel_alerta.upper()}", className="mb-0")
                    ])
                ])
            ], md=3),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H6("Máxima horaria", className="text-muted"),
                        html.H3(f"{precip_max_hora:.1f} mm/h"),
                        html.P(f"Hora: {hora_max_precip.strftime('%H:%M') if isinstance(hora_max_precip, pd.Timestamp) else 'N/A'}", 
                              className="mb-0")
                    ])
                ])
            ], md=3),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H6("Intensidad promedio", className="text-muted"),
                        html.H3(f"{df_24h['Precipitacion'].mean():.2f} mm/h"),
                        html.P("Últimas 24 horas", className="mb-0")
                    ])
                ])
            ], md=3),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H6("Horas con lluvia", className="text-muted"),
                        html.H3(f"{(df_24h['Precipitacion'] > 0).sum()} h"),
                        html.P(f"{(df_24h['Precipitacion'] > 0).sum() / len(df_24h) * 100:.0f}% del período", 
                              className="mb-0")
                    ])
                ])
            ], md=3)
        ], className="mb-4")
        
        # Tabla de intensidades
        tabla_intensidades = html.Div("Sin eventos de precipitación registrados", 
                                     className="text-muted")
        
        if intensidades:
            tabla_data = []
            for i in sorted(intensidades, key=lambda x: x['intensidad'], reverse=True)[:10]:
                tabla_data.append(html.Tr([
                    html.Td(i['hora'].strftime('%H:%M') if isinstance(i['hora'], pd.Timestamp) else str(i['hora'])),
                    html.Td(f"{i['intensidad']:.1f}"),
                    html.Td(i['categoria']),
                    html.Td(html.Span(
                        "●", 
                        style={'color': obtener_color_intensidad(i['categoria']),
                               'fontSize': '20px'}
                    ))
                ]))
            
            tabla_intensidades = dbc.Table([
                html.Thead([
                    html.Tr([
                        html.Th("Hora"),
                        html.Th("Intensidad (mm/h)"),
                        html.Th("Categoría"),
                        html.Th("Estado")
                    ])
                ]),
                html.Tbody(tabla_data)
            ], striped=True, hover=True, size="sm", className="mb-3")
        
        # Panel de riesgos
        panel_riesgos = dbc.Card([
            dbc.CardHeader(html.H5("Evaluación de Riesgos Agrícolas", className="mb-0")),
            dbc.CardBody([
                html.Div([
                    crear_indicador_riesgo(riesgo) for riesgo in riesgos
                ]) if riesgos else html.P("Sin riesgos significativos identificados", 
                                         className="text-muted")
            ])
        ], className="mb-3")
        
        # Recomendaciones
        recomendaciones = generar_recomendaciones_precipitacion(
            nivel_alerta, precip_total, precip_max_hora
        )
        
        panel_recomendaciones = dbc.Card([
            dbc.CardHeader(html.H5("Recomendaciones METGO_3D®", className="mb-0")),
            dbc.CardBody([
                html.Ul([
                    html.Li(rec) for rec in recomendaciones
                ])
            ])
        ])
        
        return html.Div([
            html.H4("Sistema de Alertas de Precipitación", className="text-primary mb-3"),
            alerta_component,
            stats_cards,
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=fig_acumulada)
                ], md=8),
                dbc.Col([
                    html.H5("Eventos de Mayor Intensidad", className="mb-3"),
                    tabla_intensidades
                ], md=4)
            ]),
            panel_riesgos,
            panel_recomendaciones
        ])
        
    except Exception as e:
        print(f"❌ Error en render_alerta_precipitacion: {str(e)}")
        import traceback
        traceback.print_exc()
        
        return html.Div([
            html.H4("Sistema de Alertas de Precipitación", className="text-primary mb-3"),
            dbc.Alert([
                html.I(className="fas fa-exclamation-triangle me-2"),
                html.Strong("Error al procesar datos de precipitación: "),
                html.P(str(e), className="mb-0"),
                html.Details([
                    html.Summary("Información técnica"),
                    html.Pre(traceback.format_exc(), style={'fontSize': '0.8em'})
                ])
            ], color="danger")
        ])

def categorizar_intensidad_lluvia(intensidad):
    """Categoriza la intensidad de la lluvia según estándares meteorológicos"""
    if intensidad == 0:
        return "Sin lluvia"
    elif intensidad < 2.5:
        return "Lluvia ligera"
    elif intensidad < 10:
        return "Lluvia moderada"
    elif intensidad < 50:
        return "Lluvia fuerte"
    else:
        return "Lluvia torrencial"

def obtener_color_intensidad(categoria):
    """Retorna el color asociado a cada categoría de intensidad"""
    colores = {
        "Sin lluvia": "#28a745",
        "Lluvia ligera": "#17a2b8",
        "Lluvia moderada": "#ffc107",
        "Lluvia fuerte": "#fd7e14",
        "Lluvia torrencial": "#dc3545"
    }
    return colores.get(categoria, "#6c757d")

def analizar_riesgos_precipitacion(total, maxima, intensidades):
    """Analiza los riesgos asociados a la precipitación para la agricultura"""
    riesgos = []
    
    # Riesgo de erosión
    if total > 30 or maxima > 20:
        riesgos.append({
            'tipo': 'Erosión del suelo',
            'nivel': 'alto' if total > 50 else 'moderado',
            'descripcion': 'Riesgo de pérdida de suelo fértil por escorrentía'
        })
    
    # Riesgo de anegamiento
    if total > 40:
        riesgos.append({
            'tipo': 'Anegamiento',
            'nivel': 'alto',
            'descripcion': 'Posible saturación del suelo y daño a raíces'
        })
    
    # Riesgo de enfermedades
    if len([i for i in intensidades if i['intensidad'] > 5]) > 3:
        riesgos.append({
            'tipo': 'Enfermedades fúngicas',
            'nivel': 'moderado',
            'descripcion': 'Condiciones favorables para hongos y bacterias'
        })
    
    # Riesgo de daño mecánico
    if maxima > 30:
        riesgos.append({
            'tipo': 'Daño mecánico',
            'nivel': 'alto',
            'descripcion': 'Posible daño a flores y frutos por impacto'
        })
    
    return riesgos

def crear_indicador_riesgo(riesgo):
    """Crea un indicador visual para cada tipo de riesgo"""
    colores = {
        'bajo': 'success',
        'moderado': 'warning',
        'alto': 'danger'
    }
    
    iconos = {
        'Erosión del suelo': 'fa-mountain',
        'Anegamiento': 'fa-water',
        'Enfermedades fúngicas': 'fa-virus',
        'Daño mecánico': 'fa-cloud-showers-heavy'
    }
    
    return dbc.Alert([
        html.Div([
            html.I(className=f"fas {iconos.get(riesgo['tipo'], 'fa-exclamation-circle')} me-2"),
            html.Strong(riesgo['tipo'], className="me-2"),
            dbc.Badge(riesgo['nivel'].upper(), color=colores.get(riesgo['nivel'], 'secondary'))
        ]),
        html.P(riesgo['descripcion'], className="mb-0 mt-2")
    ], color=colores.get(riesgo['nivel'], 'secondary'), className="mb-2")

def generar_recomendaciones_precipitacion(nivel, total, maxima):
    """Genera recomendaciones específicas según las condiciones de precipitación"""
    recomendaciones = []
    
    if nivel == "extremo":
        recomendaciones.extend([
            "⚠️ Suspender inmediatamente labores de campo por seguridad del personal",
            "🚜 Verificar sistemas de drenaje y evacuación de agua en parcelas",
            "🌱 Proteger cultivos sensibles con coberturas plásticas si es posible",
            "💧 Revisar y limpiar canales de riego para evitar desbordes",
            "📊 Monitorear niveles de humedad del suelo post-lluvia",
            "🔧 Preparar bombas de achique en zonas propensas a inundación",
            "📱 Mantener comunicación constante con el equipo de campo"
        ])
    elif nivel == "alto":
        recomendaciones.extend([
            "⏸️ Posponer aplicaciones de agroquímicos por al menos 48 horas",
            "🔍 Inspeccionar terrenos con pendiente por posible erosión",
            "🌾 Verificar anclaje de tutores y estructuras de soporte",
            "🦠 Preparar tratamientos preventivos fungicidas para aplicar cuando cese la lluvia",
            "📋 Documentar daños observados para seguimiento",
            "🚧 Reforzar bordos y camellones en cultivos susceptibles"
        ])
    elif nivel == "moderado":
        recomendaciones.extend([
            "✅ Aprovechar humedad para reducir riego en próximos días",
            "📅 Ajustar calendario de aplicaciones según pronóstico",
            "🌡️ Monitorear humedad relativa para prevenir enfermedades",
            "🌿 Revisar drenaje foliar en cultivos densos",
            "📈 Evaluar necesidad de fertilización post-lluvia"
        ])
    else:
        recomendaciones.extend([
            "✅ Condiciones normales para labores agrícolas",
            "💧 Mantener programación regular de riego",
            "📊 Continuar monitoreo rutinario de variables climáticas",
            "🌱 Proceder con actividades planificadas"
        ])
    
    # Recomendaciones adicionales basadas en intensidad
    if maxima > 20:
        recomendaciones.append("⚡ Revisar daños por impacto directo en frutos y flores")
    
    if total > 20:
        recomendaciones.append("🏗️ Verificar infraestructura de invernaderos y mallas")
    
    return recomendaciones

def crear_grafico_tendencia_precipitacion(df_historico):
    """Crea gráfico de tendencia histórica de precipitación"""
    if df_historico.empty:
        return go.Figure().add_annotation(
            text="Sin datos históricos disponibles",
            xref="paper", yref="paper",
            x=0.5, y=0.5, showarrow=False
        )
    
    # Agrupar por día
    df_diario = df_historico.groupby(df_historico['Fecha'].dt.date).agg({
        'Precipitacion': 'sum'
    }).reset_index()
    
    fig = go.Figure()
    
    # Barras de precipitación diaria
    fig.add_trace(go.Bar(
        x=df_diario['Fecha'],
        y=df_diario['Precipitacion'],
        name='Precipitación diaria',
        marker_color='lightblue',
        opacity=0.7
    ))
    
    # Media móvil de 7 días
    if len(df_diario) >= 7:
        df_diario['media_movil'] = df_diario['Precipitacion'].rolling(window=7).mean()
        fig.add_trace(go.Scatter(
            x=df_diario['Fecha'],
            y=df_diario['media_movil'],
            name='Media móvil 7 días',
            line=dict(color='darkblue', width=2)
        ))
    
    fig.update_layout(
        title="Tendencia Histórica de Precipitación",
        xaxis_title="Fecha",
        yaxis_title="Precipitación (mm)",
        hovermode='x unified',
        showlegend=True,
        height=350
    )
    
    return fig

def calcular_estadisticas_precipitacion(df):
    """Calcula estadísticas detalladas de precipitación"""
    stats = {
        'total': df['Precipitacion'].sum(),
        'promedio': df['Precipitacion'].mean(),
        'maximo': df['Precipitacion'].max(),
        'dias_lluvia': (df.groupby(df['Fecha'].dt.date)['Precipitacion'].sum() > 0.1).sum(),
        'percentil_95': df['Precipitacion'].quantile(0.95),
        'coef_variacion': df['Precipitacion'].std() / df['Precipitacion'].mean() if df['Precipitacion'].mean() > 0 else 0
    }
    return stats

def generar_alerta_texto(nivel, stats):
    """Genera texto descriptivo para las alertas"""
    textos = {
        'normal': f"Las condiciones de precipitación son normales. Total acumulado: {stats['total']:.1f} mm",
        'moderado': f"Se registra precipitación moderada. Total acumulado: {stats['total']:.1f} mm. Se recomienda precaución.",
        'alto': f"ALERTA: Precipitación intensa detectada. Total acumulado: {stats['total']:.1f} mm. Tome medidas preventivas.",
        'extremo': f"ALERTA EXTREMA: Precipitación muy intensa. Total acumulado: {stats['total']:.1f} mm. Suspenda actividades de campo."
    }
    return textos.get(nivel, "Evaluando condiciones...")

def evaluar_impacto_cultivos(precipitacion, cultivo_tipo="general"):
    """Evalúa el impacto de la precipitación en diferentes tipos de cultivos"""
    impactos = {
        'hortalizas': {
            'umbral_bajo': 5,
            'umbral_alto': 30,
            'sensibilidad': 'alta',
            'riesgos': ['anegamiento', 'enfermedades', 'pérdida de calidad']
        },
        'frutales': {
            'umbral_bajo': 10,
            'umbral_alto': 50,
            'sensibilidad': 'media',
            'riesgos': ['caída de frutos', 'rajado', 'pudriciones']
        },
        'cereales': {
            'umbral_bajo': 15,
            'umbral_alto': 60,
            'sensibilidad': 'baja',
            'riesgos': ['acame', 'germinación en espiga', 'enfermedades']
        }
    }
    
    if cultivo_tipo in impactos:
        info = impactos[cultivo_tipo]
        if precipitacion > info['umbral_alto']:
            return {
                'nivel': 'crítico',
                'mensaje': f"Precipitación excede umbral crítico para {cultivo_tipo}",
                'riesgos': info['riesgos']
            }
        elif precipitacion > info['umbral_bajo']:
            return {
                'nivel': 'moderado',
                'mensaje': f"Precipitación en rango de precaución para {cultivo_tipo}",
                'riesgos': info['riesgos'][:2]
            }
    
    return {
        'nivel': 'normal',
        'mensaje': "Precipitación dentro de rangos normales",
        'riesgos': []
    }

def crear_mapa_precipitacion(estaciones_data):
    """Crea un mapa de calor con la precipitación por estación"""
    if not estaciones_data:
        return go.Figure()
    
    lats = [est['lat'] for est in estaciones_data]
    lons = [est['lon'] for est in estaciones_data]
    precips = [est['precipitacion'] for est in estaciones_data]
    nombres = [est['nombre'] for est in estaciones_data]
    
    fig = go.Figure()
    
    # Mapa de calor
    fig.add_trace(go.Densitymapbox(
        lat=lats,
        lon=lons,
        z=precips,
        radius=30,
        colorscale='Blues',
        showscale=True,
        colorbar=dict(title="Precipitación (mm)")
    ))
    
    # Marcadores de estaciones
    fig.add_trace(go.Scattermapbox(
        lat=lats,
        lon=lons,
        mode='markers+text',
        marker=dict(size=15, color='darkblue'),
        text=nombres,
        textposition="top center",
        hovertemplate='<b>%{text}</b><br>Precipitación: %{customdata:.1f} mm<extra></extra>',
        customdata=precips
    ))
    
    fig.update_layout(
        mapbox=dict(
            style="carto-positron",
            center=dict(lat=sum(lats)/len(lats), lon=sum(lons)/len(lons)),
            zoom=10
        ),
        height=400,
        margin=dict(l=0, r=0, t=0, b=0)
    )
    
    return fig

def exportar_reporte_precipitacion(df, nivel_alerta, recomendaciones):
    """Genera un reporte exportable de precipitación"""
    reporte = {
        'fecha_generacion': datetime.now().strftime('%Y-%m-%d %H:%M'),
        'periodo_analisis': {
            'inicio': df['Fecha'].min().strftime('%Y-%m-%d %H:%M'),
            'fin': df['Fecha'].max().strftime('%Y-%m-%d %H:%M')
        },
        'resumen': {
            'precipitacion_total': df['Precipitacion'].sum(),
            'precipitacion_maxima': df['Precipitacion'].max(),
            'horas_con_lluvia': (df['Precipitacion'] > 0).sum(),
            'intensidad_promedio': df[df['Precipitacion'] > 0]['Precipitacion'].mean() if (df['Precipitacion'] > 0).any() else 0,
            'nivel_alerta': nivel_alerta
        },
        'estadisticas_detalladas': {
            'percentiles': {
                'p25': df['Precipitacion'].quantile(0.25),
                'p50': df['Precipitacion'].quantile(0.50),
                'p75': df['Precipitacion'].quantile(0.75),
                'p95': df['Precipitacion'].quantile(0.95)
            },
            'distribucion_horaria': df.groupby(df['Fecha'].dt.hour)['Precipitacion'].sum().to_dict(),
            'eventos_intensos': df[df['Precipitacion'] > 10][['Fecha', 'Precipitacion']].to_dict('records')
        },
        'recomendaciones': recomendaciones,
        'datos_horarios': df[['Fecha', 'Precipitacion']].to_dict('records')
    }
    
    return reporte

def validar_datos_precipitacion(df):
    """Valida la calidad de los datos de precipitación"""
    validacion = {
        'total_registros': len(df),
        'registros_validos': len(df[df['Precipitacion'].notna()]),
        'registros_nulos': len(df[df['Precipitacion'].isna()]),
        'registros_negativos': len(df[df['Precipitacion'] < 0]),
        'registros_extremos': len(df[df['Precipitacion'] > 100]),  # más de 100mm/hora es sospechoso
        'porcentaje_completitud': (len(df[df['Precipitacion'].notna()]) / len(df) * 100) if len(df) > 0 else 0
    }
    
    # Corregir datos problemáticos
    df.loc[df['Precipitacion'] < 0, 'Precipitacion'] = 0
    df['Precipitacion'] = df['Precipitacion'].fillna(0)
    
    return validacion, df

def calcular_probabilidad_lluvia(df_historico, umbral=0.1):
    """Calcula la probabilidad de lluvia basada en datos históricos"""
    if df_historico.empty:
        return {}
    
    # Agrupar por hora del día
    prob_por_hora = {}
    for hora in range(24):
        datos_hora = df_historico[df_historico['Fecha'].dt.hour == hora]
        if len(datos_hora) > 0:
            prob_por_hora[hora] = (datos_hora['Precipitacion'] > umbral).mean() * 100
    
    # Agrupar por día de la semana
    prob_por_dia = {}
    for dia in range(7):
        datos_dia = df_historico[df_historico['Fecha'].dt.dayofweek == dia]
        if len(datos_dia) > 0:
            prob_por_dia[dia] = (datos_dia['Precipitacion'] > umbral).mean() * 100
    
    # Agrupar por mes
    prob_por_mes = {}
    for mes in range(1, 13):
        datos_mes = df_historico[df_historico['Fecha'].dt.month == mes]
        if len(datos_mes) > 0:
            prob_por_mes[mes] = (datos_mes['Precipitacion'] > umbral).mean() * 100
    
    return {
        'por_hora': prob_por_hora,
        'por_dia_semana': prob_por_dia,
        'por_mes': prob_por_mes,
        'general': (df_historico['Precipitacion'] > umbral).mean() * 100
    }

def generar_grafico_probabilidad(probabilidades):
    """Genera gráfico de probabilidad de precipitación"""
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Probabilidad por Hora', 'Por Día de Semana', 'Por Mes'),
        horizontal_spacing=0.1
    )
    
    # Por hora
    horas = list(probabilidades['por_hora'].keys())
    valores_hora = list(probabilidades['por_hora'].values())
    fig.add_trace(
        go.Bar(x=horas, y=valores_hora, name='Por hora', marker_color='lightblue'),
        row=1, col=1
    )
    
    # Por día de semana
    dias_nombres = ['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom']
    dias = list(probabilidades['por_dia_semana'].keys())
    valores_dia = list(probabilidades['por_dia_semana'].values())
    fig.add_trace(
        go.Bar(x=[dias_nombres[d] for d in dias], y=valores_dia, 
               name='Por día', marker_color='lightgreen'),
        row=1, col=2
    )
    
    # Por mes
    meses_nombres = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 
                     'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
    meses = list(probabilidades['por_mes'].keys())
    valores_mes = list(probabilidades['por_mes'].values())
    fig.add_trace(
        go.Bar(x=[meses_nombres[m-1] for m in meses], y=valores_mes, 
               name='Por mes', marker_color='lightcoral'),
        row=1, col=3
    )
    
    fig.update_yaxes(title_text="Probabilidad (%)", range=[0, 100])
    fig.update_layout(
        title=f"Análisis de Probabilidad de Precipitación (Prob. general: {probabilidades['general']:.1f}%)",
        showlegend=False,
        height=300
    )
    
    return fig

def crear_resumen_ejecutivo_precipitacion(df_24h, df_historico):
    """Crea un resumen ejecutivo de la situación de precipitación"""
    # Calcular métricas actuales
    precip_24h = df_24h['Precipitacion'].sum()
    precip_max = df_24h['Precipitacion'].max()
    
    # Calcular percentiles históricos
    if not df_historico.empty:
        # Agrupar histórico por períodos de 24h
        historico_24h = []
        for i in range(0, len(df_historico), 24):
            historico_24h.append(df_historico.iloc[i:i+24]['Precipitacion'].sum())
        
        if historico_24h:
            percentil_actual = stats.percentileofscore(historico_24h, precip_24h)
        else:
            percentil_actual = 50
    else:
        percentil_actual = 50
    
    # Determinar situación
    if percentil_actual >= 95:
        situacion = "Precipitación excepcional"
        color = "danger"
    elif percentil_actual >= 85:
        situacion = "Precipitación muy alta"
        color = "warning"
    elif percentil_actual >= 70:
        situacion = "Precipitación moderada-alta"
        color = "info"
    else:
        situacion = "Precipitación normal"
        color = "success"
    
    resumen = dbc.Card([
        dbc.CardHeader([
            html.H5([
                html.I(className="fas fa-chart-line me-2"),
                "Resumen Ejecutivo - Precipitación"
            ], className="mb-0")
        ]),
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    html.H6("Situación Actual", className="text-muted"),
                    dbc.Badge(situacion, color=color, className="fs-5")
                ], md=6),
                dbc.Col([
                    html.H6("Percentil Histórico", className="text-muted"),
                    html.H3(f"{percentil_actual:.0f}°")
                ], md=6)
            ]),
            html.Hr(),
            html.P([
                f"La precipitación acumulada en las últimas 24 horas ({precip_24h:.1f} mm) ",
                f"se encuentra en el percentil {percentil_actual:.0f} respecto a los registros históricos. ",
                f"La intensidad máxima registrada fue de {precip_max:.1f} mm/h."
            ], className="mb-0")
        ])
    ], className="mb-3")
    
    return resumen

def generar_pronostico_precipitacion(df_historico, horizonte=72):
    """Genera pronóstico de precipitación usando Prophet o ARIMA"""
    try:
        from prophet import Prophet
        
        # Preparar datos para Prophet
        df_prophet = df_historico[['Fecha', 'Precipitacion']].copy()
        df_prophet.columns = ['ds', 'y']
        
        # Crear y entrenar modelo
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=True,
            uncertainty_samples=100
        )
        model.fit(df_prophet)
        
        # Generar pronóstico
        future = model.make_future_dataframe(periods=horizonte, freq='H')
        forecast = model.predict(future)
        
        # Asegurar valores no negativos
        forecast['yhat'] = forecast['yhat'].clip(lower=0)
        
        return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(horizonte)
        
    except:
        # Fallback: pronóstico simple basado en medias móviles
        return pd.DataFrame({
            'ds': pd.date_range(start=df_historico['Fecha'].max(), 
                               periods=horizonte+1, freq='H')[1:],
            'yhat': [df_historico['Precipitacion'].mean()] * horizonte,
            'yhat_lower': [0] * horizonte,
            'yhat_upper': [df_historico['Precipitacion'].quantile(0.95)] * horizonte
        })

# Función principal actualizada para integrar todas las funcionalidades
def render_monitor_completo(df_actual, df_historico, estacion):
    """Renderiza el monitor completo con todas las alertas y análisis"""
    try:
        # Filtrar datos de las últimas 24 horas
        tiempo_actual = pd.Timestamp.now()
        hace_24h = tiempo_actual - pd.Timedelta(hours=24)
        df_24h = df_actual[df_actual['Fecha'] >= hace_24h].copy()
        
        # Validar datos
        validacion, df_24h = validar_datos_precipitacion(df_24h)
        
        # Componentes principales
        alerta_precipitacion = render_alerta_precipitacion(df_24h)
        
        # Resumen ejecutivo
        resumen_ejecutivo = crear_resumen_ejecutivo_precipitacion(df_24h, df_historico)
        
        # Probabilidades
        probabilidades = calcular_probabilidad_lluvia(df_historico)
        if probabilidades:
            grafico_prob = generar_grafico_probabilidad(probabilidades)
        else:
            grafico_prob = None
        
        # Pronóstico
        pronostico = generar_pronostico_precipitacion(df_historico)
        
        return html.Div([
            resumen_ejecutivo,
            alerta_precipitacion,
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=grafico_prob) if grafico_prob else html.Div("Sin datos históricos")
                ], md=12)
            ]) if probabilidades else None
        ])
        
    except Exception as e:
        print(f"❌ Error en render_monitor_completo: {str(e)}")
        return html.Div([
            dbc.Alert([
                html.I(className="fas fa-exclamation-triangle me-2"),
                f"Error al renderizar monitor: {str(e)}"
            ], color="danger")
        ])
    
    return recomendaciones

# ============================================================================
# FUNCIONES AUXILIARES PARA GRÁFICOS
# ============================================================================

def crear_grafico_temperatura_detallado(df, indice_confort_actual):
    """Crear gráfico detallado de temperatura con múltiples métricas"""
    
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.7, 0.3],
        subplot_titles=("Temperatura y Sensación Térmica", "Gradiente Térmico")
    )
    
    # Gráfico principal - Temperatura
    fig.add_trace(
        go.Scatter(
            x=df['fecha_hora'],
            y=df['temperatura'],
            mode='lines+markers',
            name='Temperatura',
            line=dict(color=COLORES_GRAFICOS['temperatura'], width=3),
            marker=dict(size=6),
            hovertemplate='<b>Temperatura</b>: %{y:.1f}°C<br>Hora: %{x}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Sensación térmica
    fig.add_trace(
        go.Scatter(
            x=df['fecha_hora'],
            y=df['temperatura_aparente'],
            mode='lines',
            name='Sensación Térmica',
            line=dict(color='orange', width=2, dash='dash'),
            hovertemplate='<b>Sensación</b>: %{y:.1f}°C<br>Hora: %{x}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Banda de temperatura máxima y mínima del día
    temp_max_dia = df.groupby(df['fecha_hora'].dt.date)['temperatura'].transform('max')
    temp_min_dia = df.groupby(df['fecha_hora'].dt.date)['temperatura'].transform('min')
    
    fig.add_trace(
        go.Scatter(
            x=df['fecha_hora'],
            y=temp_max_dia,
            mode='lines',
            name='Máxima del día',
            line=dict(color=COLORES_GRAFICOS['temperatura_max'], width=1, dash='dot'),
            showlegend=True
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['fecha_hora'],
            y=temp_min_dia,
            mode='lines',
            name='Mínima del día',
            line=dict(color=COLORES_GRAFICOS['temperatura_min'], width=1, dash='dot'),
            fill='tonexty',
            fillcolor='rgba(100, 100, 200, 0.1)',
            showlegend=True
        ),
        row=1, col=1
    )
    
    # Gradiente térmico (cambio de temperatura)
    df['gradiente'] = df['temperatura'].diff() / df['fecha_hora'].diff().dt.total_seconds() * 3600
    
    fig.add_trace(
        go.Bar(
            x=df['fecha_hora'],
            y=df['gradiente'],
            name='Cambio T° (°C/h)',
            marker_color=df['gradiente'].apply(
                lambda x: '#FF5252' if x > 0 else '#2196F3'
            ),
            hovertemplate='<b>Cambio</b>: %{y:.2f}°C/h<br>Hora: %{x}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Línea de referencia en 0 para gradiente
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)
    
    # Actualizar diseño
    fig.update_xaxes(title_text="Hora", row=2, col=1)
    fig.update_yaxes(title_text="Temperatura (°C)", row=1, col=1)
    fig.update_yaxes(title_text="°C/hora", row=2, col=1)
    
    # Añadir anotación con índice de confort actual
    fig.add_annotation(
        text=f"Índice de Confort: {indice_confort_actual:.1f}°C",
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.8)",
        bordercolor="gray",
        borderwidth=1
    )
    
    fig.update_layout(
        height=600,
        hovermode='x unified',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig

def crear_grafico_precipitacion_humedad(df):
    """Crear gráfico de precipitación con colores diferenciados y humedad"""
    
    fig = make_subplots(
        rows=1, cols=1,
        specs=[[{"secondary_y": True}]]
    )
    
    # Crear colores para cada barra según intensidad
    colores_precip = [get_color_precipitacion(p) for p in df['precipitacion']]
    
    # Precipitación con colores diferenciados
    fig.add_trace(
        go.Bar(
            x=df['fecha_hora'],
            y=df['precipitacion'],
            name='Precipitación',
            marker_color=colores_precip,
            customdata=df['precipitacion'],
            hovertemplate='<b>Precipitación</b>: %{y:.1f} mm<br>%{x}<br>' +
                         '<b>Intensidad</b>: %{customdata:.1f} mm/h<extra></extra>',
            yaxis='y'
        ),
        secondary_y=False
    )
    
        # Humedad relativa
    fig.add_trace(
        go.Scatter(
            x=df['fecha_hora'],
            y=df['humedad_relativa'],
            mode='lines+markers',
            name='Humedad Relativa',
            line=dict(color=COLORES_GRAFICOS['humedad'], width=3),
            marker=dict(size=4),
            hovertemplate='<b>Humedad</b>: %{y:.0f}%<br>%{x}<extra></extra>',
            yaxis='y2'
        ),
        secondary_y=True
    )
    
    # Agregar áreas de referencia para humedad
    fig.add_hrect(
        y0=0, y1=30,
        fillcolor="orange", opacity=0.1,
        layer="below", line_width=0,
        secondary_y=True,
        annotation_text="Muy Seco", annotation_position="right"
    )
    
    fig.add_hrect(
        y0=30, y1=60,
        fillcolor="green", opacity=0.1,
        layer="below", line_width=0,
        secondary_y=True,
        annotation_text="Óptimo", annotation_position="right"
    )
    
    fig.add_hrect(
        y0=80, y1=100,
        fillcolor="blue", opacity=0.1,
        layer="below", line_width=0,
        secondary_y=True,
        annotation_text="Muy Húmedo", annotation_position="right"
    )
    
    # Calcular precipitación acumulada
    df_sorted = df.sort_values('fecha_hora')
    precip_acum = df_sorted['precipitacion'].cumsum()
    
    # Añadir línea de precipitación acumulada
    fig.add_trace(
        go.Scatter(
            x=df_sorted['fecha_hora'],
            y=precip_acum,
            mode='lines',
            name='Precip. Acumulada',
            line=dict(color='darkblue', width=2, dash='dot'),
            hovertemplate='<b>Acumulado</b>: %{y:.1f} mm<extra></extra>',
            yaxis='y'
        ),
        secondary_y=False
    )
    
    # Actualizar ejes
    fig.update_xaxes(title_text="Hora")
    fig.update_yaxes(title_text="Precipitación (mm)", secondary_y=False)
    fig.update_yaxes(title_text="Humedad Relativa (%)", secondary_y=True)
    
    # Añadir leyenda de intensidad de precipitación
    fig.add_annotation(
        text="<b>Intensidad de Precipitación:</b><br>" +
             "🟢 Leve (0-2mm) | 🟡 Moderada (2-10mm)<br>" +
             "🟠 Fuerte (10-25mm) | 🔴 Intensa (25-50mm)<br>" +
             "🟣 Torrencial (>50mm)",
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.9)",
        bordercolor="gray",
        borderwidth=1,
        font=dict(size=10)
    )
    
    fig.update_layout(
        height=500,
        hovermode='x unified',
        bargap=0.1,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5
        )
    )
    
    return fig

def crear_grafico_viento(df):
    """Crear gráfico polar de viento con velocidad y dirección"""
    
    # Preparar datos para gráfico polar
    df_viento = df[['fecha_hora', 'velocidad_viento', 'direccion_viento', 'rafaga_viento']].copy()
    
    # Crear subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Rosa de Vientos', 'Velocidad del Viento', 
                       'Distribución de Direcciones', 'Análisis de Ráfagas'),
        specs=[[{'type': 'polar'}, {'type': 'scatter'}],
               [{'type': 'bar'}, {'type': 'scatter'}]],
        row_heights=[0.6, 0.4]
    )
    
    # Rosa de vientos
    fig.add_trace(
        go.Scatterpolar(
            r=df_viento['velocidad_viento'],
            theta=df_viento['direccion_viento'],
            mode='markers',
            marker=dict(
                size=8,
                color=df_viento['velocidad_viento'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="km/h", x=0.45)
            ),
            customdata=df_viento['fecha_hora'],
            hovertemplate='<b>Velocidad</b>: %{r:.1f} km/h<br>' +
                         '<b>Dirección</b>: %{theta}°<br>' +
                         '<b>Hora</b>: %{customdata}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Velocidad del viento en el tiempo
    fig.add_trace(
        go.Scatter(
            x=df_viento['fecha_hora'],
            y=df_viento['velocidad_viento'],
            mode='lines+markers',
            name='Velocidad',
            line=dict(color=COLORES_GRAFICOS['viento'], width=2),
            fill='tozeroy',
            fillcolor='rgba(255, 167, 38, 0.2)'
        ),
        row=1, col=2
    )
    
    # Añadir ráfagas
    fig.add_trace(
        go.Scatter(
            x=df_viento['fecha_hora'],
            y=df_viento['rafaga_viento'],
            mode='lines',
            name='Ráfagas',
            line=dict(color='red', width=2, dash='dash')
        ),
        row=1, col=2
    )
    
    # Distribución de direcciones (histograma circular)
    direcciones_bins = np.histogram(df_viento['direccion_viento'], bins=16, range=(0, 360))
    direcciones_centro = (direcciones_bins[1][:-1] + direcciones_bins[1][1:]) / 2
    
    fig.add_trace(
        go.Bar(
            x=['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE',
               'S', 'SSO', 'SO', 'OSO', 'O', 'ONO', 'NO', 'NNO'],
            y=direcciones_bins[0],
            marker_color=COLORES_GRAFICOS['viento']
        ),
        row=2, col=1
    )
    
    # Análisis de ráfagas vs velocidad
    fig.add_trace(
        go.Scatter(
            x=df_viento['velocidad_viento'],
            y=df_viento['rafaga_viento'],
            mode='markers',
            marker=dict(
                size=8,
                color=df_viento.index,
                colorscale='Plasma',
                showscale=False
            ),
            hovertemplate='<b>Velocidad</b>: %{x:.1f} km/h<br>' +
                         '<b>Ráfaga</b>: %{y:.1f} km/h<extra></extra>'
        ),
        row=2, col=2
    )
    
    # Línea de referencia 1:1
    max_vel = max(df_viento['velocidad_viento'].max(), df_viento['rafaga_viento'].max())
    fig.add_trace(
        go.Scatter(
            x=[0, max_vel],
            y=[0, max_vel],
            mode='lines',
            line=dict(color='gray', dash='dash'),
            showlegend=False
        ),
        row=2, col=2
    )
    
    # Actualizar diseño
    fig.update_layout(
        height=800,
        showlegend=True,
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, df_viento['velocidad_viento'].max() * 1.1]
            )
        )
    )
    
    fig.update_xaxes(title_text="Hora", row=1, col=2)
    fig.update_yaxes(title_text="Velocidad (km/h)", row=1, col=2)
    fig.update_xaxes(title_text="Dirección", row=2, col=1)
    fig.update_yaxes(title_text="Frecuencia", row=2, col=1)
    fig.update_xaxes(title_text="Velocidad (km/h)", row=2, col=2)
    fig.update_yaxes(title_text="Ráfaga (km/h)", row=2, col=2)
    
    return fig

def crear_grafico_radiacion_et(df):
    """Crear gráfico de radiación solar y evapotranspiración"""
    
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=('Radiación Solar e Índice UV', 'Evapotranspiración y Balance Hídrico')
    )
    
    # Radiación solar
    fig.add_trace(
        go.Scatter(
            x=df['fecha_hora'],
            y=df['radiacion_solar'],
            mode='lines',
            name='Radiación Solar',
            line=dict(color=COLORES_GRAFICOS['radiacion'], width=3),
            fill='tozeroy',
            fillcolor='rgba(255, 215, 79, 0.3)',
            hovertemplate='<b>Radiación</b>: %{y:.0f} W/m²<br>%{x}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Índice UV en eje secundario
    fig.add_trace(
        go.Scatter(
            x=df['fecha_hora'],
            y=df['indice_uv'],
            mode='lines+markers',
            name='Índice UV',
            line=dict(color='purple', width=2),
            marker=dict(size=6),
            yaxis='y2',
            hovertemplate='<b>UV</b>: %{y:.1f}<br>%{x}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Evapotranspiración
    fig.add_trace(
        go.Bar(
            x=df['fecha_hora'],
            y=df['evapotranspiracion'],
            name='Evapotranspiración',
            marker_color=COLORES_GRAFICOS['evapotranspiracion'],
            hovertemplate='<b>ET</b>: %{y:.2f} mm<br>%{x}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Balance hídrico (precipitación - ET)
    df['balance_hidrico'] = df['precipitacion'] - df['evapotranspiracion']
    
    fig.add_trace(
        go.Scatter(
            x=df['fecha_hora'],
            y=df['balance_hidrico'].cumsum(),
            mode='lines',
            name='Balance Hídrico Acum.',
            line=dict(
                color='blue' if df['balance_hidrico'].sum() > 0 else 'red',
                width=3
            ),
            hovertemplate='<b>Balance</b>: %{y:.2f} mm<br>%{x}<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Línea de referencia en 0 para balance
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)
    
    # Añadir áreas de referencia para UV
    uv_levels = [
        (0, 2, 'green', 'Bajo'),
        (2, 5, 'yellow', 'Moderado'),
        (5, 7, 'orange', 'Alto'),
        (7, 10, 'red', 'Muy Alto'),
        (10, 15, 'purple', 'Extremo')
    ]
    
    for min_uv, max_uv, color, label in uv_levels:
        if df['indice_uv'].max() >= min_uv:
            fig.add_hrect(
                y0=min_uv, y1=min(max_uv, 15),
                fillcolor=color, opacity=0.1,
                layer="below", line_width=0,
                row=1, col=1,
                annotation_text=label,
                annotation_position="right"
            )
    
    # Actualizar diseño
    fig.update_xaxes(title_text="Hora", row=2, col=1)
    fig.update_yaxes(title_text="Radiación (W/m²)", row=1, col=1)
    fig.update_yaxes(title_text="Índice UV", secondary_y=True, row=1, col=1)
    fig.update_yaxes(title_text="mm", row=2, col=1)
    
    fig.update_layout(
        height=600,
        hovermode='x unified',
        showlegend=True
    )
    
    return fig

def crear_tabla_resumen_estadistico(df):
    """Crear tabla con resumen estadístico detallado"""
    
    # Calcular estadísticas
    stats = {
        'Variable': ['Temperatura', 'Humedad', 'Precipitación', 'Viento', 'Radiación', 'ET'],
        'Mínimo': [
            f"{df['temperatura'].min():.1f}°C",
            f"{df['humedad_relativa'].min():.0f}%",
            f"{df['precipitacion'].min():.1f} mm",
            f"{df['velocidad_viento'].min():.1f} km/h",
            f"{df['radiacion_solar'].min():.0f} W/m²",
            f"{df['evapotranspiracion'].min():.2f} mm"
        ],
        'Máximo': [
            f"{df['temperatura'].max():.1f}°C",
            f"{df['humedad_relativa'].max():.0f}%",
            f"{df['precipitacion'].max():.1f} mm",
            f"{df['velocidad_viento'].max():.1f} km/h",
            f"{df['radiacion_solar'].max():.0f} W/m²",
            f"{df['evapotranspiracion'].max():.2f} mm"
        ],
        'Promedio': [
            f"{df['temperatura'].mean():.1f}°C",
            f"{df['humedad_relativa'].mean():.0f}%",
            f"{df['precipitacion'].mean():.1f} mm",
            f"{df['velocidad_viento'].mean():.1f} km/h",
            f"{df['radiacion_solar'].mean():.0f} W/m²",
            f"{df['evapotranspiracion'].mean():.2f} mm"
        ],
        'Desv. Est.': [
            f"{df['temperatura'].std():.1f}°C",
            f"{df['humedad_relativa'].std():.0f}%",
            f"{df['precipitacion'].std():.1f} mm",
            f"{df['velocidad_viento'].std():.1f} km/h",
            f"{df['radiacion_solar'].std():.0f} W/m²",
            f"{df['evapotranspiracion'].std():.2f} mm"
        ],
        'Total/Acum.': [
            '-',
            '-',
            f"{df['precipitacion'].sum():.1f} mm",
            '-',
            f"{df['radiacion_solar'].sum()/1000:.1f} MJ/m²",
            f"{df['evapotranspiracion'].sum():.2f} mm"
        ]
    }
    
    df_stats = pd.DataFrame(stats)
    
    # Crear tabla con formato
    table = dash_table.DataTable(
        data=df_stats.to_dict('records'),
        columns=[{"name": i, "id": i} for i in df_stats.columns],
        style_cell={
            'textAlign': 'center',
            'padding': '10px',
            'fontFamily': 'Arial',
            'fontSize': '14px'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold',
            'border': '1px solid black'
        },
        style_data={
            'border': '1px solid gray',
            'backgroundColor': 'white'
        },
        style_data_conditional=[
            {
                'if': {'column_id': 'Variable'},
                'fontWeight': 'bold',
                'backgroundColor': 'rgb(240, 240, 240)'
            }
        ]
    )
    
    return table

# ============================================================================
# FUNCIONES DE RENDERIZADO - PRONÓSTICOS
# ============================================================================

def render_pronosticos(estacion_id, fecha_inicio, fecha_fin, cultivo):
    """Renderizar pronósticos meteorológicos con énfasis en heladas"""
    
    engine = crear_conexion_db()
    if not engine:
        return dbc.Alert("No hay conexión a la base de datos", color="warning")
    
    try:
        # Convertir fechas
        if isinstance(fecha_inicio, str):
            fecha_inicio = pd.to_datetime(fecha_inicio).date()
        if isinstance(fecha_fin, str):
            fecha_fin = pd.to_datetime(fecha_fin).date()
        
        # Obtener pronósticos
        query = """
            SELECT * FROM pronosticos 
            WHERE estacion_id = %(estacion_id)s 
            AND fecha_prediccion BETWEEN %(fecha_inicio)s AND %(fecha_fin)s
            AND fecha_pronostico >= NOW() - INTERVAL '1 day'
            ORDER BY fecha_prediccion
        """
        
        df_pronosticos = pd.read_sql(
            query, 
            engine, 
            params={
                'estacion_id': estacion_id,
                'fecha_inicio': fecha_inicio,
                'fecha_fin': fecha_fin
            }
        )
        engine.dispose()
        
        if df_pronosticos.empty:
            return dbc.Alert("No hay pronósticos disponibles para el período seleccionado", color="info")
        
        # Obtener configuración del cultivo
        config_cultivo = CULTIVOS_CONFIG.get(cultivo, CULTIVOS_CONFIG['palta'])
        
        return html.Div([
            # Panel de alertas de heladas
            render_panel_alertas_heladas(df_pronosticos, config_cultivo),
            
            # Gráfico principal de pronóstico
            html.Div([
                html.H5([
                    html.I(className="fas fa-chart-line me-2"),
                    "Pronóstico de Temperatura"
                ], className="mb-3"),
                dcc.Graph(
                    figure=crear_grafico_pronostico_temperatura(df_pronosticos, config_cultivo),
                    config={'displayModeBar': True, 'displaylogo': False}
                )
            ], className="graph-container"),
            
            # Pronóstico de precipitación
            html.Div([
                html.H5([
                    html.I(className="fas fa-cloud-rain me-2"),
                    "Pronóstico de Precipitación"
                ], className="mb-3"),
                dcc.Graph(
                    figure=crear_grafico_pronostico_precipitacion(df_pronosticos),
                    config={'displayModeBar': True, 'displaylogo': False}
                )
            ], className="graph-container"),
            
            # Tabla de pronóstico detallado
            html.Div([
                html.H5([
                    html.I(className="fas fa-table me-2"),
                    "Pronóstico Detallado por Día"
                ], className="mb-3"),
                crear_tabla_pronostico_detallado(df_pronosticos, config_cultivo)
            ], className="graph-container"),
            
            # Panel de recomendaciones
            render_panel_recomendaciones_pronostico(df_pronosticos, config_cultivo)
        ])
        
    except Exception as e:
        return dbc.Alert(f"Error cargando pronósticos: {str(e)}", color="danger")

def render_panel_alertas_heladas(df_pronosticos, config_cultivo):
    """Renderizar panel de alertas de heladas con niveles detallados"""
    
    # Filtrar pronósticos con riesgo de helada
    df_heladas = df_pronosticos[
        (df_pronosticos['temperatura_min_pred'] <= config_cultivo['temp_critica_helada']) |
        (df_pronosticos['probabilidad_helada'] > 0.3)
    ].copy()
    
    if df_heladas.empty:
        return dbc.Alert([
            html.I(className="fas fa-check-circle me-2"),
            "Sin riesgo de heladas en el período analizado"
        ], color="success", className="mb-3")
    
    # Crear cards de alerta por cada evento de helada
    alertas_helada = []
    
    for _, row in df_heladas.iterrows():
        temp_min = row['temperatura_min_pred']
        prob_helada = row.get('probabilidad_helada', 0) * 100
        fecha = pd.to_datetime(row['fecha_prediccion'])
        
        # Clasificar nivel de helada
        nivel, config_nivel = clasificar_nivel_helada(temp_min)
        
        # Crear card de alerta
        card_content = dbc.Card([
            dbc.CardHeader([
                html.Span(config_nivel['icono'], className="me-2"),
                f"{fecha.strftime('%d/%m/%Y')} - {config_nivel['descripcion']}"
            ], style={'backgroundColor': config_nivel['color'], 'color': 'white'}),
            dbc.CardBody([
                dbc.Row([
                    dbc.Col([
                        html.H6("Temperatura Mínima"),
                        html.H3(f"{temp_min:.1f}°C", className="text-center"),
                        dbc.Progress(
                            value=abs(temp_min - config_cultivo['temp_critica_helada']) * 20,
                            color="danger" if temp_min < 0 else "warning",
                            striped=True
                        )
                    ], md=4),
                    dbc.Col([
                        html.H6("Probabilidad"),
                        html.H3(f"{prob_helada:.0f}%", className="text-center"),
                        dbc.Progress(
                            value=prob_helada,
                            color="danger" if prob_helada > 70 else "warning",
                            striped=True
                        )
                    ], md=4),
                    dbc.Col([
                        html.H6("Impacto en Cultivo"),
                        html.P(f"🌱 {config_cultivo['nombre']}", className="mb-1"),
                        html.Small(f"T° crítica: {config_cultivo['temp_critica_helada']}°C"),
                        html.Br(),
                        html.Small(f"T° daño: {config_cultivo['temp_daño_helada']}°C")
                    ], md=4)
                ]),
                html.Hr(),
                html.H6("Acciones Recomendadas:"),
                html.Ul([
                    html.Li(accion, className="small") 
                    for accion in config_nivel['acciones'][:3]
                ])
            ])
        ], className="mb-3")
        
        alertas_helada.append(card_content)
    
    return html.Div([
        html.H5([
            html.I(className="fas fa-snowflake me-2"),
            f"Alertas de Heladas ({len(df_heladas)} eventos)"
        ], className="mb-3"),
        html.Div(alertas_helada)
    ])

def crear_grafico_pronostico_temperatura(df_pronosticos, config_cultivo):
    """Crear gráfico de pronóstico de temperatura con umbrales de cultivo"""
    
    # Agrupar por fecha para obtener valores diarios
    df_daily = df_pronosticos.groupby(pd.to_datetime(df_pronosticos['fecha_prediccion']).dt.date).agg({
        'temperatura_pred': 'mean',
        'temperatura_max_pred': 'max',
        'temperatura_min_pred': 'min',
        'probabilidad_helada': 'max'
    }).reset_index()
    df_daily.columns = ['fecha', 'temp_media', 'temp_max', 'temp_min', 'prob_helada']
    
    fig = go.Figure()
    
    # Banda de temperatura óptima del cultivo
    fig.add_hrect(
        y0=config_cultivo['temp_min'], 
        y1=config_cultivo['temp_max'],
        fillcolor="green", opacity=0.1,
        annotation_text=f"Rango óptimo {config_cultivo['nombre']}",
        annotation_position="top right"
    )
    
    # Línea de temperatura crítica de helada
    fig.add_hline(
        y=config_cultivo['temp_critica_helada'],
        line_dash="dash", line_color="orange", line_width=2,
        annotation_text=f"T° crítica helada ({config_cultivo['temp_critica_helada']}°C)"
    )
    
    # Línea de temperatura de daño por helada
    fig.add_hline(
        y=config_cultivo['temp_daño_helada'],
        line_dash="dash", line_color="red", line_width=2,
        annotation_text=f"T° daño helada ({config_cultivo['temp_daño_helada']}°C)"
    )
    
    # Temperatura máxima
    fig.add_trace(go.Scatter(
        x=df_daily['fecha'],
        y=df_daily['temp_max'],
        mode='lines+markers',
        name='Máxima',
        line=dict(color=COLORES_GRAFICOS['temperatura_max'], width=2),
        marker=dict(size=8)
    ))
    
    # Temperatura media
    fig.add_trace(go.Scatter(
        x=df_daily['fecha'],
        y=df_daily['temp_media'],
        mode='lines+markers',
        name='Media',
        line=dict(color=COLORES_GRAFICOS['temperatura'], width=3),
        marker=dict(size=8)
    ))
    
    # Temperatura mínima con color según riesgo
    colores_minima = []
    for temp in df_daily['temp_min']:
        if temp <= config_cultivo['temp_daño_helada']:
            colores_minima.append('darkred')
        elif temp <= config_cultivo['temp_critica_helada']:
            colores_minima.append('red')
        elif temp <= 5:
            colores_minima.append('orange')
        else:
            colores_minima.append(COLORES_GRAFICOS['temperatura_min'])
    
    fig.add_trace(go.Scatter(
        x=df_daily['fecha'],
        y=df_daily['temp_min'],
        mode='lines+markers',
        name='Mínima',
        line=dict(color=COLORES_GRAFICOS['temperatura_min'], width=2),
        marker=dict(size=10, color=colores_minima),
        customdata=df_daily['prob_helada'],
        hovertemplate='<b>T° Mínima</b>: %{y:.1f}°C<br>' +
                     '<b>Prob. Helada</b>: %{customdata:.0%}<br>' +
                     '<b>Fecha</b>: %{x}<extra></extra>'
    ))
    
    # Área de incertidumbre
    fig.add_trace(go.Scatter(
        x=df_daily['fecha'].tolist() + df_daily['fecha'].tolist()[::-1],
        y=df_daily['temp_max'].tolist() + df_daily['temp_min'].tolist()[::-1],
        fill='toself',
        fillcolor='rgba(100, 100, 200, 0.1)',
        line=dict(color='rgba(255,255,255,0)'),
        showlegend=False,
        hoverinfo='skip'
    ))
    
    fig.update_layout(
        title=f"Pronóstico de Temperatura - Cultivo: {config_cultivo['nombre']}",
        xaxis_title="Fecha",
        yaxis_title="Temperatura (°C)",
        height=600,
        hovermode='x unified',
        showlegend=True
    )
    
    return fig

def crear_grafico_pronostico_precipitacion(df_pronosticos):
    """Crear gráfico de pronóstico de precipitación"""
    
        # Agrupar por fecha
    df_daily = df_pronosticos.groupby(pd.to_datetime(df_pronosticos['fecha_prediccion']).dt.date).agg({
        'precipitacion_pred': 'sum',
        'humedad_pred': 'mean',
        'confianza': 'mean'
    }).reset_index()
    df_daily.columns = ['fecha', 'precipitacion', 'humedad', 'confianza']
    
    # Crear figura con subplots
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=('Precipitación Esperada', 'Probabilidad de Lluvia y Humedad'),
        row_heights=[0.6, 0.4]
    )
    
    # Precipitación con colores según intensidad
    colores_precip = [get_color_precipitacion(p) for p in df_daily['precipitacion']]
    
    fig.add_trace(
        go.Bar(
            x=df_daily['fecha'],
            y=df_daily['precipitacion'],
            name='Precipitación',
            marker_color=colores_precip,
            customdata=df_daily['confianza'],
            hovertemplate='<b>Precipitación</b>: %{y:.1f} mm<br>' +
                         '<b>Confianza</b>: %{customdata:.0%}<br>' +
                         '<b>Fecha</b>: %{x}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Precipitación acumulada
    precip_acum = df_daily['precipitacion'].cumsum()
    fig.add_trace(
        go.Scatter(
            x=df_daily['fecha'],
            y=precip_acum,
            mode='lines',
            name='Acumulado',
            line=dict(color='darkblue', width=2, dash='dot'),
            yaxis='y2'
        ),
        row=1, col=1
    )
    
    # Probabilidad de lluvia (basada en precipitación > 0.1mm)
    prob_lluvia = (df_daily['precipitacion'] > 0.1).astype(float) * df_daily['confianza'] * 100
    
    fig.add_trace(
        go.Scatter(
            x=df_daily['fecha'],
            y=prob_lluvia,
            mode='lines+markers',
            name='Prob. Lluvia',
            line=dict(color='blue', width=2),
            marker=dict(size=8)
        ),
        row=2, col=1
    )
    
    # Humedad esperada
    fig.add_trace(
        go.Scatter(
            x=df_daily['fecha'],
            y=df_daily['humedad'],
            mode='lines+markers',
            name='Humedad',
            line=dict(color=COLORES_GRAFICOS['humedad'], width=2),
            marker=dict(size=6),
            yaxis='y3'
        ),
        row=2, col=1
    )
    
    # Actualizar diseño
    fig.update_xaxes(title_text="Fecha", row=2, col=1)
    fig.update_yaxes(title_text="Precipitación (mm)", row=1, col=1)
    fig.update_yaxes(title_text="Acumulado (mm)", secondary_y=True, row=1, col=1)
    fig.update_yaxes(title_text="Probabilidad (%)", row=2, col=1)
    fig.update_yaxes(title_text="Humedad (%)", secondary_y=True, row=2, col=1)
    
    # Añadir anotación con total esperado
    total_precip = df_daily['precipitacion'].sum()
    fig.add_annotation(
        text=f"Total esperado: {total_precip:.1f} mm",
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        showarrow=False,
        bgcolor="white",
        bordercolor="gray"
    )
    
    fig.update_layout(
        height=600,
        hovermode='x unified',
        showlegend=True
    )
    
    return fig

def crear_tabla_pronostico_detallado(df_pronosticos, config_cultivo):
    """Crear tabla con pronóstico detallado por día"""
    
    # Agrupar por fecha
    df_daily = df_pronosticos.groupby(pd.to_datetime(df_pronosticos['fecha_prediccion']).dt.date).agg({
        'temperatura_max_pred': 'max',
        'temperatura_min_pred': 'min',
        'temperatura_pred': 'mean',
        'precipitacion_pred': 'sum',
        'humedad_pred': 'mean',
        'probabilidad_helada': 'max'
    }).reset_index()
    
    # Preparar datos para la tabla
    data_tabla = []
    for _, row in df_daily.iterrows():
        # Clasificar riesgo de helada
        nivel_helada, config_nivel = clasificar_nivel_helada(row['temperatura_min_pred'])
        
        # Determinar aptitud para el cultivo
        aptitud = "Óptimo"
        color_aptitud = "green"
        if row['temperatura_max_pred'] > config_cultivo['temp_max']:
            aptitud = "Calor excesivo"
            color_aptitud = "orange"
        elif row['temperatura_min_pred'] < config_cultivo['temp_min']:
            aptitud = "Frío"
            color_aptitud = "blue"
        if row['temperatura_min_pred'] <= config_cultivo['temp_critica_helada']:
            aptitud = "Riesgo helada"
            color_aptitud = "red"
        
        data_tabla.append({
            'Fecha': row['fecha_prediccion'].strftime('%d/%m'),
            'T° Máx': f"{row['temperatura_max_pred']:.1f}°C",
            'T° Mín': f"{row['temperatura_min_pred']:.1f}°C",
            'T° Media': f"{row['temperatura_pred']:.1f}°C",
            'Precipitación': f"{row['precipitacion_pred']:.1f} mm",
            'Humedad': f"{row['humedad_pred']:.0f}%",
            'Riesgo Helada': f"{config_nivel['icono']} {row['probabilidad_helada']*100:.0f}%",
            'Aptitud Cultivo': aptitud,
            '_color_aptitud': color_aptitud,
            '_color_helada': config_nivel['color']
        })
    
    # Crear tabla
    table = dash_table.DataTable(
        data=data_tabla,
        columns=[
            {"name": col, "id": col} 
            for col in data_tabla[0].keys() 
            if not col.startswith('_')
        ],
        style_cell={
            'textAlign': 'center',
            'padding': '10px',
            'fontSize': '14px'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {'row_index': i, 'column_id': 'Aptitud Cultivo'},
                'color': row['_color_aptitud'],
                'fontWeight': 'bold'
            } for i, row in enumerate(data_tabla)
        ] + [
            {
                'if': {'row_index': i, 'column_id': 'Riesgo Helada'},
                'backgroundColor': row['_color_helada'],
                'color': 'white' if row['_color_helada'] in ['#F44336', '#B71C1C'] else 'black'
            } for i, row in enumerate(data_tabla)
        ],
        style_data={
            'border': '1px solid gray'
        }
    )
    
    return table

def render_panel_recomendaciones_pronostico(df_pronosticos, config_cultivo):
    """Renderizar panel de recomendaciones basadas en el pronóstico"""
    
    # Analizar condiciones pronosticadas
    temp_min_periodo = df_pronosticos['temperatura_min_pred'].min()
    temp_max_periodo = df_pronosticos['temperatura_max_pred'].max()
    precip_total = df_pronosticos['precipitacion_pred'].sum()
    dias_con_helada = (df_pronosticos['temperatura_min_pred'] <= config_cultivo['temp_critica_helada']).sum()
    
    recomendaciones = []
    
    # Recomendaciones por temperatura
    if temp_min_periodo <= config_cultivo['temp_daño_helada']:
        recomendaciones.append({
            'tipo': 'critico',
            'icono': '🚨',
            'titulo': 'Helada Severa',
            'acciones': [
                'Activar TODOS los sistemas de protección antiheladas',
                'Preparar calefactores y ventiladores',
                'Considerar cosecha de emergencia de frutos maduros',
                'Contactar seguro agrícola'
            ]
        })
    elif temp_min_periodo <= config_cultivo['temp_critica_helada']:
        recomendaciones.append({
            'tipo': 'alerta',
            'icono': '❄️',
            'titulo': 'Riesgo de Helada',
            'acciones': [
                'Activar riego por aspersión antes del amanecer',
                'Preparar cobertores para plantas jóvenes',
                'Monitorear temperatura cada 2 horas',
                'Retrasar podas hasta superar el riesgo'
            ]
        })
    
    if temp_max_periodo > config_cultivo['temp_max'] + 5:
        recomendaciones.append({
            'tipo': 'alerta',
            'icono': '🌡️',
            'titulo': 'Estrés por Calor',
            'acciones': [
                'Aumentar frecuencia de riego en 30%',
                'Aplicar mulch para conservar humedad',
                'Considerar mallas de sombreo',
                'Evitar aplicaciones foliares en horas de calor'
            ]
        })
    
    # Recomendaciones por agua
    if precip_total < 5:
        recomendaciones.append({
            'tipo': 'info',
            'icono': '💧',
            'titulo': 'Período Seco',
            'acciones': [
                f'Aplicar riego según ET estimada ({config_cultivo["umbral_et"]} mm/día)',
                'Verificar funcionamiento del sistema de riego',
                'Monitorear humedad del suelo',
                'Priorizar riego en etapas críticas'
            ]
        })
    elif precip_total > 50:
        recomendaciones.append({
            'tipo': 'advertencia',
            'icono': '🌧️',
            'titulo': 'Lluvia Abundante',
            'acciones': [
                'Verificar drenaje del huerto',
                'Monitorear enfermedades fúngicas',
                'Posponer aplicaciones fitosanitarias',
                'Reducir o suspender riego'
            ]
        })
    
    # Crear cards de recomendaciones
    cards_recomendaciones = []
    
    for rec in recomendaciones:
        color = {
            'critico': 'danger',
            'alerta': 'warning',
            'advertencia': 'info',
            'info': 'primary'
        }.get(rec['tipo'], 'secondary')
        
        card = dbc.Card([
            dbc.CardHeader([
                html.Span(rec['icono'], className="me-2"),
                rec['titulo']
            ], className=f"bg-{color} text-white"),
            dbc.CardBody([
                html.Ul([
                    html.Li(accion) for accion in rec['acciones']
                ])
            ])
        ], className="mb-3")
        
        cards_recomendaciones.append(card)
    
    if not cards_recomendaciones:
        cards_recomendaciones.append(
            dbc.Alert([
                html.I(className="fas fa-check-circle me-2"),
                "Condiciones favorables - Mantener manejo habitual"
            ], color="success")
        )
    
    return html.Div([
        html.H5([
            html.I(className="fas fa-clipboard-list me-2"),
            "Recomendaciones de Manejo"
        ], className="mb-3"),
        dbc.Row([
            dbc.Col(cards_recomendaciones)
        ])
    ])

# ============================================================================
# FUNCIONES DE RENDERIZADO - ANÁLISIS DE CULTIVOS
# ============================================================================

def render_analisis_cultivos(estacion_id, cultivo):
    """Renderizar análisis específico para cultivos con alto detalle"""
    
    engine = crear_conexion_db()
    if not engine:
        return dbc.Alert("No hay conexión a la base de datos", color="warning")
    
    try:
        # Obtener última recomendación
        query_rec = """
            SELECT * FROM recomendaciones_cultivos 
            WHERE estacion_id = %(estacion_id)s AND cultivo = %(cultivo)s
            ORDER BY fecha_analisis DESC
            LIMIT 1
        """
        
        df_rec = pd.read_sql(query_rec, engine, params={'estacion_id': estacion_id, 'cultivo': cultivo})
        
        # Obtener datos históricos para análisis
        query_hist = """
            SELECT * FROM datos_meteorologicos 
            WHERE estacion_id = %(estacion_id)s 
            AND fecha_hora >= NOW() - INTERVAL '30 days'
            ORDER BY fecha_hora
        """
        
        df_hist = pd.read_sql(query_hist, engine, params={'estacion_id': estacion_id})
        engine.dispose()
        
        if df_rec.empty and df_hist.empty:
            return dbc.Alert("No hay datos disponibles para análisis", color="info")
        
        config = CULTIVOS_CONFIG[cultivo]
        rec = df_rec.iloc[0] if not df_rec.empty else None
        
        return html.Div([
                        # Encabezado con información del cultivo
            dbc.Card([
                dbc.CardHeader([
                    html.H4([
                        html.I(className="fas fa-seedling me-2"),
                        f"Análisis para {config['nombre']}"
                    ], className="mb-0")
                ], style={'backgroundColor': config['color_theme'], 'color': 'white'}),
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col([
                            html.H6("Condiciones Óptimas", className="text-muted"),
                            html.P([
                                html.Strong("Temperatura: "),
                                f"{config['temp_optima']}°C (rango: {config['temp_min']}-{config['temp_max']}°C)"
                            ], className="mb-1"),
                            html.P([
                                html.Strong("Humedad: "),
                                f"{config['humedad_optima']}% (rango: {config['humedad_min']}-{config['humedad_max']}%)"
                            ], className="mb-1"),
                            html.P([
                                html.Strong("pH del suelo: "),
                                f"{config['ph_suelo'][0]} - {config['ph_suelo'][1]}"
                            ])
                        ], md=4),
                        dbc.Col([
                            html.H6("Requerimientos Hídricos", className="text-muted"),
                            html.P([
                                html.Strong("Agua anual: "),
                                f"{config['requerimiento_agua']} mm"
                            ], className="mb-1"),
                            html.P([
                                html.Strong("ET diaria crítica: "),
                                f"{config['umbral_et']} mm/día"
                            ], className="mb-1"),
                            html.P([
                                html.Strong("Sensibilidad heladas: "),
                                f"Crítica a {config['temp_critica_helada']}°C"
                            ])
                        ], md=4),
                        dbc.Col([
                            html.H6("Calendario Fenológico", className="text-muted"),
                            html.P([
                                html.Strong("Siembra: "),
                                ', '.join([calendar.month_name[m][:3] for m in config['meses_siembra']])
                            ], className="mb-1"),
                            html.P([
                                html.Strong("Floración: "),
                                ', '.join([calendar.month_name[m][:3] for m in config['meses_floracion']])
                            ], className="mb-1"),
                            html.P([
                                html.Strong("Cosecha: "),
                                ', '.join([calendar.month_name[m][:3] for m in config['meses_cosecha']])
                            ])
                        ], md=4)
                    ])
                ])
            ], className="mb-4"),
            
            # Indicadores principales
            render_indicadores_cultivo(rec, config, df_hist),
            
            # Análisis fenológico
            html.Div([
                html.H5([
                    html.I(className="fas fa-calendar-alt me-2"),
                    "Estado Fenológico y Calendario"
                ], className="mb-3"),
                render_calendario_fenologico(cultivo, config, df_hist)
            ], className="graph-container"),
            
            # Análisis de condiciones actuales vs óptimas
            html.Div([
                html.H5([
                    html.I(className="fas fa-chart-radar me-2"),
                    "Condiciones Actuales vs Óptimas"
                ], className="mb-3"),
                dcc.Graph(
                    figure=crear_grafico_radar_cultivo(config, df_hist),
                    config={'displayModeBar': False}
                )
            ], className="graph-container"),
            
            # Balance hídrico del cultivo
            html.Div([
                html.H5([
                    html.I(className="fas fa-tint me-2"),
                    "Balance Hídrico del Cultivo"
                ], className="mb-3"),
                dcc.Graph(
                    figure=crear_grafico_balance_hidrico_cultivo(config, df_hist),
                    config={'displayModeBar': True, 'displaylogo': False}
                )
            ], className="graph-container"),
            
            # Análisis de riesgos
            render_analisis_riesgos_cultivo(config, df_hist),
            
            # Recomendaciones específicas
            render_recomendaciones_cultivo_detalladas(rec, config, df_hist)
        ])
        
    except Exception as e:
        return dbc.Alert(f"Error en análisis de cultivos: {str(e)}", color="danger")

def render_indicadores_cultivo(rec, config, df_hist):
    """Renderizar indicadores principales del cultivo"""
    
    # Calcular indicadores actuales si no hay recomendación guardada
    if rec is None:
        # Calcular aptitud basada en condiciones actuales
        temp_actual = df_hist['temperatura'].mean()
        humedad_actual = df_hist['humedad_relativa'].mean()
        
        aptitud_temp = 1.0 - abs(temp_actual - config['temp_optima']) / 10
        aptitud_humedad = 1.0 - abs(humedad_actual - config['humedad_optima']) / 20
        aptitud_siembra = (aptitud_temp * 0.6 + aptitud_humedad * 0.4)
        
        # Calcular riesgo de helada
        dias_riesgo = (df_hist.groupby(df_hist['fecha_hora'].dt.date)['temperatura'].min() <= config['temp_critica_helada']).sum()
        riesgo_helada = dias_riesgo / 30
        
        # Calcular necesidad de riego
        et_total = df_hist['evapotranspiracion'].sum()
        precip_total = df_hist['precipitacion'].sum()
        deficit = max(0, et_total - precip_total)
        necesidad_riego = min(1, deficit / (config['umbral_et'] * 30))
        
        fase_fenologica = determinar_fase_fenologica(config, datetime.now().month)
    else:
        aptitud_siembra = rec['aptitud_siembra']
        riesgo_helada = rec['riesgo_helada']
        necesidad_riego = rec['necesidad_riego']
        fase_fenologica = rec['fase_fenologica']
    
    # Crear cards de indicadores
    return dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("Aptitud para Siembra", className="text-center mb-3"),
                    html.Div([
                        html.Div(
                            className="circular-progress",
                            style={
                                'width': '120px',
                                'height': '120px',
                                'margin': '0 auto',
                                'position': 'relative'
                            },
                            children=[
                                dbc.Progress(
                                    value=aptitud_siembra * 100,
                                    color="success" if aptitud_siembra > 0.7 else "warning" if aptitud_siembra > 0.4 else "danger",
                                    style={
                                        'height': '120px',
                                        'borderRadius': '50%'
                                    }
                                ),
                                html.Div(
                                    f"{aptitud_siembra*100:.0f}%",
                                    style={
                                        'position': 'absolute',
                                        'top': '50%',
                                        'left': '50%',
                                        'transform': 'translate(-50%, -50%)',
                                        'fontSize': '24px',
                                        'fontWeight': 'bold'
                                    }
                                )
                            ]
                        )
                    ], className="text-center"),
                    html.P(
                        "Excelente" if aptitud_siembra > 0.8 else
                        "Buena" if aptitud_siembra > 0.6 else
                        "Regular" if aptitud_siembra > 0.4 else
                        "Desfavorable",
                        className="text-center mt-3 mb-0 fw-bold"
                    )
                ])
            ], className="h-100")
        ], lg=3, md=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("Riesgo de Helada", className="text-center mb-3"),
                    html.Div([
                        html.I(
                            className=f"fas fa-snowflake fa-5x",
                            style={
                                'color': '#B71C1C' if riesgo_helada > 0.7 else
                                        '#F44336' if riesgo_helada > 0.5 else
                                        '#FF9800' if riesgo_helada > 0.3 else
                                        '#4CAF50'
                            }
                        )
                    ], className="text-center"),
                    html.H4(f"{riesgo_helada*100:.0f}%", className="text-center mt-2"),
                    html.P(
                        "Extremo" if riesgo_helada > 0.7 else
                        "Alto" if riesgo_helada > 0.5 else
                        "Moderado" if riesgo_helada > 0.3 else
                        "Bajo",
                        className="text-center mb-0 fw-bold"
                    )
                ])
            ], className="h-100")
        ], lg=3, md=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("Necesidad de Riego", className="text-center mb-3"),
                    html.Div([
                        html.I(
                            className="fas fa-tint fa-5x",
                            style={
                                'color': '#1976D2' if necesidad_riego > 0.7 else
                                        '#2196F3' if necesidad_riego > 0.5 else
                                        '#03A9F4' if necesidad_riego > 0.3 else
                                        '#81D4FA'
                            }
                        )
                    ], className="text-center"),
                    html.H4(f"{necesidad_riego*100:.0f}%", className="text-center mt-2"),
                    html.P(
                        "Urgente" if necesidad_riego > 0.7 else
                        "Alta" if necesidad_riego > 0.5 else
                        "Moderada" if necesidad_riego > 0.3 else
                        "Baja",
                        className="text-center mb-0 fw-bold"
                    )
                ])
            ], className="h-100")
        ], lg=3, md=6, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("Fase Fenológica", className="text-center mb-3"),
                    html.Div([
                        html.I(
                            className=obtener_icono_fase(fase_fenologica),
                            style={'color': config['color_theme']}
                        )
                    ], className="text-center"),
                    html.H5(fase_fenologica.title(), className="text-center mt-2 mb-1"),
                    html.P(
                        obtener_descripcion_fase(fase_fenologica),
                        className="text-center mb-0 small"
                    )
                ])
            ], className="h-100")
        ], lg=3, md=6, className="mb-3")
    ])

def determinar_fase_fenologica(config, mes_actual):
    """Determinar fase fenológica según el mes"""
    if mes_actual in config['meses_siembra']:
        return 'siembra'
    elif mes_actual in config['meses_floracion']:
        return 'floracion'
    elif mes_actual in config['meses_cosecha']:
        return 'cosecha'
    else:
        return 'vegetativo'

def obtener_icono_fase(fase):
    """Obtener icono FontAwesome para cada fase"""
    iconos = {
        'siembra': 'fas fa-seedling fa-5x',
        'germinacion': 'fas fa-seedling fa-5x',
        'vegetativo': 'fas fa-leaf fa-5x',
        'floracion': 'fas fa-spa fa-5x',
        'fructificacion': 'fas fa-apple-alt fa-5x',
        'maduracion': 'fas fa-lemon fa-5x',
        'cosecha': 'fas fa-shopping-basket fa-5x'
    }
    return iconos.get(fase, 'fas fa-plant fa-5x')

def obtener_descripcion_fase(fase):
    """Obtener descripción de la fase fenológica"""
    descripciones = {
        'siembra': 'Período óptimo para establecimiento',
        'germinacion': 'Emergencia de plántulas',
        'vegetativo': 'Crecimiento activo',
        'floracion': 'Desarrollo de flores',
        'fructificacion': 'Formación de frutos',
        'maduracion': 'Maduración de frutos',
        'cosecha': 'Recolección de producción'
    }
    return descripciones.get(fase, 'En desarrollo')

def render_calendario_fenologico(cultivo, config, df_hist):
    """Renderizar calendario fenológico anual"""
    
    # Crear datos para el calendario
    meses = list(range(1, 13))
    fases = []
    colores = []
    
    for mes in meses:
        if mes in config['meses_siembra']:
            fases.append('Siembra')
            colores.append('#4CAF50')
        elif mes in config['meses_floracion']:
            fases.append('Floración')
            colores.append('#FF9800')
        elif mes in config['meses_cosecha']:
            fases.append('Cosecha')
            colores.append('#9C27B0')
        else:
            fases.append('Vegetativo')
            colores.append('#2196F3')
    
    # Crear gráfico de calendario
    fig = go.Figure()
    
    # Añadir barras para cada mes
    fig.add_trace(go.Bar(
        x=[calendar.month_name[m][:3] for m in meses],
        y=[1] * 12,
        marker_color=colores,
        text=fases,
        textposition='inside',
        hovertemplate='<b>%{x}</b><br>Fase: %{text}<extra></extra>'
    ))
    
        # Marcar mes actual
    mes_actual = datetime.now().month
    fig.add_vline(
        x=mes_actual - 1,
        line_width=3,
        line_dash="dash",
        line_color="red",
        annotation_text="HOY",
        annotation_position="top"
    )
    
    # Añadir indicadores de condiciones actuales
    if not df_hist.empty:
        temp_actual = df_hist['temperatura'].tail(24).mean()
        condicion = "Óptima" if config['temp_min'] <= temp_actual <= config['temp_max'] else "Subóptima"
        color_condicion = "green" if condicion == "Óptima" else "orange"
        
        fig.add_annotation(
            text=f"Condición actual: {condicion}<br>Temp: {temp_actual:.1f}°C",
            xref="paper", yref="paper",
            x=0.98, y=0.98,
            showarrow=False,
            bgcolor=color_condicion,
            opacity=0.8,
            font=dict(color="white")
        )
    
    fig.update_layout(
        title="Calendario Fenológico Anual",
        xaxis_title="Mes",
        yaxis_title="",
        showlegend=False,
        height=300,
        yaxis=dict(showticklabels=False, showgrid=False),
        bargap=0.1
    )
    
    return dcc.Graph(figure=fig, config={'displayModeBar': False})

def crear_grafico_radar_cultivo(config, df_hist):
    """Crear gráfico radar comparando condiciones actuales vs óptimas"""
    
    # Calcular valores actuales
    temp_actual = df_hist['temperatura'].mean()
    humedad_actual = df_hist['humedad_relativa'].mean()
    radiacion_actual = df_hist['radiacion_solar'].mean()
    precipitacion_mensual = df_hist['precipitacion'].sum() * 30 / len(df_hist['fecha_hora'].dt.date.unique())
    viento_actual = df_hist['velocidad_viento'].mean()
    
    # Normalizar valores (0-1)
    def normalizar(valor, optimo, min_val, max_val):
        if min_val <= valor <= max_val:
            # Dentro del rango - calcular qué tan cerca está del óptimo
            distancia_optimo = abs(valor - optimo)
            rango_total = max(optimo - min_val, max_val - optimo)
            return 1 - (distancia_optimo / rango_total)
        else:
            # Fuera del rango
            if valor < min_val:
                return max(0, 1 - (min_val - valor) / min_val)
            else:
                return max(0, 1 - (valor - max_val) / max_val)
    
    # Calcular scores
    scores_actuales = [
        normalizar(temp_actual, config['temp_optima'], config['temp_min'], config['temp_max']),
        normalizar(humedad_actual, config['humedad_optima'], config['humedad_min'], config['humedad_max']),
        min(1, radiacion_actual / 600),  # Asumiendo 600 W/m² como óptimo
        min(1, precipitacion_mensual / (config['requerimiento_agua'] / 12)),
        1 - min(1, viento_actual / 30)  # Menos viento es mejor
    ]
    
    scores_optimos = [1, 1, 1, 1, 1]
    
    categorias = ['Temperatura', 'Humedad', 'Radiación', 'Precipitación', 'Viento']
    
    fig = go.Figure()
    
    # Condiciones óptimas
    fig.add_trace(go.Scatterpolar(
        r=scores_optimos,
        theta=categorias,
        fill='toself',
        fillcolor='rgba(76, 175, 80, 0.2)',
        line=dict(color='green', width=2),
        name='Óptimo'
    ))
    
    # Condiciones actuales
    fig.add_trace(go.Scatterpolar(
        r=scores_actuales,
        theta=categorias,
        fill='toself',
        fillcolor='rgba(33, 150, 243, 0.2)',
        line=dict(color='blue', width=2),
        name='Actual',
        text=[f"{s*100:.0f}%" for s in scores_actuales],
        hovertemplate='%{theta}<br>Score: %{text}<extra></extra>'
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1],
                tickformat='.0%'
            )
        ),
        showlegend=True,
        title="Condiciones Actuales vs Óptimas",
        height=400
    )
    
    return fig

def crear_grafico_balance_hidrico_cultivo(config, df_hist):
    """Crear gráfico de balance hídrico específico del cultivo"""
    
    # Agrupar por día
    df_diario = df_hist.groupby(df_hist['fecha_hora'].dt.date).agg({
        'precipitacion': 'sum',
        'evapotranspiracion': 'sum',
        'temperatura': 'mean',
        'humedad_relativa': 'mean'
    }).reset_index()
    df_diario.columns = ['fecha', 'precipitacion', 'et', 'temp', 'humedad']
    
    # Calcular ET del cultivo (ETc = ET0 * Kc)
    # Kc varía según fase fenológica
    kc_por_fase = {
        'siembra': 0.4,
        'vegetativo': 0.8,
        'floracion': 1.1,
        'fructificacion': 1.2,
        'maduracion': 0.9,
        'cosecha': 0.7
    }
    
    # Determinar Kc para cada día
    df_diario['kc'] = df_diario['fecha'].apply(
        lambda x: kc_por_fase.get(
            determinar_fase_fenologica(config, x.month), 
            0.8
        )
    )
    
    df_diario['etc'] = df_diario['et'] * df_diario['kc']
    
    # Calcular balance hídrico
    df_diario['balance'] = df_diario['precipitacion'] - df_diario['etc']
    df_diario['balance_acum'] = df_diario['balance'].cumsum()
    
    # Crear figura con subplots
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            'Precipitación vs ET del Cultivo',
            'Balance Hídrico Diario',
            'Balance Hídrico Acumulado'
        ),
        row_heights=[0.4, 0.3, 0.3]
    )
    
    # Subplot 1: Precipitación y ETc
    fig.add_trace(
        go.Bar(
            x=df_diario['fecha'],
            y=df_diario['precipitacion'],
            name='Precipitación',
            marker_color='lightblue',
            hovertemplate='Precipitación: %{y:.1f} mm<extra></extra>'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_diario['fecha'],
            y=df_diario['etc'],
            mode='lines+markers',
            name='ETc',
            line=dict(color='red', width=2),
            hovertemplate='ETc: %{y:.1f} mm<br>Kc: %{customdata:.1f}<extra></extra>',
            customdata=df_diario['kc']
        ),
        row=1, col=1
    )
    
    # Línea de umbral crítico
    fig.add_hline(
        y=config['umbral_et'],
        line_dash="dash",
        line_color="orange",
        annotation_text=f"ET crítica ({config['umbral_et']} mm/día)",
        row=1, col=1
    )
    
    # Subplot 2: Balance diario
    colores_balance = ['green' if b >= 0 else 'red' for b in df_diario['balance']]
    fig.add_trace(
        go.Bar(
            x=df_diario['fecha'],
            y=df_diario['balance'],
            name='Balance diario',
            marker_color=colores_balance,
            hovertemplate='Balance: %{y:.1f} mm<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Subplot 3: Balance acumulado
    fig.add_trace(
        go.Scatter(
            x=df_diario['fecha'],
            y=df_diario['balance_acum'],
            mode='lines+markers',
            name='Balance acumulado',
            line=dict(
                color='blue' if df_diario['balance_acum'].iloc[-1] >= 0 else 'red',
                width=3
            ),
            fill='tozeroy',
            fillcolor='rgba(0,100,200,0.2)' if df_diario['balance_acum'].iloc[-1] >= 0 else 'rgba(200,0,0,0.2)'
        ),
        row=3, col=1
    )
    
    # Línea de referencia en 0
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)
    fig.add_hline(y=0, line_dash="dash", line_color="gray", row=3, col=1)
    
    # Actualizar diseño
    fig.update_xaxes(title_text="Fecha", row=3, col=1)
    fig.update_yaxes(title_text="mm", row=1, col=1)
    fig.update_yaxes(title_text="mm", row=2, col=1)
    fig.update_yaxes(title_text="mm acum.", row=3, col=1)
    
    # Añadir anotaciones con estadísticas
    total_precip = df_diario['precipitacion'].sum()
    total_etc = df_diario['etc'].sum()
    balance_final = df_diario['balance_acum'].iloc[-1]
    
    fig.add_annotation(
        text=f"Total precipitación: {total_precip:.1f} mm<br>" +
             f"Total ETc: {total_etc:.1f} mm<br>" +
             f"Balance final: {balance_final:.1f} mm",
        xref="paper", yref="paper",
        x=0.02, y=0.98,
        showarrow=False,
        bgcolor="white",
        bordercolor="gray",
        borderwidth=1
    )
    
    fig.update_layout(
        height=800,
        showlegend=True,
        hovermode='x unified'
    )
    
    return fig

def render_analisis_riesgos_cultivo(config, df_hist):
    """Renderizar análisis de riesgos específicos del cultivo"""
    
    # Analizar diferentes tipos de riesgo
    riesgos = []
    
    # 1. Riesgo de helada
    temps_min = df_hist.groupby(df_hist['fecha_hora'].dt.date)['temperatura'].min()
    dias_helada = (temps_min <= config['temp_critica_helada']).sum()
    if dias_helada > 0:
        severidad = 'alto' if dias_helada > 5 else 'medio' if dias_helada > 2 else 'bajo'
        riesgos.append({
            'tipo': 'Helada',
            'severidad': severidad,
            'descripcion': f"{dias_helada} días con riesgo en últimos 30 días",
            'impacto': 'Daño en flores y frutos jóvenes',
            'mitigacion': 'Activar sistemas antiheladas, riego por aspersión'
        })
    
    # 2. Riesgo de estrés térmico
    temps_max = df_hist.groupby(df_hist['fecha_hora'].dt.date)['temperatura'].max()
    dias_calor = (temps_max > config['temp_max']).sum()
    if dias_calor > 0:
        severidad = 'alto' if dias_calor > 10 else 'medio' if dias_calor > 5 else 'bajo'
        riesgos.append({
            'tipo': 'Estrés Térmico',
            'severidad': severidad,
            'descripcion': f"{dias_calor} días sobre temperatura máxima",
            'impacto': 'Reducción de fotosíntesis, caída de flores',
            'mitigacion': 'Aumentar riego, mallas de sombreo'
        })
    
    # 3. Riesgo de déficit hídrico
    balance_hidrico = df_hist['precipitacion'].sum() - df_hist['evapotranspiracion'].sum()
    if balance_hidrico < -50:
        severidad = 'alto' if balance_hidrico < -100 else 'medio'
        riesgos.append({
            'tipo': 'Déficit Hídrico',
            'severidad': severidad,
            'descripcion': f"Déficit acumulado: {abs(balance_hidrico):.0f} mm",
            'impacto': 'Estrés hídrico, reducción de crecimiento',
            'mitigacion': 'Implementar riego suplementario urgente'
        })
    
        # 4. Riesgo fitosanitario
    dias_alta_humedad = (df_hist['humedad_relativa'] > 85).sum() / 24
    if dias_alta_humedad > 7:
        severidad = 'alto' if dias_alta_humedad > 14 else 'medio'
        riesgos.append({
            'tipo': 'Enfermedades Fúngicas',
            'severidad': severidad,
            'descripcion': f"{dias_alta_humedad:.0f} días con humedad >85%",
            'impacto': 'Desarrollo de hongos patógenos',
            'mitigacion': 'Aplicación preventiva de fungicidas, mejorar ventilación'
        })
    
    # 5. Riesgo por viento
    vientos_fuertes = (df_hist['velocidad_viento'] > 40).sum()
    if vientos_fuertes > 0:
        severidad = 'alto' if vientos_fuertes > 50 else 'medio' if vientos_fuertes > 20 else 'bajo'
        riesgos.append({
            'tipo': 'Daño por Viento',
            'severidad': severidad,
            'descripcion': f"{vientos_fuertes} horas con viento >40 km/h",
            'impacto': 'Daño mecánico, deshidratación',
            'mitigacion': 'Reforzar tutoraje, cortavientos'
        })
    
    if not riesgos:
        return dbc.Alert([
            html.I(className="fas fa-shield-alt me-2"),
            "No se detectaron riesgos significativos"
        ], color="success", className="mb-3")
    
    # Crear cards de riesgos
    cards_riesgos = []
    
    for riesgo in riesgos:
        color = {
            'alto': 'danger',
            'medio': 'warning',
            'bajo': 'info'
        }[riesgo['severidad']]
        
        icono = {
            'Helada': 'fas fa-snowflake',
            'Estrés Térmico': 'fas fa-temperature-high',
            'Déficit Hídrico': 'fas fa-tint-slash',
            'Enfermedades Fúngicas': 'fas fa-virus',
            'Daño por Viento': 'fas fa-wind'
        }.get(riesgo['tipo'], 'fas fa-exclamation-triangle')
        
        card = dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className=f"{icono} me-2"),
                    riesgo['tipo']
                ], className=f"bg-{color} text-white"),
                dbc.CardBody([
                    html.P([
                        html.Strong("Severidad: "),
                        html.Span(riesgo['severidad'].upper(), className=f"badge bg-{color}")
                    ]),
                    html.P(riesgo['descripcion'], className="mb-2"),
                    html.Hr(),
                    html.P([html.Strong("Impacto: "), riesgo['impacto']], className="small mb-1"),
                    html.P([html.Strong("Mitigación: "), riesgo['mitigacion']], className="small mb-0")
                ])
            ], className="h-100 mb-3")
        ], lg=6)
        
        cards_riesgos.append(card)
    
    return html.Div([
        html.H5([
            html.I(className="fas fa-exclamation-triangle me-2"),
            f"Análisis de Riesgos ({len(riesgos)} identificados)"
        ], className="mb-3"),
        dbc.Row(cards_riesgos)
    ])

def render_recomendaciones_cultivo_detalladas(rec, config, df_hist):
    """Renderizar recomendaciones detalladas para el cultivo"""
    
    # Analizar condiciones actuales
    temp_actual = df_hist['temperatura'].tail(24).mean()
    humedad_actual = df_hist['humedad_relativa'].tail(24).mean()
    et_diaria = df_hist['evapotranspiracion'].tail(24).sum()
    mes_actual = datetime.now().month
    fase_actual = determinar_fase_fenologica(config, mes_actual)
    
    # Generar recomendaciones por categoría
    recomendaciones = {
        'inmediatas': [],
        'semanales': [],
        'mensuales': []
    }
    
    # Recomendaciones por fase fenológica
    if fase_actual == 'siembra':
        recomendaciones['inmediatas'].append({
            'accion': 'Preparar terreno con análisis de suelo',
            'prioridad': 'alta',
            'icono': 'fas fa-tractor'
        })
        recomendaciones['semanales'].append({
            'accion': 'Verificar disponibilidad de plántulas/semillas',
            'prioridad': 'media',
            'icono': 'fas fa-seedling'
        })
    elif fase_actual == 'floracion':
        recomendaciones['inmediatas'].append({
            'accion': 'Monitorear polinizadores',
            'prioridad': 'alta',
            'icono': 'fas fa-bee'
        })
        recomendaciones['semanales'].append({
            'accion': 'Aplicación de boro para cuaje',
            'prioridad': 'alta',
            'icono': 'fas fa-flask'
        })
    elif fase_actual == 'cosecha':
        recomendaciones['inmediatas'].append({
            'accion': 'Evaluar madurez de frutos',
            'prioridad': 'alta',
            'icono': 'fas fa-search'
        })
        recomendaciones['semanales'].append({
            'accion': 'Coordinar logística de cosecha',
            'prioridad': 'alta',
            'icono': 'fas fa-truck'
        })
    
    # Recomendaciones por condiciones actuales
    if et_diaria > config['umbral_et']:
        recomendaciones['inmediatas'].append({
            'accion': f'Aplicar riego de {et_diaria * 1.2:.1f} mm',
            'prioridad': 'alta',
            'icono': 'fas fa-shower'
        })
    
    if temp_actual < config['temp_min']:
        recomendaciones['inmediatas'].append({
            'accion': 'Activar medidas de protección contra frío',
            'prioridad': 'alta',
            'icono': 'fas fa-shield-alt'
        })
    
    if humedad_actual > 85:
        recomendaciones['semanales'].append({
            'accion': 'Aplicación preventiva de fungicidas',
            'prioridad': 'media',
            'icono': 'fas fa-spray-can'
        })
    
    # Recomendaciones generales mensuales
    recomendaciones['mensuales'].extend([
        {
            'accion': 'Análisis foliar para ajuste nutricional',
            'prioridad': 'media',
            'icono': 'fas fa-microscope'
        },
        {
            'accion': 'Evaluación de sistema de riego',
            'prioridad': 'baja',
            'icono': 'fas fa-wrench'
        }
    ])
    
    # Si hay recomendación guardada, usar esa
    if rec is not None and pd.notna(rec['recomendacion']):
        recomendaciones['inmediatas'].insert(0, {
            'accion': rec['recomendacion'],
            'prioridad': 'alta',
            'icono': 'fas fa-star'
        })
    
    # Crear interfaz de recomendaciones
    tabs_recomendaciones = dbc.Tabs([
        dbc.Tab(
            label=f"Inmediatas ({len(recomendaciones['inmediatas'])})",
            tab_id="tab-inmediatas",
            label_style={"color": "#d32f2f"}
        ),
        dbc.Tab(
            label=f"Esta Semana ({len(recomendaciones['semanales'])})",
            tab_id="tab-semanales",
            label_style={"color": "#f57c00"}
        ),
        dbc.Tab(
            label=f"Este Mes ({len(recomendaciones['mensuales'])})",
            tab_id="tab-mensuales",
            label_style={"color": "#388e3c"}
        )
    ], id="tabs-recomendaciones", active_tab="tab-inmediatas")
    
    contenido_tabs = html.Div(id="contenido-recomendaciones")
    
    @app.callback(
        Output('contenido-recomendaciones', 'children'),
        Input('tabs-recomendaciones', 'active_tab')
    )
    def mostrar_recomendaciones_tab(active_tab):
        if active_tab == "tab-inmediatas":
            items = recomendaciones['inmediatas']
        elif active_tab == "tab-semanales":
            items = recomendaciones['semanales']
        else:
            items = recomendaciones['mensuales']
        
        if not items:
            return html.P("No hay recomendaciones en esta categoría", className="text-muted")
        
        lista = []
        for item in items:
            color_prioridad = {
                'alta': 'danger',
                'media': 'warning',
                'baja': 'success'
            }[item['prioridad']]
            
            lista.append(
                dbc.ListGroupItem([
                    html.Div([
                        html.I(className=f"{item['icono']} fa-2x me-3", 
                              style={'color': config['color_theme']}),
                        html.Div([
                            html.P(item['accion'], className="mb-1"),
                            html.Small([
                                "Prioridad: ",
                                html.Span(item['prioridad'].upper(), 
                                         className=f"badge bg-{color_prioridad}")
                            ])
                        ])
                    ], className="d-flex align-items-center")
                ])
            )
        
        return dbc.ListGroup(lista)
    
    return html.Div([
        html.H5([
            html.I(className="fas fa-clipboard-check me-2"),
            "Plan de Acción Recomendado"
        ], className="mb-3"),
        dbc.Card([
            dbc.CardBody([
                tabs_recomendaciones,
                html.Div(contenido_tabs, className="mt-3")
            ])
        ])
    ])

# ============================================================================
# FUNCIONES DE RENDERIZADO - HISTÓRICOS
# ============================================================================

def render_historicos(estacion_id, fecha_inicio, fecha_fin):
    """Renderizar datos históricos con análisis comparativo"""
    
    engine = crear_conexion_db()
    if not engine:
        return dbc.Alert("No hay conexión a la base de datos", color="warning")
    
    try:
        # Obtener datos históricos mensuales
        query_hist = """
            SELECT * FROM analisis_historicos 
            WHERE estacion_id = %(estacion_id)s
            ORDER BY año DESC, mes DESC
            LIMIT 24
        """
        
        df_hist = pd.read_sql(query_hist, engine, params={'estacion_id': estacion_id})
        
        # Obtener datos detallados para el período
        query_detalle = """
            SELECT * FROM datos_meteorologicos 
            WHERE estacion_id = %(estacion_id)s 
            AND fecha_hora BETWEEN %(fecha_inicio)s AND %(fecha_fin)s
            ORDER BY fecha_hora
        """
        
        df_detalle = pd.read_sql(
            query_detalle, 
            engine, 
            params={
                'estacion_id': estacion_id,
                'fecha_inicio': fecha_inicio,
                'fecha_fin': fecha_fin
            }
        )
        engine.dispose()
        
        if df_hist.empty and df_detalle.empty:
            return dbc.Alert("No hay datos históricos disponibles", color="info")
        
        return html.Div([
            # Resumen estadístico
            render_resumen_historico(df_hist),
            
            # Gráficos de tendencias históricas
            html.Div([
                html.H5([
                    html.I(className="fas fa-chart-line me-2"),
                    "Tendencias Climáticas Históricas"
                ], className="mb-3"),
                dcc.Graph(
                    figure=crear_grafico_tendencias_historicas(df_hist),
                    config={'displayModeBar': True, 'displaylogo': False}
                )
            ], className="graph-container"),
            
            # Comparación interanual
            html.Div([
                html.H5([
                    html.I(className="fas fa-calendar-alt me-2"),
                    "Comparación Interanual"
                ], className="mb-3"),
                dcc.Graph(
                    figure=crear_grafico_comparacion_anual(df_hist),
                    config={'displayModeBar': True, 'displaylogo': False}
                )
            ], className="graph-container"),
            
                        # Análisis de extremos
            html.Div([
                html.H5([
                    html.I(className="fas fa-exclamation-triangle me-2"),
                    "Análisis de Eventos Extremos"
                ], className="mb-3"),
                render_analisis_extremos(df_hist, df_detalle)
            ], className="graph-container"),
            
            # Matriz de correlaciones
            html.Div([
                html.H5([
                    html.I(className="fas fa-project-diagram me-2"),
                    "Correlaciones entre Variables"
                ], className="mb-3"),
                dcc.Graph(
                    figure=crear_matriz_correlaciones(df_detalle),
                    config={'displayModeBar': True, 'displaylogo': False}
                )
            ], className="graph-container")
        ])
        
    except Exception as e:
        return dbc.Alert(f"Error cargando históricos: {str(e)}", color="danger")

def render_resumen_historico(df_hist):
    """Renderizar resumen de datos históricos"""
    
    if df_hist.empty:
        return None
    
    # Calcular estadísticas
    temp_promedio_anual = df_hist['temp_promedio'].mean()
    temp_max_historica = df_hist['temp_max_promedio'].max()
    temp_min_historica = df_hist['temp_min_promedio'].min()
    precip_anual_promedio = df_hist.groupby('año')['precipitacion_total'].sum().mean()
    total_dias_helada = df_hist['dias_helada'].sum()
    
    # Identificar tendencias
    años = df_hist['año'].unique()
    if len(años) > 1:
        temps_anuales = df_hist.groupby('año')['temp_promedio'].mean()
        tendencia_temp = "ascendente" if temps_anuales.iloc[-1] > temps_anuales.iloc[0] else "descendente"
    else:
        tendencia_temp = "sin datos suficientes"
    
    return dbc.Card([
        dbc.CardHeader([
            html.I(className="fas fa-chart-bar me-2"),
            "Resumen Climático Histórico"
        ]),
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    html.Div([
                        html.H6("Temperatura", className="text-muted"),
                        html.H4(f"{temp_promedio_anual:.1f}°C", className="mb-0"),
                        html.Small("Promedio histórico"),
                        html.Hr(),
                        html.P([
                            html.Strong("Máx. histórica: "),
                            f"{temp_max_historica:.1f}°C"
                        ], className="mb-1"),
                        html.P([
                            html.Strong("Mín. histórica: "),
                            f"{temp_min_historica:.1f}°C"
                        ], className="mb-1"),
                        html.P([
                            html.Strong("Tendencia: "),
                            html.Span(
                                tendencia_temp,
                                className=f"badge bg-{'danger' if tendencia_temp == 'ascendente' else 'info'}"
                            )
                        ])
                    ])
                ], md=3),
                
                dbc.Col([
                    html.Div([
                        html.H6("Precipitación", className="text-muted"),
                        html.H4(f"{precip_anual_promedio:.0f} mm", className="mb-0"),
                        html.Small("Promedio anual"),
                        html.Hr(),
                        html.P([
                            html.Strong("Mes más lluvioso: "),
                            calendar.month_name[
                                df_hist.groupby('mes')['precipitacion_total'].sum().idxmax()
                            ]
                        ], className="mb-1"),
                        html.P([
                            html.Strong("Mes más seco: "),
                            calendar.month_name[
                                df_hist.groupby('mes')['precipitacion_total'].sum().idxmin()
                            ]
                        ])
                    ])
                ], md=3),
                
                dbc.Col([
                    html.Div([
                        html.H6("Eventos Extremos", className="text-muted"),
                        html.H4(f"{total_dias_helada}", className="mb-0"),
                        html.Small("Días con helada (total)"),
                        html.Hr(),
                        html.P([
                            html.Strong("Promedio anual: "),
                            f"{total_dias_helada / len(años):.1f} días"
                        ], className="mb-1"),
                        html.P([
                            html.Strong("Meses críticos: "),
                            ", ".join([
                                calendar.month_name[m][:3] 
                                for m in df_hist[df_hist['dias_helada'] > 0]['mes'].unique()
                            ])
                        ])
                    ])
                ], md=3),
                
                dbc.Col([
                    html.Div([
                        html.H6("Balance Hídrico", className="text-muted"),
                        html.H4(
                            f"{df_hist['evapotranspiracion_total'].sum():.0f} mm",
                            className="mb-0"
                        ),
                        html.Small("ET total histórica"),
                        html.Hr(),
                        html.P([
                            html.Strong("Humedad promedio: "),
                            f"{df_hist['humedad_promedio'].mean():.0f}%"
                        ], className="mb-1"),
                        html.P([
                            html.Strong("Días lluvia/año: "),
                            f"{df_hist.groupby('año')['dias_lluvia'].sum().mean():.0f}"
                        ])
                    ])
                ], md=3)
            ])
        ])
    ], className="mb-4")

def crear_grafico_tendencias_historicas(df_hist):
    """Crear gráfico de tendencias históricas con regresión"""
    
    # Preparar datos
    df_hist['fecha'] = pd.to_datetime(
        df_hist[['año', 'mes']].assign(day=1).rename(
            columns={'año': 'year', 'mes': 'month'}
        )
    )
    df_hist_sorted = df_hist.sort_values('fecha')
    
    # Crear figura con subplots
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            'Temperatura Media Mensual',
            'Precipitación Mensual',
            'Días con Eventos Extremos'
        )
    )
    
    # 1. Temperatura con tendencia
    fig.add_trace(
        go.Scatter(
            x=df_hist_sorted['fecha'],
            y=df_hist_sorted['temp_promedio'],
            mode='lines+markers',
            name='Temp. Promedio',
            line=dict(color=COLORES_GRAFICOS['temperatura'], width=2)
        ),
        row=1, col=1
    )
    
    # Añadir línea de tendencia
    z = np.polyfit(range(len(df_hist_sorted)), df_hist_sorted['temp_promedio'], 1)
    p = np.poly1d(z)
    fig.add_trace(
        go.Scatter(
            x=df_hist_sorted['fecha'],
            y=p(range(len(df_hist_sorted))),
            mode='lines',
            name='Tendencia',
            line=dict(color='red', width=2, dash='dash')
        ),
        row=1, col=1
    )
    
    # Banda de máximas y mínimas
    fig.add_trace(
        go.Scatter(
            x=df_hist_sorted['fecha'],
            y=df_hist_sorted['temp_max_promedio'],
            mode='lines',
            name='Máx. Promedio',
            line=dict(color=COLORES_GRAFICOS['temperatura_max'], width=1),
            showlegend=False
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_hist_sorted['fecha'],
            y=df_hist_sorted['temp_min_promedio'],
            mode='lines',
            name='Mín. Promedio',
            line=dict(color=COLORES_GRAFICOS['temperatura_min'], width=1),
            fill='tonexty',
            fillcolor='rgba(100,100,200,0.1)',
            showlegend=False
        ),
        row=1, col=1
    )
    
    # 2. Precipitación
    fig.add_trace(
        go.Bar(
            x=df_hist_sorted['fecha'],
            y=df_hist_sorted['precipitacion_total'],
            name='Precipitación',
            marker_color=df_hist_sorted['precipitacion_total'].apply(
                lambda x: get_color_precipitacion(x/30)  # Aproximar a diario
            )
        ),
        row=2, col=1
    )
    
    # Media móvil de precipitación
    precip_ma = df_hist_sorted['precipitacion_total'].rolling(window=3, center=True).mean()
    fig.add_trace(
        go.Scatter(
            x=df_hist_sorted['fecha'],
            y=precip_ma,
            mode='lines',
            name='Media móvil (3 meses)',
            line=dict(color='darkblue', width=2)
        ),
        row=2, col=1
    )
    
    # 3. Eventos extremos
    fig.add_trace(
        go.Bar(
            x=df_hist_sorted['fecha'],
            y=df_hist_sorted['dias_helada'],
            name='Días con helada',
            marker_color='lightblue'
        ),
        row=3, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=df_hist_sorted['fecha'],
            y=df_hist_sorted['dias_lluvia'],
            name='Días con lluvia',
            marker_color='darkgreen',
            opacity=0.6
        ),
        row=3, col=1
    )
    
    # Actualizar diseño
    fig.update_xaxes(title_text="Fecha", row=3, col=1)
    fig.update_yaxes(title_text="°C", row=1, col=1)
    fig.update_yaxes(title_text="mm", row=2, col=1)
    fig.update_yaxes(title_text="Días", row=3, col=1)
    
    # Añadir anotación con cambio de temperatura
    if len(df_hist_sorted) > 1:
        cambio_temp = (df_hist_sorted['temp_promedio'].iloc[-1] - 
                      df_hist_sorted['temp_promedio'].iloc[0])
        fig.add_annotation(
            text=f"Cambio temperatura: {cambio_temp:+.2f}°C",
            xref="paper", yref="paper",
            x=0.02, y=0.98,
            showarrow=False,
            bgcolor="white",
            bordercolor="gray"
        )
    
    fig.update_layout(
        height=800,
        showlegend=True,
        hovermode='x unified'
    )
    
    return fig

def crear_grafico_comparacion_anual(df_hist):
    """Crear gráfico de comparación entre años"""
    
    # Preparar datos por año y mes
    df_pivot = df_hist.pivot_table(
        values='temp_promedio',
        index='mes',
        columns='año',
        aggfunc='mean'
    )
    
    fig = go.Figure()
    
    # Añadir una línea por cada año
    colors = px.colors.qualitative.Set3
    for i, año in enumerate(df_pivot.columns):
        fig.add_trace(
            go.Scatter(
                x=[calendar.month_name[m][:3] for m in df_pivot.index],
                y=df_pivot[año],
                mode='lines+markers',
                name=str(año),
                line=dict(width=2, color=colors[i % len(colors)]),
                marker=dict(size=6)
            )
        )
    
    # Añadir promedio histórico
    promedio_mensual = df_hist.groupby('mes')['temp_promedio'].mean()
    fig.add_trace(
        go.Scatter(
            x=[calendar.month_name[m][:3] for m in promedio_mensual.index],
            y=promedio_mensual.values,
            mode='lines',
            name='Promedio Histórico',
            line=dict(width=4, color='black', dash='dash')
        )
    )
    
    fig.update_layout(
        title="Comparación de Temperatura Media por Año",
        xaxis_title="Mes",
        yaxis_title="Temperatura (°C)",
        hovermode='x unified',
        height=500
    )
    
    return fig

def render_analisis_extremos(df_hist, df_detalle):
    """Renderizar análisis de eventos extremos"""
    
    # Identificar eventos extremos en el período
    eventos = []
    
    if not df_detalle.empty:
        # Temperaturas extremas
        temp_p95 = df_detalle['temperatura'].quantile(0.95)
        temp_p5 = df_detalle['temperatura'].quantile(0.05)
        
        dias_muy_calurosos = df_detalle[df_detalle['temperatura'] > temp_p95]
        if len(dias_muy_calurosos) > 0:
            eventos.append({
                'tipo': 'Calor Extremo',
                'cantidad': len(dias_muy_calurosos),
                'descripcion': f"Temperatura > {temp_p95:.1f}°C (percentil 95)",
                'fechas': dias_muy_calurosos['fecha_hora'].dt.date.unique()[:5]
            })
        
        dias_muy_frios = df_detalle[df_detalle['temperatura'] < temp_p5]
        if len(dias_muy_frios) > 0:
            eventos.append({
                'tipo': 'Frío Extremo',
                'cantidad': len(dias_muy_frios),
                'descripcion': f"Temperatura < {temp_p5:.1f}°C (percentil 5)",
                'fechas': dias_muy_frios['fecha_hora'].dt.date.unique()[:5]
            })
        
        # Precipitación intensa
        precip_diaria = df_detalle.groupby(df_detalle['fecha_hora'].dt.date)['precipitacion'].sum()
        dias_lluvia_intensa = precip_diaria[precip_diaria > 25]
        if len(dias_lluvia_intensa) > 0:
            eventos.append({
                'tipo': 'Lluvia Intensa',
                'cantidad': len(dias_lluvia_intensa),
                                'descripcion': "Precipitación > 25mm/día",
                'fechas': dias_lluvia_intensa.index[:5]
            })
        
        # Vientos fuertes
        vientos_fuertes = df_detalle[df_detalle['rafaga_viento'] > 50]
        if len(vientos_fuertes) > 0:
            eventos.append({
                'tipo': 'Viento Fuerte',
                'cantidad': len(vientos_fuertes),
                'descripcion': "Ráfagas > 50 km/h",
                'fechas': vientos_fuertes['fecha_hora'].dt.date.unique()[:5]
            })
    
    if not eventos:
        return dbc.Alert("No se detectaron eventos extremos en el período", color="info")
    
    # Crear cards de eventos
    cards_eventos = []
    for evento in eventos:
        card = dbc.Card([
            dbc.CardHeader([
                html.I(className="fas fa-exclamation-circle me-2"),
                evento['tipo']
            ]),
            dbc.CardBody([
                html.H4(f"{evento['cantidad']} eventos", className="text-center mb-3"),
                html.P(evento['descripcion']),
                html.Hr(),
                html.P("Fechas recientes:", className="mb-1"),
                html.Small(", ".join([f.strftime('%d/%m') for f in evento['fechas']]))
            ])
        ], className="mb-3")
        cards_eventos.append(dbc.Col(card, md=6))
    
    return dbc.Row(cards_eventos)

def crear_matriz_correlaciones(df_detalle):
    """Crear matriz de correlaciones entre variables meteorológicas"""
    
    if df_detalle.empty:
        return go.Figure().add_annotation(
            text="No hay datos suficientes",
            xref="paper", yref="paper",
            x=0.5, y=0.5,
            showarrow=False
        )
    
    # Seleccionar variables para correlación
    variables = [
        'temperatura', 'humedad_relativa', 'precipitacion',
        'velocidad_viento', 'radiacion_solar', 'evapotranspiracion',
        'presion_atmosferica'
    ]
    
    # Filtrar solo columnas disponibles
    variables_disponibles = [v for v in variables if v in df_detalle.columns]
    df_corr = df_detalle[variables_disponibles].corr()
    
    # Crear heatmap
    fig = go.Figure(data=go.Heatmap(
        z=df_corr.values,
        x=df_corr.columns,
        y=df_corr.columns,
        colorscale='RdBu',
        zmid=0,
        text=np.round(df_corr.values, 2),
        texttemplate='%{text}',
        textfont={"size": 10},
        hoverongaps=False
    ))
    
    fig.update_layout(
        title="Matriz de Correlaciones entre Variables",
        height=600,
        xaxis={'side': 'bottom'},
        yaxis={'side': 'left'}
    )
    
    return fig

# ============================================================================
# FUNCIONES DE RENDERIZADO - MAPA
# ============================================================================

def render_mapa():
    """Renderizar mapa interactivo con todas las estaciones"""
    
    # Crear mapa base
    m = folium.Map(
        location=[QUILLOTA_CONFIG['centro']['lat'], QUILLOTA_CONFIG['centro']['lon']],
        zoom_start=11,
        tiles='OpenStreetMap'
    )
    
    # Añadir capa de satélite
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Satélite',
        overlay=False,
        control=True
    ).add_to(m)
    
    # Obtener datos actuales de cada estación
    engine = crear_conexion_db()
    datos_estaciones = {}
    
    if engine:
        for estacion in QUILLOTA_CONFIG['estaciones']:
            query = """
                SELECT * FROM datos_meteorologicos
                WHERE estacion_id = %(estacion_id)s
                ORDER BY fecha_hora DESC
                LIMIT 1
            """
            df = pd.read_sql(query, engine, params={'estacion_id': estacion['id']})
            if not df.empty:
                datos_estaciones[estacion['id']] = df.iloc[0]
        engine.dispose()
    
    # Añadir marcadores para cada estación
    for estacion in QUILLOTA_CONFIG['estaciones']:
        datos = datos_estaciones.get(estacion['id'])
        
        if datos is not None:
            # Crear contenido del popup con datos actuales
            popup_html = f"""
            <div style="width: 300px;">
                <h4>{estacion['nombre']}</h4>
                <hr>
                <table style="width: 100%;">
                    <tr>
                        <td><b>🌡️ Temperatura:</b></td>
                        <td>{datos['temperatura']:.1f}°C</td>
                    </tr>
                    <tr>
                        <td><b>💧 Humedad:</b></td>
                        <td>{datos['humedad_relativa']:.0f}%</td>
                    </tr>
                    <tr>
                        <td><b>🌧️ Precipitación:</b></td>
                        <td>{datos['precipitacion']:.1f} mm</td>
                    </tr>
                    <tr>
                        <td><b>💨 Viento:</b></td>
                        <td>{datos['velocidad_viento']:.1f} km/h</td>
                    </tr>
                    <tr>
                        <td><b>☀️ Radiación:</b></td>
                        <td>{datos['radiacion_solar']:.0f} W/m²</td>
                    </tr>
                </table>
                <hr>
                <small>Actualizado: {datos['fecha_hora'].strftime('%d/%m %H:%M')}</small>
            </div>
            """
            
            # Determinar color del marcador según temperatura
            if datos['temperatura'] < 10:
                color = 'blue'
            elif datos['temperatura'] < 20:
                color = 'green'
            elif datos['temperatura'] < 30:
                color = 'orange'
            else:
                color = 'red'
        else:
            popup_html = f"<b>{estacion['nombre']}</b><br>Sin datos disponibles"
            color = 'gray'
        
        # Añadir marcador
        folium.Marker(
            location=[estacion['lat'], estacion['lon']],
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=estacion['nombre'],
            icon=folium.Icon(color=color, icon='info-sign')
        ).add_to(m)
    
    # Añadir círculos de influencia
    for estacion in QUILLOTA_CONFIG['estaciones']:
        folium.Circle(
            location=[estacion['lat'], estacion['lon']],
            radius=5000,  # 5 km
            popup=f"Área de influencia - {estacion['nombre']}",
            color='blue',
            fill=True,
            fillOpacity=0.1
        ).add_to(m)
    
    # Añadir control de capas
    folium.LayerControl().add_to(m)
    
    # Añadir minimapa
    minimap = plugins.MiniMap(toggle_display=True)
    m.add_child(minimap)
    
    # Convertir a HTML
    map_html = m._repr_html_()
    
    return html.Div([
        html.H4([
            html.I(className="fas fa-map-marked-alt me-2"),
            "Mapa de Estaciones Meteorológicas"
        ]),
        html.Iframe(
            srcDoc=map_html,
            width='100%',
            height='600px',
            style={'border': 'none', 'borderRadius': '10px'}
        ),
        dbc.Alert([
            html.I(className="fas fa-info-circle me-2"),
            "Los colores de los marcadores indican la temperatura actual: ",
            html.Span("🔵 Frío (<10°C) ", style={'color': 'blue'}),
            html.Span("🟢 Templado (10-20°C) ", style={'color': 'green'}),
            html.Span("🟠 Cálido (20-30°C) ", style={'color': 'orange'}),
            html.Span("🔴 Caluroso (>30°C)", style={'color': 'red'})
        ], color="info", className="mt-3")
    ])

# ============================================================================
# FUNCIONES DE RENDERIZADO - ANÁLISIS AVANZADO
# ============================================================================

def render_analisis_avanzado(estacion_id, cultivo, fecha_inicio, fecha_fin):
    """Renderizar análisis avanzado con modelos predictivos"""
    
    return html.Div([
        html.H4([
            html.I(className="fas fa-brain me-2"),
            "Análisis Avanzado - En Desarrollo"
        ]),
        dbc.Alert(
            "Esta sección incluirá análisis predictivos avanzados, modelos de machine learning y simulaciones.",
            color="info"
        )
    ])

# ============================================================================
# CALLBACKS PARA MODALES Y NOTIFICACIONES
# ============================================================================

@app.callback(
    [Output("modal-export", "is_open"),
     Output("notification-toast", "is_open"),
     Output("notification-toast", "header"),
     Output("notification-toast", "children")],
    [Input("btn-export", "n_clicks"),
     Input("close-export", "n_clicks"),
     Input("confirm-export", "n_clicks")],
    [State("modal-export", "is_open"),
     State("export-format", "value"),
     State("export-options", "value")],
    prevent_initial_call=True
)
def toggle_modal_export(n1, n2, n3, is_open, format_export, options):
    ctx = dash.callback_context
    
    if not ctx.triggered:
        return is_open, False, "", ""
    
    button_id = ctx.triggered[0]["prop_id"].split(".")[0]
    
    if button_id == "btn-export":
        return True, False, "", ""
    elif button_id == "close-export":
        return False, False, "", ""
    elif button_id == "confirm-export":
        # Aquí iría la lógica de exportación
        return False, True, "Exportación Exitosa", f"Datos exportados en formato {format_export}"
    
    return is_open, False, "", ""

# ============================================================================
# FUNCIONES AUXILIARES PARA ACTUALIZACIÓN AUTOMÁTICA
# ============================================================================

def actualizar_datos_automatico():
    """Función para actualización automática de datos"""
    print(f"\n🔄 Actualizando datos automáticamente - {datetime.now()}")
    # Aquí iría la lógica de actualización
    pass

# ============================================================================
# EJECUTAR APLICACIÓN
# ============================================================================

if __name__ == '__main__':
    print("\n" + "="*60)
    print("🚀 INICIANDO DASHBOARD MIP QUILLOTA v2.0")
    print("="*60)
    print(f"📅 Fecha: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
    print(f"🖥️  Sistema: Dashboard Meteorológico y Agrícola")
    print(f"📍 Ubicación: Quillota, Chile")
    print("="*60)
    
    # Verificar conexión a base de datos
    engine = crear_conexion_db()
    if engine:
        print("✅ Conexión a base de datos establecida")
        engine.dispose()
    else:
        print("⚠️  ADVERTENCIA: No se pudo conectar a la base de datos")
    
    print("\n📌 OPCIONES DE ACCESO:")
    print("   • Local: http://localhost:8050")
    print("   • Red: http://[IP_LOCAL]:8050")
    print("\n🛑 Para detener: Presione Ctrl+C")
    print("="*60 + "\n")
    
    # Programar actualización automática
    def run_schedule():
        while True:
            schedule.run_pending()
            time.sleep(60)
    
    # Crear thread para actualizaciones
    schedule.every(30).minutes.do(actualizar_datos_automatico)
    update_thread = threading.Thread(target=run_schedule, daemon=True)
    update_thread.start()
    
    # Ejecutar servidor
    app.run_server(
        debug=True,
        host='127.0.0.1',  # Usar localhost para evitar el error de Windows
        port=8050,
        dev_tools_hot_reload=True
    )


🚀 INICIANDO DASHBOARD MIP QUILLOTA v2.0
📅 Fecha: 25/07/2025 00:15:18
🖥️  Sistema: Dashboard Meteorológico y Agrícola
📍 Ubicación: Quillota, Chile
✅ Conexión a base de datos establecida

📌 OPCIONES DE ACCESO:
   • Local: http://localhost:8050
   • Red: http://[IP_LOCAL]:8050

🛑 Para detener: Presione Ctrl+C



📊 Columnas disponibles en df_24h: ['id', 'estacion', 'fecha_hora', 'temperatura', 'temperatura_max', 'temperatura_min', 'humedad', 'precipitacion', 'velocidad_viento', 'direccion_viento', 'presion', 'radiacion_solar', 'created_at', 'estacion_id', 'estacion_nombre', 'temperatura_aparente', 'humedad_relativa', 'precipitacion_probabilidad', 'rafaga_viento', 'presion_atmosferica', 'evapotranspiracion', 'indice_uv', 'punto_rocio', 'gradiente']
📊 Primeras filas del DataFrame:
      id estacion          fecha_hora  temperatura  temperatura_max  \
0  16010     None 2025-08-09 03:00:00      11.2515          11.9515   
1   8282     None 2025-08-09 03:00:00       9.9205          10.2205   
2  11594     None 2025-08-09 03:00:00       9.9205          10.2205   
3  13802     None 2025-08-09 03:00:00      11.2515          11.9515   
4  18218     None 2025-08-09 03:00:00      11.2515          11.9515   

   temperatura_min humedad  precipitacion  velocidad_viento  direccion_viento  \
0        11.20150

00:18:01 - cmdstanpy - INFO - Chain [1] start processing


✅ 2208 registros guardados en datos_meteorologicos

📊 Generando pronóstico integrado para La Cruz-La Cantera
🔄 Entrenando modelo para temperatura...


00:18:02 - cmdstanpy - INFO - Chain [1] done processing
00:18:02 - cmdstanpy - INFO - Chain [1] start processing


...

✅ Pronóstico completado para precipitacion
🔄 Entrenando modelo para velocidad_viento...


00:18:52 - cmdstanpy - INFO - Chain [1] done processing


✅ Pronóstico completado para velocidad_viento
🔄 Entrenando modelo para presion_atmosferica...


00:18:52 - cmdstanpy - INFO - Chain [1] start processing
00:18:52 - cmdstanpy - INFO - Chain [1] done processing


✅ Pronóstico completado para presion_atmosferica
✅ 10536 pronósticos guardados
✅ 1 alertas guardadas
✅ Actualización completada
📊 Columnas disponibles en df_24h: ['id', 'estacion', 'fecha_hora', 'temperatura', 'temperatura_max', 'temperatura_min', 'humedad', 'precipitacion', 'velocidad_viento', 'direccion_viento', 'presion', 'radiacion_solar', 'created_at', 'estacion_id', 'estacion_nombre', 'temperatura_aparente', 'humedad_relativa', 'precipitacion_probabilidad', 'rafaga_viento', 'presion_atmosferica', 'evapotranspiracion', 'indice_uv', 'punto_rocio', 'gradiente']
📊 Primeras filas del DataFrame:
      id estacion          fecha_hora  temperatura  temperatura_max  \
0  20426     None 2025-08-09 03:00:00      11.2515          11.9515   
1   1658     None 2025-08-09 03:00:00       9.9205          10.2205   
2  22634     None 2025-08-09 03:00:00      11.2515          11.9515   
3  16010     None 2025-08-09 03:00:00      11.2515          11.9515   
4  11594     None 2025-08-09 03:00:00     